In [ ]:
!df -h

In [1]:
!cat .env

STAGE=ENGIE_STAGING


In [2]:
Company.objects.all()

<QuerySet [<Company: c85df77c-3e92-4f60-b9c1-de5866400d2d: PRE-GSO - UHE-CANA BRAVA>, <Company: 33e4f910-db35-46ce-95c8-56cad68eeb94: PRE-GSO - UHE-ESTREITO>, <Company: ed749424-27e4-4342-82cd-f924de71830b: PRE-Limbo - UTPS>, <Company: e9fce210-f652-4d7d-befe-32e02a8f0f60: PRE-GSO - UHE-ITA>, <Company: 1b1511d1-6f28-4aed-b02c-7c820c5f9dbb: PRE-GSO - PCH-RONDONÓPOLIS>, <Company: 62ad80d9-b785-4037-933a-e23238728220: PRE-GSO - UHE-MIRANDA>, <Company: 37b4c80d-3f2b-4032-8cac-6ba1560d3144: PRE-GSO - UHE-SALTO SANTIAGO>, <Company: 1021dc11-d12c-4a27-88d5-dde0fad2be7d: PRE-GSO - UHE-PONTE DE PEDRA>, <Company: 5f0ef84c-9985-4519-95de-e87fb4fcea21: PRE-GSO - PCH-JOSÉ GELÁZIO DA ROCHA>, <Company: 571581e3-da3a-469e-8676-0dc58c056ef7: PRE-GSO - UHE-SALTO OSORIO>, <Company: f0bdd035-6af9-401c-9233-26c5cdd808b0: PRE-GSO - UHE-JAGUARA>, <Company: cff59760-69dd-4d07-bba5-61aa8b6faeea: PRE-GES - UHE-CANA BRAVA>, <Company: 42ddc389-3258-4daf-868a-097235427e0f: PRE-GES - UHE-ESTREITO>, <Company: 4a5e01

In [ ]:
UserInCompany.objects.filter(user__username='rlcs')

In [94]:
from helpers.strings import get_obj_from_path
from django.db.models import Q
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd

In [ ]:
ui = UserInCompany.objects.filter(permissions=UserPermission.objects.get(name="DEFESA CIVIL")).order_by('company').distinct()
dados=[]
for a in ui:
    print(a.user.full_name,'|',a.user.username,'|',a.user.email,'|',a.company.name)


In [ ]:
def translate_operational_control(record,item='name'):
    company_name = record.company.name if record.company else None
    company_custom = record.company.custom_options['operationalControl']['fields']['kind']['selectOptions']['options']
    operational_position = next(
                option[item]
                for option in company_custom
                if option['value'] == record.operational_control.kind
            )
    return operational_position

In [ ]:
record = OccurrenceRecord.objects.get(uuid="00121d62-42d3-42fe-a729-f5ea4913082f")
record.__dict__

In [ ]:
company=Company.objects.filter(Q(name__startswith='UHE') | Q(name__startswith='PCH'))
company

In [ ]:
for a in tqdm(company):
    print(a.name)
    for b in OccurrenceType.objects.filter(occurrence_kind='701',company=a).order_by('name'):
        print('\t',b.name)
        for c in OccurrenceRecord.objects.filter(occurrence_type=b,company=a,form_data__popular_name__isnull=False).values('form_data__popular_name','form_data__matrix_serial','form_data__family').order_by('form_data__popular_name'):
#             print('\t','\t',c.form_data['popular_name'])
            for d in b.form_fields['fields']:
                if d['apiName'] == 'family':
                    print(option['name'] for option in d['selectOptions']['options'] if option['value'] == c['form_data__family'])
            print('\t','\t',c['form_data__matrix_serial'],c['form_data__popular_name'])

In [ ]:
OccurrenceRecord.objects.filter(occurrence_type__name='Cadastro de Matriz Vegetal',company__name='UHE-ITA',form_data__popular_name='Acca sellowiana').count()

In [ ]:
occ[0].__dict__

In [ ]:
for a in occ:
    for b in a.company.all():
        print(b)
    print(a.name)

In [ ]:
Company._meta.get_fields()

In [ ]:
empresa = Company.objects.filter(name__icontains="ponte de pedra").values('name','subcompany__firm').annotate(registro=Count('occurrencerecord'))
empresa

In [ ]:
test= OccurrenceRecord.objects.values('company__name').annotate(total_registro=Count('uuid'))
test

In [ ]:
for a in test:
    print(a['company__name'], a['total_registro'])

In [ ]:
ats=ApprovalTransition.objects.filter(origin__approval_flow__company__name="UHE-ITA",
                                      origin__approval_flow__target_model__icontains='access',
                                     origin__name='Aguardando Aprovação do Usuário-Chave do Sistema')

In [ ]:
for a in ats:
    print(a.origin,a.destination)

In [ ]:
ats=ApprovalTransition.objects.filter(origin__approval_flow__company__name="UHE-ITA",
                                      origin__approval_flow__target_model__icontains='record')

In [ ]:
for a in ats:
    print(a.name,a.origin,' > ',a.destination,'\nCONDIÇÃO:',a.condition,'\nBUTTON: ',a.button,'\nCALLBACK: ',a.callback)
    print('-----------------------------------------------')

In [ ]:
import pandas as pd

In [ ]:
occ = OccurrenceType.objects.all().values('form_fields__fields')

In [ ]:
lista = [len(a['form_fields__fields']) if a['form_fields__fields'] is not None else 0 for a in occ]

In [ ]:
from collections import Counter

count = dict(Counter(lista))

In [ ]:
df2=pd.DataFrame(data=count,index=['a']).to_excel('RelProdutoEngie.xlsx')

In [ ]:
ore = OccurrenceRecord.objects.get(number__icontains='4267',company__name="UHE-ITA")

In [ ]:
company=Company.objects.filter(name__startswith="GES").order_by('name')
company[0]

In [ ]:
def trad_kind(name,company):
    options=company.custom_options['operationalControl']['fields']['kind']['selectOptions']['options']
    try:
        valor = [a['value'] for a in options if a['name'] == name]
        return valor[0]
    except:
        if not OperationalControl.objects.filter(kind='709',firm__company=company).exists():
            options.append({'name': 'Cadastro de espécies','value':'709'})
            company.save()
            oc = OperationalControl(kind='709',
                                   firm=Firm.objects.get(name__icontains='Meio Ambiente',company=company),
                                   responsible = Firm.objects.get(name__icontains='Meio Ambiente',company=company).manager)
            oc.save()
            return '709'
    else:
        print('ja existe OperationalControl')

In [ ]:
from openpyxl import load_workbook

wb = load_workbook(filename='MATRIZES - Dados Matrizes Completo (Anexo) (1).xlsx')
sheet = wb.active
data = []

headers = [cell.value for cell in sheet[1]]

for row in sheet.iter_rows(min_row=2):
    row_dict = {}
    for key, cell in zip(headers, row):
        row_dict[key] = cell.value

    data.append(row_dict)

In [ ]:
company=Company.objects.filter(name__startswith="GES").order_by('name').exclude(name='GES - UHE-PONTE DE PEDRA')
c = company[9]
c

In [ ]:
user=User.objects.get(username='engie')
kind=trad_kind("Cadastro de espécies",c)

operational_control=OperationalControl.objects.get(firm__company=c,kind=kind)

occ_cadastros = OccurrenceType.objects.get(uuid='c16b1d6b-856c-4ff7-ae2f-f68d5223a077',company=c) #Cadastros Especies

c,operational_control,kind,occ_cadastros

In [ ]:
erros=[]
objects=[]
counter=0
for index,planilha in tqdm(enumerate(data)):
    if planilha['Nome Cientifíco'] is not None:
        occRecords = OccurrenceRecord.objects.filter(occurrence_type=occ_cadastros,company=c,
                                                     form_data__species__icontains=planilha['Nome Cientifíco'].split(' ')[0],
                                                     form_data__popular_name__icontains=planilha['Nome Comum:'].split(' ')[0].split('-')[0]).first()
        if (occRecords == None or occRecords == '') and planilha['Nome Comum:'] not in erros:
            try:
                properties=[]
                properties.append({})
                objects.append(OccurrenceRecord(company=c,
                                      occurrence_type=occ_cadastros,
                                      operational_control=operational_control,
                                      created_by=user,
                                      properties=properties,
                                      datetime='2023-8-7',
                                      form_data={'kingdom': '2',
                                                 'family': planilha['Família'],
                                                  'species': planilha['Nome Cientifíco'],
                                                  'popular_name': planilha['Nome Comum:']}))
                erros.append(planilha['Nome Comum:'])
                erros.append(planilha['Nome Cientifíco'])
                counter+=1
            except Exception as e:
                print(e,occRecords,planilha['Nº Matriz'],planilha['Nome Comum:'],planilha['Nome Cientifíco'],index+2)
            

In [ ]:
len(objects)

In [ ]:
for a in tqdm(objects):
    a.save()

In [ ]:
ocon = OccurrenceRecord.objects.filter(occurrence_type=occ_cadastros).values('company__name').annotate(total=Count('uuid'))
ocon

In [ ]:
# OccurrenceRecord.objects.filter(company='GES - UHE-CANA BRAVA',occurrence_type=occ_cadastros,created_by=user,created_at__gte='2023-8-7').update(operational_control=operational_control)

In [ ]:
occs = OccurrenceRecord.objects.filter(occurrence_type__uuid='f200613b-8862-4a24-8c16-d6babd215fd7').values('company__name').annotate(total=Count('uuid'))

In [ ]:
occs

In [ ]:
from django.core.exceptions import ObjectDoesNotExist
dados=[]
counter=0
for i,a in enumerate(data):
    dado={}
    try:
        occ = OccurrenceRecord.objects.filter(occurrence_type__uuid='f200613b-8862-4a24-8c16-d6babd215fd7',
                                          form_data__matrix_size = a['Altura da Árvore (Metros)'],
                                          form_data__matrix_dap = int(a['DAP (Centímetros)']))
        if occ!=None:
            if occ.count() > 1:
                for o in occ:
                    counter_unico=0
                    dado[o.company.name] = counter_unico+1
                    dados.append(dado)
            else:
                counter+=1
                dado['GES - UHE-ITA'] = counter
                
        else:
            print(i+2,a['Altura da Árvore (Metros)'],a['DAP (Centímetros)'])
        dados.append(dado)
    except ObjectDoesNotExist:
        pass

In [ ]:
dados

In [ ]:
data

In [ ]:
for i,a in enumerate(data):
    dado={}
    try:
        occs=OccurrenceRecord.objects.filter(occurrence_type__uuid='f200613b-8862-4a24-8c16-d6babd215fd7',
                                        form_data__matrix_size = a['Altura da Árvore (Metros)'],
                                          form_data__matrix_dap = int(a['DAP (Centímetros)'])).values('company__name').annotate(total=Count('uuid'))
        print(occs)
    except:
        pass

In [ ]:
engie=UserInCompany.objects.filter(user__username='engie', company__name__startswith='GES')
for a in engie:
    print(a.permissions.name)

In [ ]:
engie=UserInCompany.objects.filter(user__username='engie', company__name__startswith='GES')
for a in engie:
    if a.permissions.name.lower()=='colaborador externo':
        a.permissions=UserPermission.objects.get(companies=a.company, name='ADMINISTRADOR')
        a.save()

In [ ]:
from django.db.models import Count, Q
import pandas as pd

In [ ]:
def generate_excel(lista_df,name=''):   
    
    writer = pd.ExcelWriter(name+'engieRel.xlsx', engine='xlsxwriter')
    for i,a in enumerate(lista_df):
    # Write each DataFrame to a different worksheet
        a.to_excel(writer, sheet_name=name or str(i))
    writer.save()

In [ ]:
lista_df=[]

### Registros (sociofamiliar, edificação de uso coletivo, edificação sem habilitação, etc.): unidade + número de registros para aquela unidade, para o que está dentro da ZAS e o que está na ZSS, separado

In [ ]:
zas=OccurrenceRecord.objects.filter(company__name__startswith='GSO').values('company__name').annotate(total_zas=Count('place_on_dam', filter=Q(place_on_dam='ZAS')))
zss=OccurrenceRecord.objects.filter(company__name__startswith='GSO').values('company__name').annotate(total_zss=Count('place_on_dam', filter=Q(place_on_dam='ZSS')))
geral_s_zssEZas=OccurrenceRecord.objects.filter(company__name__startswith="GES").exclude(Q(place_on_dam='ZAS') | Q(place_on_dam='ZSS')).values('company__name').annotate(total_registro=Count('uuid'))

In [ ]:
dfZas=pd.DataFrame(data=zas)
dfZss=pd.DataFrame(data=zss)
dfGeral=pd.DataFrame(data=geral_s_zssEZas)

In [ ]:
lista_df.append(dfZas)
lista_df.append(dfZss)
lista_df.append(dfGeral)

In [ ]:
generate_excel(lista_df,'')
lista_df=[]

### Composição por Idade (faixa etária e número de pessoas por faixa)

In [ ]:
faixa_etaria=OccurrenceRecord.objects.filter(company__name__startswith='GSO',
                                             form_data__has_key='age').values('company__name',
                                                                              'form_data__age').annotate(qtd_por_faixa=Count('form_data__age'))

In [ ]:
dfFaixa=pd.DataFrame(data=faixa_etaria)

In [ ]:
lista_df.append(dfFaixa)

In [ ]:
generate_excel(lista_df,'faixa_etaria')
lista_df=[]

### Residentes Fixos e de Mobilidade Reduzida (com descrição do motivo, como gestação ou obesidade) 

In [ ]:
faixa_etaria=OccurrenceRecord.objects.filter(company__name__startswith='GSO',
                                             form_data__has_key='age').values('company__name',
                                                                              'form_data__age').annotate(qtd_por_faixa=Count('form_data__age'))

In [1]:
for a in Company.objects.all():
    print(a.name, a.metadata['company_prefix'])

POC-Projeto APR APR
GSO - UHE-SALTO OSORIO UHSO
GSO - UTPS UTPS
GSB - UHE-MACHADINHO UHMA
GSB - UHE-SALTO OSORIO UHSO
GSB - UHE-SALTO SANTIAGO UHSS
GSO - UHE-JAGUARA UHJA
TESTE UHIT
GSO - UHE-CANA BRAVA UHCB
GSO - UHE-MIRANDA UHMI
GSO - UHE-SALTO SANTIAGO UHSS
GES - UHE-CANA BRAVA UHCB
Engie Umburanas Engie
Energia Compartilhado GSO UHE
GSO - UHE-MACHADINHO UHMA
GSO - PCH-RONDONÓPOLIS PHRO
GSB - UHE-ITA UHIT
GSB - UHE-ESTREITO UHET
GSO - UHE-ITA UHIT
GSB - UHE-JAGUARA UHJA
GSB - UHE-MIRANDA UHMI
GSO - PCH-JOSÉ GELÁZIO DA ROCHA PHJG
GSO - UHE-ESTREITO UHET
GSB - UHE-PONTE DE PEDRA UHPP
GSB - UHE-SÃO SALVADOR UHSA
GSB - UHE-CANA BRAVA UHCB
GSO - UHE-PASSO FUNDO UHPF
GES - UHE-ESTREITO UHET
Energia Compartilhado UHE
GSB - UHE-PASSO FUNDO UHPF
GSO - UHE-SÃO SALVADOR UHSA
RECURSOS HIDRICOS RH
GSO - UHE-PONTE DE PEDRA UHPP
GES - PCH-RONDONÓPOLIS PHRO
GES - UHE-MACHADINHO UHMA
Limbo - UTPS UTPS
GES - UHE-SALTO SANTIAGO UHSS
GES - PCH-JOSÉ GELÁZIO DA ROCHA PHJG
GES - UHE-SÃO SALVADOR UHSA
GES 

In [ ]:
st=SearchTag.objects.filter(name='Ambiental',company__name='GES - UHE-ITA')
st.count(),st[0].uuid

In [ ]:
co=Company.objects.get(name='PRE-RECURSOS HIDRICOS')

In [ ]:
def trad_basin(value,company):
    options=company.custom_options['company']['fields']['hidroBasins']['selectOptions']['options']
    valor = [a['name'] for a in options if a['value'] == value]
    return valor[0]

In [ ]:
trad_basin(4,co)

In [ ]:
uuid="20b32c2f-23e7-4ade-a428-95e0194958d8"
ocrteste=OccurrenceRecord.objects.filter(uuid=uuid)
ocrteste[0].__dict__

In [ ]:
ocr = OccurrenceRecord.objects.filter(company__name='PRE-RECURSOS HIDRICOS',form_data__has_key='hidro_basin')
ocr[0].__dict__

In [ ]:
for a in ocr:
    print(trad_basin(a.form_data['hidro_basin'],co))

In [ ]:
ot=OccurrenceType.objects.get(name='Ponto de monitoramento')

In [ ]:
ot.__dict__

In [ ]:
co

In [1]:
pe=list(OccurrenceRecord.objects.filter(company__name__startswith="GSO", 
                                occurrence_type__name='Ponto de encontro').values('company__name','occurrence_type__name').annotate(total=Count('uuid')))

In [2]:
rf=list(OccurrenceRecord.objects.filter(company__name__startswith="GSO", 
                                occurrence_type__name='Sinalização da rota de fuga').values('company__name','occurrence_type__name').annotate(total=Count('uuid')))

In [3]:
import pandas as pd
df = pd.DataFrame(data=[{"Rotas de Fuga": a['total']} for a in rf],index=[a['company__name'] for a in rf])
df2 = pd.DataFrame(data=[{"Pontos de Encontro": a['total']} for a in pe],index=[a['company__name'] for a in pe])

In [4]:
df

,Rotas de Fuga
GSO - PCH-JOSÉ GELÁZIO DA ROCHA,1
GSO - PCH-RONDONÓPOLIS,1
GSO - UHE-CANA BRAVA,19
GSO - UHE-ITA,94
GSO - UHE-JAGUARA,42
GSO - UHE-MACHADINHO,24
GSO - UHE-MIRANDA,57
GSO - UHE-PASSO FUNDO,24
GSO - UHE-PONTE DE PEDRA,51
GSO - UHE-SALTO OSORIO,9


In [5]:
df2

,Pontos de Encontro
GSO - PCH-JOSÉ GELÁZIO DA ROCHA,1
GSO - PCH-RONDONÓPOLIS,1
GSO - UHE-CANA BRAVA,9
GSO - UHE-ITA,36
GSO - UHE-JAGUARA,23
GSO - UHE-MACHADINHO,19
GSO - UHE-MIRANDA,26
GSO - UHE-PASSO FUNDO,21
GSO - UHE-PONTE DE PEDRA,12
GSO - UHE-SALTO OSORIO,9


In [5]:
stot=SearchTagOccurrenceType.objects.filter(company__name='GES - UHE-ITA',search_tags__name='Requisição')
stot

<QuerySet [<SearchTagOccurrenceType: GES - UHE-ITA - Requisições>]>

In [6]:
stot[0].occurrence_type

<OccurrenceType: Requisições - ['GES - UHE-SALTO SANTIAGO', 'GES - UHE-ITA', 'GES - UHE-ESTREITO', 'GES - UHE-JAGUARA', 'GES - UHE-MIRANDA', 'GES - UHE-SALTO OSORIO', 'GES - UHE-CANA BRAVA', 'GES - UHE-SÃO SALVADOR', 'GES - UHE-PASSO FUNDO', 'GES - UHE-MACHADINHO', 'GES - UHE-PONTE DE PEDRA', 'GES - PCH-JOSÉ GELÁZIO DA ROCHA', 'GES - PCH-RONDONÓPOLIS', 'GES - GEH']>

In [7]:
import re

def convert_string(string: str) -> str:
    if '_' in string:
        # Convert snake_case to camelCase
        return re.sub(r'_([a-z])', lambda x: x.group(1).upper(), string)
    else:
        # Convert camelCase to snake_case
        return re.sub(r'([A-Z])', lambda x: '_' + x.group(1).lower(), string)

In [4]:
company=Company.objects.get(name='GES - UHE-ESTREITO')

In [3]:
occ = OccurrenceType.objects.get(company=company,name='Edificação grupo familiar')
occ.uuid

UUID('29668d37-4ece-41a2-a98e-bf4efc3bb696')

In [12]:
ocr=OccurrenceRecord.objects.filter(occurrence_type=occ2)
ocr.count()

7715

In [9]:
ocr[0]

<OccurrenceRecord: [GSO - UHE-MACHADINHO] - UHMA-RG-2022.0061 - 2022-01-17 22:03:49.440000+00:00>

In [8]:
for a in occ.form_fields['fields']:
    if a.get("apiName"):
        print(f'"{convert_string(a["apiName"])}": a["{a["displayName"]}"]{a["dataType"]}')
    else:
        print(f'"{convert_string(a["api_name"])}": a["{a["display_name"]}"]{a["data_type"]}')
    if a.get('dataType') == 'arrayOfObjects':
        for b in a['innerFields']:
            print(f'\t"{convert_string(b["apiName"])}": a["{b["displayName"]}"]{b["dataType"]}')
    if a.get('data_type') == 'arrayOfObjects':
        for b in a['inner_fields']:
            print(f'\t"{convert_string(b["api_name"])}": a["{b["display_name"]}"]{b["data_type"]}')

"restriction": a["Restrições"]arrayOfObjects
	"registration_prohibited": a["Existem restrições?"]select
	"reason": a["Motivo"]select
	"other_reason": a["Outros - Descrição"]string
	"date": a["Data da tentativa"]timestamp
"main_contact_name": a["Nome do contato em caso de emergência"]string
"main_contact_phone": a["Tel. p/ cadastro no sistema de alerta"]float
"conditions": a["Condição"]select
"other_conditions": a["Outro(a)"]number
"property_area": a["Área total declarada do imóvel"]float
"area_unity": a["Unidade"]select
"other_areas": a["Outra unidade"]string
"prop_services": a["Prestação de serviços na unidade levantada "]textArea
"prop_production": a["Produção de bens na unidade levantada"]textArea
"prop_institutional_services": a["Prestação de serviços institucionais "]textArea
"prop_cultures": a["Produção agrícola"]arrayOfObjects
	"is_permanent": a["É permanente?"]boolean
	"culture": a["Cultura"]string
	"culture_area": a["Área (hectares)"]float
"prop_animal": a["Produção pecuária"]

In [17]:
occ2 = OccurrenceType.objects.get(company=company,name='Uso e Ocupação Irregular')
occ2.form_fields

{'id': 11,
 'name': 'ocupacaoIrregular',
 'fields': [{'id': 1,
   'logic': {'method': [{'map': [{'var': 'formData.propertyIntersections'},
       {'cat': [{'var': 'accumulator'},
         'Obra: ',
         {'var': 'attributes.oBRA'},
         '; Sequencial: ',
         {'var': 'attributes.sEQUENCIAL'},
         '; Identificador: ',
         {'var': 'attributes.iDENTIFICA'}]}]},
     'join',
     ['\n']]},
   'apiName': 'propertyIntersections',
   'dataType': 'jsonLogic',
   'fullWidth': True,
   'multiline': True,
   'displayName': 'Propriedades envolvidas'},
  {'id': 4,
   'apiName': 'irregularities',
   'dataType': 'arrayOfObjects',
   'displayName': '',
   'innerFields': [{'order': 1,
     'apiName': 'placeId',
     'dataType': 'select',
     'displayName': 'Identificação do Local',
     'selectOptions': {'options': [{'name': 'Área remanescente', 'value': '1'},
       {'name': 'Área de preservação permanente', 'value': '2'},
       {'name': 'Faixa de oscilação', 'value': '3'},
    

In [11]:
for a in occ2.form_fields['fields']:
    if a.get("apiName"):
        print(f'"{convert_string(a["apiName"])}": a["{a["displayName"]}"]{a["dataType"]}')
    else:
        print(f'"{convert_string(a["api_name"])}": a["{a["display_name"]}"]{a["data_type"]}')
    if a.get('dataType') == 'arrayOfObjects':
        for b in a['innerFields']:
            print(f'\t"{convert_string(b["apiName"])}": a["{b["displayName"]}"]{b["dataType"]}')
    if a.get('data_type') == 'arrayOfObjects':
        for b in a['inner_fields']:
            print(f'\t"{convert_string(b["api_name"])}": a["{b["display_name"]}"]{b["data_type"]}')

"property_intersections": a["Propriedades envolvidas"]jsonLogic
"irregularities": a[""]arrayOfObjects
	"place_id": a["Identificação do Local"]select
	"irregularity": a["Irregularidade"]selectMultiple
	"display_length": a["Comprimento (m)"]boolean
	"length": a["Comprimento (m)"]float
	"display_width": a["Largura (m)"]boolean
	"width": a["Largura (m)"]float
	"display_height": a["Altura (m)"]boolean
	"height": a["Altura (m)"]float
	"display_area": a["Área (m2)"]boolean
	"area": a["Área (m2)"]float
	"display_amount": a["Quantidade (un)"]boolean
	"amount": a["Quantidade (un)"]float
	"display_volume": a["Volume (m3)"]boolean
	"volume": a["Volume (m3)"]float
	"description": a["Descrição"]textArea
"people": a[""]arrayOfObjects
	"condition": a["Condição da Parte Envolvida"]select
	"full_name": a["Nome Completo"]string
	"display_r_g": a["RG"]boolean
	"rg": a["RG"]string
	"display_c_p_f": a["CPF"]boolean
	"cpf": a["CPF"]string
	"display_phone": a["Telefone"]boolean
	"phone": a["Telefone"]string
	

In [16]:
for a in occ2.form_fields['fields']:
    print(a['displayName'])
    
    if a.get('dataType') == 'arrayOfObjects':
        for b in a['innerFields']:
            print(b['displayName'])

Propriedades envolvidas

Identificação do Local
Irregularidade
Comprimento (m)
Comprimento (m)
Largura (m)
Largura (m)
Altura (m)
Altura (m)
Área (m2)
Área (m2)
Quantidade (un)
Quantidade (un)
Volume (m3)
Volume (m3)
Descrição

Condição da Parte Envolvida
Nome Completo
RG
RG
CPF
CPF
Telefone
Telefone
E-mail
E-mail
Logradouro
Número
Bairro
Complemento
UF
Cidade
CEP
Houve Acordo Verbal?
Prazo de Cumprimento
Acordo realizado com
Área Total Afetada
Observações
Medidas a Providenciar


In [7]:
ocr=OccurrenceRecord.objects.get(uuid='732fcfba-2d0d-4cba-9de8-189db62c123c')

In [19]:
ocr.__dict__

{'_state': <django.db.models.base.ModelState at 0x7f591f1e5e10>,
 'uuid': UUID('732fcfba-2d0d-4cba-9de8-189db62c123c'),
 'datetime': datetime.datetime(2023, 3, 16, 3, 0, tzinfo=<UTC>),
 'number': 'UHET-RG-2023.0210',
 'created_at': datetime.datetime(2023, 3, 16, 20, 9, 1, 835160, tzinfo=<UTC>),
 'updated_at': datetime.datetime(2023, 3, 16, 20, 9, 2, 291801, tzinfo=<UTC>),
 'reviews': 0,
 'company_id': UUID('42ddc389-3258-4daf-868a-097235427e0f'),
 'editable': True,
 'is_approved': False,
 'uf_code': None,
 'city_id': None,
 'location_id': None,
 'place_on_dam': '',
 'river_id': None,
 'point': None,
 'geometry': <GeometryCollection object at 0x7f591eceeac0>,
 'properties': [{}],
 'distance_from_dam': 0.0,
 'other_reference': 'Próximo à (RM 11)-Palmeiras-TO.',
 'origin': None,
 'origin_media': None,
 'informer': {},
 'territorial_administration': '',
 'created_by_id': UUID('0aa50773-b368-4a50-9f12-4a7d8dfaf256'),
 'search_tag_description': '',
 'occurrence_type_id': UUID('f200613b-8862-

In [1]:
occs=OccurrenceType.objects.all().distinct()

In [2]:
for a in occs:
    for b in a.form_fields['fields']:
        if b.get('displayName') == "Elemento":
            print(b.get('displayName'),b.get('apiName'),a.name)
        if b.get('display_name') == "Elemento":
            print(b.get('display_name'),b.get('api_name'),a.name)

Elemento affectedMaterials Deslizamento de Taludes Marginais
Elemento affectedAnimals Deslizamento de Taludes Marginais
Elemento affectedWhater Deslizamento de Taludes Marginais
Elemento affectedSoil Deslizamento de Taludes Marginais
Elemento affectedMaterials Ocorrências Gerais [NOVO]
Elemento affectedAnimals Ocorrências Gerais [NOVO]
Elemento affectedWhater Ocorrências Gerais [NOVO]
Elemento affectedSoil Ocorrências Gerais [NOVO]
Elemento affectedMaterials Acidente em Área Sob Gestão do Empreendimento
Elemento affectedAnimals Acidente em Área Sob Gestão do Empreendimento
Elemento affectedWhater Acidente em Área Sob Gestão do Empreendimento
Elemento affectedSoil Acidente em Área Sob Gestão do Empreendimento
Elemento affectedMaterials Proliferação de Plantas Aquáticas
Elemento affectedAnimals Proliferação de Plantas Aquáticas
Elemento affectedWhater Proliferação de Plantas Aquáticas
Elemento affectedSoil Proliferação de Plantas Aquáticas


In [41]:
telefone,cpf=0,0
for a in occs:
    for b in a.form_fields['fields']:
        if b.get('displayName') == "Telefone" or b.get('displayName') == "TELEFONE":
            print(b.get('displayName'),b.get('dataType'),a.name)
            telefone+=1
        if b.get('display_name') == "Telefone" or b.get('display_name') == "TELEFONE":
            print(b.get('display_name'),b.get('dataType'),a.name)
            telefone+=1
        if b.get('displayName') == "cpf" or b.get('displayName') == "CPF":
            print(b.get('displayName'),b.get('dataType'),a.name)
            cpf+=1
        if b.get('display_name') == "cpf" or b.get('display_name') == "CPF":
            print(b.get('display_name'),b.get('dataType'),a.name)
            cpf+=1

Telefone string Guia de Fornecimento de Mudas
Telefone textArea Agendamento
CPF string Cadastro
Telefone boolean Cadastro
CPF float Ficha descrita da estação remota
Telefone float Ficha descrita da estação remota
CPF float Ficha descrita da estação remota
Telefone float Ficha descrita da estação remota
Telefone textArea Agendamento
Telefone string Visita Técnica a Parte Interessada
Telefone string Requisição de Mudas
CPF string Cadastro da parte interessada
Telefone boolean Cadastro da parte interessada
Telefone textArea Agendamento
Telefone textArea Agendamento


In [42]:
telefone,cpf

(11, 4)

In [108]:
# data = list(SearchTag.objects.filter(company__name="GES - UHE-ITA").values('name','l))
data2 = SearchTag.objects.filter(company__name="GES - UHE-ITA")

In [105]:
data2[0].parent_tags.all()[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f2fa547e790>,
 'uuid': UUID('07e71ad1-112a-4900-abd9-636f85729c75'),
 'company_id': UUID('d436bd3c-e145-44f2-a5ee-797a67110efd'),
 'name': 'Ocorrência',
 'kind': 'select_option',
 'level': 1,
 'description': 'São formulários para registrar: ocorrências, anomalias e fatos identificados por equipes habilitadas.',
 'redirect': ''}

In [116]:
data_export=[]
for a in data2:
    datas={}
    datas['name'] = a.name
    datas['level'] = a.level
    datas['parents'] = []
    for b in a.parent_tags.all():
        datas['parents'].append({'name':b.name,'level':b.level})
    data_export.append(datas)

In [117]:
import json
with open('dataST_comp.json', 'w') as f:
    json.dump(data_export, f)

In [8]:
import json
with open('data.json', 'w') as f:
    json.dump(data, f)

In [30]:
import uuid

def is_valid_uuid(uuid_str):
    try:
        uuid.UUID(uuid_str)
        return True
    except ValueError:
        return False

In [61]:
# data = list(SearchTagOccurrenceType.objects.filter(company__name="GES - UHE-ITA").values())
data = list(OccurrenceType.objects.filter(company__name="GES - UHE-ITA").values())

In [52]:
for a in data:
    a['uuid'] = str(a['uuid'])
    a['previous_version_id'] = None
    a['monitoring_plan_id'] = str(a['monitoring_plan_id'])
    a['created_at'] = None
    a['updated_at'] = None
    a['created_by_id'] = None

In [54]:
import json
with open('dataOccurrenceType.json', 'w') as f:
    json.dump(data, f)

In [60]:
data[1]

{'uuid': '040500ff-cea2-4d11-9cb6-830792b1d62f',
 'name': 'Agendamento',
 'occurrence_kind': '706',
 'form_fields': {'id': 51,
  'name': 'agendament',
  'fields': [{'id': 1,
    'apiName': 'records',
    'dataType': 'occurrenceRecordDialog',
    'displayName': 'Instituições e parceiros',
    'selectOptions': {'reference': {'filter': '{"occurrence_type": "71702b2c-0d0c-41c1-b5c4-243a40785967"}',
      'resource': 'OccurrenceRecord',
      'filterApp': {'relationships.occurrenceType.data.id': '71702b2c-0d0c-41c1-b5c4-243a40785967'},
      'optionText': 'formData.intitutionName',
      'optionValue': 'uuid'}}},
   {'id': 2,
    'apiName': 'responsibleName',
    'dataType': 'textArea',
    'displayName': 'Nome'},
   {'id': 3,
    'apiName': 'responsibleEmail',
    'dataType': 'textArea',
    'displayName': 'Email'},
   {'id': 4,
    'apiName': 'responsiblePhone',
    'dataType': 'textArea',
    'displayName': 'Telefone'},
   {'id': 5,
    'apiName': 'responsibleFunction',
    'dataType': '

In [80]:
SearchTagOccurrenceType._meta.get_fields()

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: company>,
 <django.db.models.fields.related.ForeignKey: occurrence_type>,
 <django.db.models.fields.related.ManyToManyField: search_tags>)

In [90]:
dataST = SearchTagOccurrenceType.objects.filter(company__name="GES - UHE-ITA")

In [95]:
dataST[0].search_tags.all(),dataST[0].__dict__

(<QuerySet [<SearchTag: GES - UHE-ITA - 2 - select_option - Resumo executivo>, <SearchTag: GES - UHE-ITA - 1 - select_option - Relatório>]>,
 {'_state': <django.db.models.base.ModelState at 0x7f2fa50eb250>,
  'uuid': UUID('6624d177-5efe-4894-b666-a49a5194591d'),
  'company_id': UUID('d436bd3c-e145-44f2-a5ee-797a67110efd'),
  'occurrence_type_id': UUID('df7b367d-893b-40cc-af39-2019a2e764b9')})

In [119]:
data_export=[]
for a in dataST:
    datas={}
    datas['occ_name'] = a.occurrence_type.name
    datas['parents'] = []
    for b in a.search_tags.all():
        datas['parents'].append({'name':b.name,'level':b.level})
    data_export.append(datas)

In [120]:
import json
with open('dataSearchTagOccurrenceType_comp.json', 'w') as f:
    json.dump(data_export, f)

In [68]:
dataFirm = list(Firm.objects.values())

In [69]:
dataFirm

[{'uuid': UUID('f76b8aa4-71fa-438a-9f4d-4a1c53461d5b'),
  'name': 'Equipe Itiquira',
  'manager_id': UUID('0aa50773-b368-4a50-9f12-4a7d8dfaf256'),
  'company_id': UUID('1021dc11-d12c-4a27-88d5-dde0fad2be7d'),
  'subcompany_id': None,
  'entity_id': None,
  'cnpj': '',
  'is_company_team': False,
  'active': True,
  'logo': '',
  'custom_options': {},
  'metadata': {},
  'members_amount': 0,
  'street_address': '',
  'city_id': None,
  'can_use_ecm_integration': False,
  'created_by_id': None},
 {'uuid': UUID('6e4a34e4-fbaa-4953-8bb2-cd91b5a68f7f'),
  'name': 'Cadastro de rotas de fuga',
  'manager_id': UUID('0aa50773-b368-4a50-9f12-4a7d8dfaf256'),
  'company_id': UUID('1021dc11-d12c-4a27-88d5-dde0fad2be7d'),
  'subcompany_id': None,
  'entity_id': None,
  'cnpj': '',
  'is_company_team': False,
  'active': True,
  'logo': '',
  'custom_options': {},
  'metadata': {},
  'members_amount': 0,
  'street_address': '',
  'city_id': None,
  'can_use_ecm_integration': False,
  'created_by_id':

In [1]:
dataOC = OperationalControl.objects.filter(company__name="GES - UHE-ITA")

FieldError: Cannot resolve keyword 'company' into field. Choices are: config_occurrence_types, contract, contract_id, control_material_items, control_op_records, files, firm, firm_id, integrations_reading, kind, map_default_filters, metadata, op_control_records, operational_control_cycles, responsible, responsible_id, show_map, show_materials, uuid

In [25]:
river=River.objects.filter(company__name="GES - UHE-ITA")
river[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f360ffa0850>,
 'uuid': UUID('90d3fb69-5f5a-49b6-8125-fc14e113cddf'),
 'company_id': UUID('d436bd3c-e145-44f2-a5ee-797a67110efd'),
 'name': 'Rio dos Queimados'}

In [26]:
riverData=[]
for a in river:
    riverData.append({"name": a.name})

In [27]:
riverData

[{'name': 'Rio dos Queimados'},
 {'name': 'Rio Fragosos'},
 {'name': 'Arroio Lambedor'},
 {'name': 'Rio Palomas'},
 {'name': 'Rio Ligeiro'},
 {'name': 'Rio Novo'},
 {'name': 'Rio Engano'},
 {'name': 'Rio Dourado'},
 {'name': 'Rio Jacutinga'},
 {'name': 'Rio Rancho Grande'},
 {'name': 'Rio Suzana'},
 {'name': 'Lajeado Teixeira Soares'},
 {'name': 'Rio Suruvi'},
 {'name': 'Rio do Peixe'},
 {'name': 'Rio Uruguai'},
 {'name': 'Rio Mirim'}]

In [28]:
import json
with open('dataRiver.json', 'w') as f:
    json.dump(riverData, f)

# LOCALIDADE

In [31]:
local=Location.objects.filter(company__name="GES - UHE-ITA")
local[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f360efa5490>,
 'uuid': UUID('d14ef1b5-c3f7-48ce-a70b-e341c8ae7537'),
 'company_id': UUID('d436bd3c-e145-44f2-a5ee-797a67110efd'),
 'city_id': UUID('a243c7ae-8891-4c2e-aa53-c166efc80684'),
 'name': 'Distrito Uruguai',
 'coordinates': None}

In [38]:
localData=[]
for a in local:
    localData.append({"name": a.name, "city":a.city.name})

In [39]:
localData

[{'name': 'Distrito Uruguai', 'city': 'Piratuba'},
 {'name': 'Linha Terra Vermelha', 'city': 'Concórdia'},
 {'name': 'Linha Engenho Velho', 'city': 'Concórdia'},
 {'name': 'Linha Meneghetti', 'city': 'Concórdia'},
 {'name': 'Praia Bonita', 'city': 'Mariano Moro'},
 {'name': 'Mirim', 'city': 'Severiano de Almeida'},
 {'name': 'Linha Várzea', 'city': 'Mariano Moro'},
 {'name': 'Linha Rancho Grande', 'city': 'Concórdia'},
 {'name': 'Linha Entre Rios', 'city': 'Alto Bela Vista'},
 {'name': 'Linha Suzana', 'city': 'Marcelino Ramos'},
 {'name': 'Volta do Uvá', 'city': 'Aratiba'},
 {'name': 'Bigarela', 'city': 'Mariano Moro'},
 {'name': 'Barra do Suzana', 'city': 'Marcelino Ramos'},
 {'name': 'Vitalina', 'city': 'Mariano Moro'},
 {'name': 'Encruzilhada da Várzea', 'city': 'Aratiba'},
 {'name': 'Polonesa', 'city': 'Aratiba'},
 {'name': 'Porto Lucas', 'city': 'Aratiba'},
 {'name': 'Linha Pinheiro Preto', 'city': 'Concórdia'},
 {'name': 'Pinca Baixa', 'city': 'Aratiba'},
 {'name': 'Tamanduá', 'c

In [40]:
with open('dataLocalidade.json', 'w') as f:
    json.dump(localData, f)

In [5]:
ocr=OccurrenceRecord.objects.get(uuid="0f90b0a4-090f-447a-992c-506bdbfeceb6")
ocr

<OccurrenceRecord: [GES - UHE-ESTREITO] - UHET-RG-2023.1045 - 2023-08-22 13:36:44.328000+00:00>

In [6]:
ocr.__dict__

{'_state': <django.db.models.base.ModelState at 0x7f4aedbbb650>,
 'uuid': UUID('0f90b0a4-090f-447a-992c-506bdbfeceb6'),
 'datetime': datetime.datetime(2023, 8, 22, 13, 36, 44, 328000, tzinfo=<UTC>),
 'number': 'UHET-RG-2023.1045',
 'created_at': datetime.datetime(2023, 8, 22, 13, 46, 7, 640255, tzinfo=<UTC>),
 'updated_at': datetime.datetime(2023, 8, 22, 13, 46, 7, 640272, tzinfo=<UTC>),
 'reviews': 0,
 'company_id': UUID('42ddc389-3258-4daf-868a-097235427e0f'),
 'editable': True,
 'is_approved': False,
 'uf_code': 0,
 'city_id': None,
 'location_id': None,
 'place_on_dam': '',
 'river_id': None,
 'point': None,
 'geometry': None,
 'properties': [],
 'distance_from_dam': 0.0,
 'other_reference': '',
 'origin': '',
 'origin_media': '',
 'informer': {'mail': '',
  'phone': '',
  'mobile': '',
  'last_name': '',
  'first_name': '',
  'qualification': ''},
 'territorial_administration': '',
 'created_by_id': UUID('0aa50773-b368-4a50-9f12-4a7d8dfaf256'),
 'search_tag_description': '',
 'occ

In [12]:
ocr.monitoring_plan_id

UUID('f3356e23-0b49-4dd7-81e5-450d6187b9c7')

In [2]:
UserInCompany.objects.filter(user__username='engie')

<QuerySet [<UserInCompany: [1021dc11-d12c-4a27-88d5-dde0fad2be7d: GSO - UHE-PONTE DE PEDRA] engie>, <UserInCompany: [110bce05-5146-421e-a5a7-ffb32ee92a91: GSO - UHE-PASSO FUNDO] engie>, <UserInCompany: [17663a30-5f71-4e7c-8ac2-adf472342b42: GES - UHE-JAGUARA] engie>, <UserInCompany: [1b1511d1-6f28-4aed-b02c-7c820c5f9dbb: GSO - PCH-RONDONÓPOLIS] engie>, <UserInCompany: [1f30f493-c653-46c7-b073-0046837347f7: GSO - UHE-SÃO SALVADOR] engie>, <UserInCompany: [24072e01-a4a7-4779-94a3-7e45e937cce8: GSB - UHE-CANA BRAVA] engie>, <UserInCompany: [247c952a-9371-483d-8039-9a65b41b8db3: GSB - UHE-SÃO SALVADOR] engie>, <UserInCompany: [296edae1-b154-40ff-87b1-3b78e3441906: GES - UHE-PONTE DE PEDRA] engie>, <UserInCompany: [2fd0ca76-b741-47d7-af9c-607e6ed1eca2: GES - UHE-MIRANDA] engie>, <UserInCompany: [3167c3b2-95fa-4ecb-b759-dc1d63067ba4: GES - UHE-SALTO OSORIO] engie>, <UserInCompany: [33e4f910-db35-46ce-95c8-56cad68eeb94: GSO - UHE-ESTREITO] engie>, <UserInCompany: [37b4c80d-3f2b-4032-8cac-6ba1

In [5]:
from tqdm.notebook import tqdm
for a in tqdm(UserInCompany.objects.filter(user__username='engie')):
    a.uuid=None
    a.user = User.objects.get(username='karla.zorzam')
    a.save()

  0%|          | 0/40 [00:00<?, ?it/s]

In [6]:
Procedure.objects.first().__dict__,Procedure.objects.first().firm.company.name

({'_state': <django.db.models.base.ModelState at 0x7f80e694a450>,
  'uuid': UUID('d54fd3dc-0297-4087-809a-e2f1b0192973'),
  'action_id': UUID('80b7421f-7047-40e4-a2d4-3ab077d1ff61'),
  'occurrence_kind': '',
  'occurrence_type_id': None,
  'service_order_action_status_id': UUID('33ccc2d4-a195-49db-a6e5-ad5edfce0e09'),
  'firm_id': UUID('91d79c63-b050-4902-862a-767c88127785'),
  'responsible_id': UUID('a1fd9da3-1c5f-4b68-bf6d-5076263e21b0'),
  'procedure_previous_id': None,
  'form_data': {},
  'to_do': 'Emitir os pareceres e efetuar a medição',
  'created_by_id': UUID('31c72676-8481-4d13-b7aa-bc52b5f77abb'),
  'created_at': datetime.datetime(2023, 9, 19, 18, 10, 24, 286690, tzinfo=<UTC>),
  'deadline': datetime.datetime(2024, 3, 29, 18, 10, tzinfo=<UTC>),
  'done_at': datetime.datetime(2023, 9, 19, 18, 10, 0, 795000, tzinfo=<UTC>),
  '_fieldsignals_originals': {(140192164256336,
    140191655024400): {'uuid': UUID('d54fd3dc-0297-4087-809a-e2f1b0192973'), 'action': UUID('80b7421f-7047-4

In [24]:
ac=AccessRequest.objects.get(uuid='c1576f77-f4ac-453d-8f08-8226ef0c0a00')

In [25]:
ac.approval_step_id = ApprovalStep.objects.get(name='Aprovado',
                                               approval_flow__target_model__icontains='access',
                                              approval_flow__company=ac.company).uuid

In [26]:
ac.approved = True
ac.done = True

In [27]:
ac.save()

In [28]:
UserInCompany.objects.create(user=ac.user,company=ac.company,permissions=ac.permissions)

<UserInCompany: [3167c3b2-95fa-4ecb-b759-dc1d63067ba4: GES - UHE-SALTO OSORIO] paulo.feitosa>

In [8]:
mb = MeasurementBulletin.objects.get(number='UHIT-BM-2023.0058')
approval_step=ApprovalStep.objects.get(approval_flow__company__name='GES - UHE-ITA',
                                       approval_flow__target_model__icontains='measurement',
                                      name__icontains='Aguardando Aprovação do Preposto da Contratante')
mb.approval_step,approval_step

(<ApprovalStep: [GES - UHE-ITA]: Aprovado>,
 <ApprovalStep: [GES - UHE-ITA]: Aguardando Aprovação do Preposto da Contratante>)

In [9]:
mb.approval_step = approval_step
mb.save()

In [7]:
mb.history.all()[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x7fd33d363810>,
 'uuid': UUID('f386d49f-ecf6-4c22-8a7d-18d46e8685d1'),
 'number': 'UHJA-BM-2023.0016',
 'creation_date': datetime.datetime(2023, 9, 11, 14, 4, 1, 150080, tzinfo=<UTC>),
 'measurement_date': datetime.datetime(2023, 9, 11, 14, 3, 12, 516000, tzinfo=<UTC>),
 'total_price': 24216.35,
 'extra_info': {'accounting_classification': '100-615011121013-01244071-1202027-1103-0-0'},
 'description': 'Atividades realizadas durante a campanha de agosto de 2023.',
 'editable': False,
 'period_starts_at': None,
 'period_ends_at': None,
 'work_day': None,
 'firm_id': UUID('070cca28-d954-4aa2-bc79-be7b91d0b4d3'),
 'firm_manager_id': UUID('bdb70b20-4e78-4d6b-91dd-714438f3a5a8'),
 'created_by_id': UUID('fdd8761a-0f32-47da-8122-e23db29c0ae2'),
 'contract_id': UUID('7e0932f6-112e-44b4-b287-9f1422ad9265'),
 'approval_step_id': UUID('616451d9-c3d1-4819-b0f8-4b3264dff21a'),
 'history_id': UUID('d35edb66-cc7c-4caf-a6ed-2e61e6c50ee6'),
 'history_date

In [10]:
for a in mb.history.all():
    print(a.approval_step.name, a.history_date.strftime('%d-%m-%Y'))

Aguardando Aprovação do Preposto da Contratante 11-09-2023
Aguardando Aprovação do Terceiro 11-09-2023
Aguardando Aprovação do Terceiro 11-09-2023
Aguardando Aprovação do Terceiro 11-09-2023
Aguardando Aprovação do Terceiro 11-09-2023
Aguardando Aprovação do Terceiro 11-09-2023
Aguardando Aprovação do Terceiro 11-09-2023
Aguardando Aprovação do Terceiro 11-09-2023


In [6]:
for a in ApprovalTransition.objects.filter(
                    origin__approval_flow__company__name="GES - UHE-SALTO SANTIAGO",
                    origin__approval_flow__target_model__icontains='OccurrenceRecord'):
    print(a.origin.name,' > Botão:',a.name,' > ',a.destination.name)

Aguardando Homologação  > Botão: Solicitar Revisão  >  Necessita Revisão
Aguardando Homologação  > Botão: Homologar  >  Registro Deferido
Aguardando Homologação  > Botão: Rejeitar  >  Registro Indeferido
Registro Deferido  > Botão: Sinalizar como Atendido  >  Atendido
Registro em Elaboração  > Botão: Solicitar aprovação técnica  >  Aguardando aprovação técnica
Necessita Revisão  > Botão: Solicitar aprovação técnica  >  Aguardando aprovação técnica
Aguardando aprovação técnica  > Botão: Solicitar revisão  >  Necessita Revisão
Aguardando aprovação técnica  > Botão: Solicitar Homologação  >  Aguardando Homologação


In [5]:
for a in UserInCompany.objects.filter(company__name__startswith="GSO",user__username="cid"):
    try:
        a.uuid=None
        a.user=User.objects.get(username='nilson.berticelli')
        a.save()
    except:
        print('ja existe aqui: ',a.company.name)

ja existe aqui:  GSO - UHE-PASSO FUNDO
ja existe aqui:  GSO - UHE-MACHADINHO
ja existe aqui:  GSO - UHE-ITA


In [6]:
for a in UserInCompany.objects.filter(company__name__startswith="GSB",user__email__icontains="tractebel"):
    for b in Company.objects.filter(name__startswith="GSB").exclude(name='GSB - UHE-SALTO SANTIAGO'):
        a.uuid=None
        a.company = b
        a.save()

In [8]:
UserInCompany.objects.filter(company__name__startswith="GSB",user__email__icontains="tractebel").count()

132

In [22]:
from tqdm.notebook import tqdm
from django.contrib.gis.geos import Point, GeometryCollection

In [23]:
def transform_coordinates(x, y, srid=31982):

    pnt = Point(x, y, srid=srid)
    pnt.transform(4326)

    lng = pnt.coords[0]
    lat = pnt.coords[1]
    return Point(lng, lat)

In [15]:
teste=OccurrenceRecord.objects.filter(company__name__startswith="GSO")[0]

In [11]:
teste.geometry[0][0],teste.geometry[0][1]

(-47.41736501455307, -6.33347904432452)

In [ ]:
for a in OccurrenceRecord.objects.filter(company__startswith="GSO"):

In [22]:
for a in UserInFirm.objects.filter(firm__company__name__startswith='GES', firm__name__icontains="Gerência de ativos"):
    try:
        a.uuid=None
        a.user = User.objects.get(username='cid')
        a.save()
    except Exception as e:
        print(e)
#     print("Usina:",a.firm.company.name,a.user.full_name)

duplicate key value violates unique constraint "companies_userinfirm_user_id_firm_id_840c3e4f_uniq"
DETAIL:  Key (user_id, firm_id)=(df3e437a-8a19-4a0d-8632-99bdd573e7fb, a35f9b79-ea8c-47f7-83b8-5be653aeca4b) already exists.

duplicate key value violates unique constraint "companies_userinfirm_user_id_firm_id_840c3e4f_uniq"
DETAIL:  Key (user_id, firm_id)=(df3e437a-8a19-4a0d-8632-99bdd573e7fb, cb3c7c9e-a361-40f7-8e86-825619f7b756) already exists.

duplicate key value violates unique constraint "companies_userinfirm_user_id_firm_id_840c3e4f_uniq"
DETAIL:  Key (user_id, firm_id)=(df3e437a-8a19-4a0d-8632-99bdd573e7fb, 0a4312cf-7fab-4519-82ce-618a8c48f1c3) already exists.

duplicate key value violates unique constraint "companies_userinfirm_user_id_firm_id_840c3e4f_uniq"
DETAIL:  Key (user_id, firm_id)=(df3e437a-8a19-4a0d-8632-99bdd573e7fb, 406a674b-3473-424f-8cf9-b785d0ae3347) already exists.

duplicate key value violates unique constraint "companies_userinfirm_user_id_firm_id_840c3e4f_un

In [23]:
for b in Company.objects.filter(name__startswith='GES',metadata__has_key='report_email_firms'):
    metadata = b.metadata
#     print(metadata['report_email_firms'])
    for c in metadata['report_email_firms']:
        uf = UserInFirm.objects.filter(firm__company=b, firm__uuid=c, firm__name__icontains="Gerência de ativos")
        for d in uf:
            print("Usina:",d.firm.company.name," | Usuário:",d.user.full_name)

Usina: GES - UHE-CANA BRAVA  | Usuário: José Magno Ferreira
Usina: GES - UHE-CANA BRAVA  | Usuário: Cid Ionceck
Usina: GES - UHE-MACHADINHO  | Usuário: Ederson Maia
Usina: GES - UHE-MACHADINHO  | Usuário: Cid Ionceck
Usina: GES - UHE-SALTO SANTIAGO  | Usuário: Cid Ionceck
Usina: GES - UHE-SALTO SANTIAGO  | Usuário: Paulo Henrique Santos Feitosa
Usina: GES - UHE-SÃO SALVADOR  | Usuário: Hikaro Batista Moura
Usina: GES - UHE-SÃO SALVADOR  | Usuário: Cid Ionceck
Usina: GES - UHE-ITA  | Usuário: Ederson Maia
Usina: GES - UHE-ITA  | Usuário: Cid Ionceck
Usina: GES - UHE-ITA  | Usuário: Diego Collet
Usina: GES - UHE-PASSO FUNDO  | Usuário: Ederson Maia
Usina: GES - UHE-PASSO FUNDO  | Usuário: Cid Ionceck
Usina: GES - UHE-SALTO OSORIO  | Usuário: Paulo Henrique Santos Feitosa
Usina: GES - UHE-SALTO OSORIO  | Usuário: Cid Ionceck
Usina: GES - UHE-SALTO OSORIO  | Usuário: Antônio C. Martins
Usina: GES - UHE-SALTO OSORIO  | Usuário: Dirceu Rafanhin
Usina: GES - UHE-MIRANDA  | Usuário: Welviston 

In [26]:
for a in Entity.objects.filter(company__name__startswith='GES'):
    print("Usina:",a.company.name," | Entidade:",a.name)

Usina: GES - UHE-CANA BRAVA  | Entidade: Engie
Usina: GES - UHE-ESTREITO  | Entidade: CESTE
Usina: GES - UHE-ESTREITO  | Entidade: Engie
Usina: GES - PCH-RONDONÓPOLIS  | Entidade: ENGIE
Usina: GES - PCH-RONDONÓPOLIS  | Entidade: ENGIE
Usina: GES - UHE-MACHADINHO  | Entidade: Consórcio
Usina: GES - UHE-MACHADINHO  | Entidade: Engie
Usina: GES - UHE-SALTO SANTIAGO  | Entidade: Engie
Usina: GES - PCH-JOSÉ GELÁZIO DA ROCHA  | Entidade: ENGIE
Usina: GES - PCH-JOSÉ GELÁZIO DA ROCHA  | Entidade: ENGIE
Usina: GES - UHE-SÃO SALVADOR  | Entidade: Engie
Usina: GES - GEH  | Entidade: Engie
Usina: GES - UHE-ITA  | Entidade: Engie
Usina: GES - UHE-ITA  | Entidade: Consórcio
Usina: GES - UHE-PASSO FUNDO  | Entidade: Engie
Usina: GES - UHE-SALTO OSORIO  | Entidade: Engie
Usina: GES - UHE-JAGUARA  | Entidade: Engie
Usina: GES - UHE-MIRANDA  | Entidade: Engie
Usina: GES - UHE-PONTE DE PEDRA  | Entidade: Engie


In [ ]:
company=Company.objects.get
aps = ApprovalStep.objects.filter(approval_flow__company=company,approval_flow__target_model__icontains="multiple")
aps

In [39]:
for b in Company.objects.filter(name__startswith='GES'):
    aps = ApprovalStep.objects.filter(approval_flow__company=b,approval_flow__target_model__icontains="measurement")
    print(b.name, " - Fluxo Boletins de Medição")
    for c in aps:
        print(c)
        print('\tUsuários:')
        for d in c.responsible_users.all():
            print('\t',d.full_name,d.username)
        print('\tEquipes:')
        for e in c.responsible_firms.all():
            print('\t',e.name)
#         print("\tResponsável pela criação: ",'Verdadeiro' if c.responsible_created_by == True else "Falso")
#         print("\tSupervisor: ",'Verdadeiro' if c.responsible_supervisor == True else "Falso")
#         print("\tResponsável pela Entidade: ",'Verdadeiro' if c.responsible_firm_entity == True else "Falso")
#         print("\tResponsável pela Equipe: ",'Verdadeiro' if c.responsible_firm_manager == True else "Falso")
#         print("\tAuto Execução na Transação: ",'Verdadeiro' if c.auto_execute_transition == True else "Falso")
    print()
    print()
    print()

GES - UHE-CANA BRAVA  - Fluxo Boletins de Medição
[GES - UHE-CANA BRAVA]: Aguardando Aprovação do Terceiro
	Usuários:
	Equipes:
[GES - UHE-CANA BRAVA]: Aguardando Aprovação do Preposto da Contratante
	Usuários:
	Equipes:
	 Homologadores
[GES - UHE-CANA BRAVA]: Aprovado
	Usuários:
	Equipes:



GES - UHE-ESTREITO  - Fluxo Boletins de Medição
[GES - UHE-ESTREITO]: Aguardando Aprovação do Terceiro
	Usuários:
	Equipes:
[GES - UHE-ESTREITO]: Aguardando Aprovação do Preposto da Contratante
	Usuários:
	Equipes:
	 Homologadores
[GES - UHE-ESTREITO]: Aprovado
	Usuários:
	Equipes:



GES - PCH-RONDONÓPOLIS  - Fluxo Boletins de Medição
[GES - PCH-RONDONÓPOLIS]: Aguardando Aprovação do Terceiro
	Usuários:
	Equipes:
[GES - PCH-RONDONÓPOLIS]: Aguardando Aprovação do Preposto da Contratante
	Usuários:
	 Cid Ionceck cid.ionceck
	 Rafael  Campos Da Silva PT5384
	 Cid Ionceck cid
	Equipes:
[GES - PCH-RONDONÓPOLIS]: Aprovado
	Usuários:
	Equipes:



GES - UHE-MACHADINHO  - Fluxo Boletins de Medição
[GES - 

In [28]:
c.__dict__

{'_state': <django.db.models.base.ModelState at 0x7f4f63a71c50>,
 'uuid': UUID('a9bbcf64-40b0-4a7a-b5a9-80c767698fb3'),
 'name': 'Aprovado',
 'approval_flow_id': UUID('945a60bb-c2cc-4a46-8ce4-85ba44b6e0e1'),
 'field_options': {'required': ['relationships.permissions.data.id']},
 'responsible_created_by': False,
 'responsible_supervisor': False,
 'responsible_firm_entity': False,
 'responsible_firm_manager': False,
 'auto_execute_transition': False,
 'responsible_json_logic': {},
 'color': '#FF0000'}

In [41]:
for b in Company.objects.filter(name__startswith='GES'):
    at=ApprovalTransition.objects.filter(
                                        destination__approval_flow__company=b,
                                        destination__approval_flow__target_model__icontains="measurement")
    print(b.name, " - Fluxo Boletins de Medição")
    for a in at:
        print(f"{a.origin.name} > BOTÃO: {str(a).split(':')[0]} > {a.destination.name}")
#         print("\tResponsável pela criação: ",'Verdadeiro' if c.responsible_created_by == True else "Falso")
#         print("\tSupervisor: ",'Verdadeiro' if c.responsible_supervisor == True else "Falso")
#         print("\tResponsável pela Entidade: ",'Verdadeiro' if c.responsible_firm_entity == True else "Falso")
#         print("\tResponsável pela Equipe: ",'Verdadeiro' if c.responsible_firm_manager == True else "Falso")
#         print("\tAuto Execução na Transação: ",'Verdadeiro' if c.auto_execute_transition == True else "Falso")
    print()
    print()
    print()

GES - UHE-CANA BRAVA  - Fluxo Boletins de Medição
Aguardando Aprovação do Terceiro > BOTÃO: [Aprovar] > Aguardando Aprovação do Preposto da Contratante
Aguardando Aprovação do Preposto da Contratante > BOTÃO: [Aprovar] > Aprovado



GES - UHE-ESTREITO  - Fluxo Boletins de Medição
Aguardando Aprovação do Terceiro > BOTÃO: [Aprovar] > Aguardando Aprovação do Preposto da Contratante
Aguardando Aprovação do Preposto da Contratante > BOTÃO: [Aprovar] > Aprovado



GES - PCH-RONDONÓPOLIS  - Fluxo Boletins de Medição
Aguardando Aprovação do Terceiro > BOTÃO: [Aprovar] > Aguardando Aprovação do Preposto da Contratante
Aguardando Aprovação do Preposto da Contratante > BOTÃO: [Aprovar] > Aprovado



GES - UHE-MACHADINHO  - Fluxo Boletins de Medição
Aguardando Aprovação do Terceiro > BOTÃO: [Aprovar] > Aguardando Aprovação do Preposto da Contratante
Aguardando Aprovação do Preposto da Contratante > BOTÃO: [Aprovar] > Aprovado



GES - UHE-SALTO SANTIAGO  - Fluxo Boletins de Medição
Aguardando Apr

In [4]:
ServiceOrder.objects.filter(number='UHCB-OS-2022.0023').update(is_closed=False)

1

In [83]:
ocr=OccurrenceRecord.objects.filter(company__name__startswith='GSO',occurrence_type_id='69513ccd-b8f9-439c-a418-158e95524d85')
ocr

<QuerySet [<OccurrenceRecord: [GSO - UHE-PASSO FUNDO] - UHPF-RF-2023.0084 - 2023-09-19 13:51:28.084638+00:00>, <OccurrenceRecord: [GSO - UHE-ITA] - UHIT-RF-2023.0013 - 2023-09-18 21:21:36.117097+00:00>, <OccurrenceRecord: [GSO - UHE-PASSO FUNDO] - UHPF-RF-2023.0086 - 2023-09-19 13:51:28.084638+00:00>, <OccurrenceRecord: [GSO - UHE-PASSO FUNDO] - UHPF-RF-2023.0075 - 2023-09-19 13:51:28.084638+00:00>, <OccurrenceRecord: [GSO - UHE-PASSO FUNDO] - UHPF-RF-2023.0079 - 2023-09-19 13:51:28.084638+00:00>, <OccurrenceRecord: [GSO - UHE-MACHADINHO] - UHMA-RF-2023.0104 - 2023-09-19 13:00:55.265196+00:00>, <OccurrenceRecord: [GSO - UHE-PASSO FUNDO] - UHPF-RF-2023.0075 - 2023-09-19 13:51:28.084638+00:00>, <OccurrenceRecord: [GSO - UHE-ITA] - UHIT-RF-2023.0029 - 2023-09-18 21:21:36.117097+00:00>, <OccurrenceRecord: [GSO - UHE-ITA] - UHIT-RF-2023.0065 - 2023-09-18 21:21:36.117097+00:00>, <OccurrenceRecord: [GSO - UHE-ITA] - UHIT-RF-2023.0062 - 2023-09-18 21:21:36.117097+00:00>, <OccurrenceRecord: [GS

In [53]:
from tqdm.notebook import tqdm
for a in tqdm(ocr):
    try:
        a.form_data['xcoord']=a.geometry[0][0][0]
        a.form_data['ycoord']=a.geometry[0][0][1]
#         a.form_data['escapeRoute'] = [{'xcoord':'','ycoord':''}]
        a.save()
    except Exception as e:
        a.form_data['xcoord']=a.geometry[0][0][0][0]
        a.form_data['ycoord']=a.geometry[0][0][0][1]
#         a.form_data['escapeRoute'] = [{'xcoord':'','ycoord':''}]
        a.save()
#         print(a.company,a.number, e)
#     print(a.geometry[0][0],a.geometry[0][1])

  0%|          | 0/643 [00:00<?, ?it/s]

In [82]:
for a in tqdm(ocr):
    if isinstance(a.form_data.get('xcoord'),list):
        a.form_data['ycoord'] = a.form_data['xcoord'][1]
        a.form_data['xcoord'] = a.form_data['xcoord'][0]
        a.save()

  0%|          | 0/643 [00:00<?, ?it/s]

In [87]:
rep_edit=[]
excluir = []
for a in tqdm(ocr):
    if a.number not in rep_edit:
        rep_edit.append(a.number)
    elif a.number in rep_edit:
        excluir.append(a)

  0%|          | 0/643 [00:00<?, ?it/s]

In [88]:
excluir

[<OccurrenceRecord: [GSO - UHE-PASSO FUNDO] - UHPF-RF-2023.0075 - 2023-09-19 13:51:28.084638+00:00>,
 <OccurrenceRecord: [GSO - UHE-SÃO SALVADOR] - UHSA-RF-2023.0002 - 2023-09-19 14:14:04.278204+00:00>,
 <OccurrenceRecord: [GSO - UHE-SALTO SANTIAGO] - UHSS-RF-2023.0003 - 2023-09-19 14:29:55.504413+00:00>,
 <OccurrenceRecord: [GSO - UHE-SALTO SANTIAGO] - UHSS-RF-2023.0001 - 2023-09-19 14:29:55.504413+00:00>,
 <OccurrenceRecord: [GSO - UHE-SALTO SANTIAGO] - UHSS-RF-2023.0000 - 2023-09-19 14:29:55.504413+00:00>,
 <OccurrenceRecord: [GSO - UHE-SALTO SANTIAGO] - UHSS-RF-2023.0001 - 2023-09-19 14:29:55.504413+00:00>,
 <OccurrenceRecord: [GSO - UHE-MIRANDA] - UHMI-RF-2023.0001 - 2023-09-19 13:20:32.059411+00:00>,
 <OccurrenceRecord: [GSO - UHE-PONTE DE PEDRA] - UHPP-RF-2023.0011 - 2023-09-19 14:01:39.012115+00:00>,
 <OccurrenceRecord: [GSO - UHE-PONTE DE PEDRA] - UHPP-RF-2023.0009 - 2023-09-19 14:01:39.012115+00:00>,
 <OccurrenceRecord: [GSO - UHE-PONTE DE PEDRA] - UHPP-RF-2023.0005 - 2023-09

In [89]:
rep_edit

['UHPF-RF-2023.0084',
 'UHIT-RF-2023.0013',
 'UHPF-RF-2023.0086',
 'UHPF-RF-2023.0075',
 'UHPF-RF-2023.0079',
 'UHMA-RF-2023.0104',
 'UHIT-RF-2023.0029',
 'UHIT-RF-2023.0065',
 'UHIT-RF-2023.0062',
 'UHIT-RF-2023.0074',
 'UHIT-RF-2023.0019',
 'UHIT-RF-2023.0025',
 'UHMA-RF-2023.0105',
 'UHMA-RF-2023.0093',
 'UHMA-RF-2023.0106',
 'UHIT-RF-2023.0011',
 'UHIT-RF-2023.0001',
 'UHIT-RF-2023.0020',
 'UHPF-RF-2023.0087',
 'UHPF-RF-2023.0073',
 'UHPP-RF-2023.0009',
 'UHSA-RF-2023.0002',
 'UHMA-RF-2023.0103',
 'UHJA-RF-2023.0020',
 'UHIT-RF-2023.0079',
 'UHET-RF-2023.0086',
 'UHSS-RF-2023.0001',
 'UHPP-RF-2023.0005',
 'UHSS-RF-2023.0040',
 'UHSA-RF-2023.0001',
 'UHET-RF-2023.0064',
 'UHIT-RF-2023.0039',
 'UHET-RF-2023.0076',
 'UHJA-RF-2023.0009',
 'UHSS-RF-2023.0003',
 'UHSS-RF-2023.0000',
 'UHSS-RF-2023.0014',
 'UHJA-RF-2023.0001',
 'UHMI-RF-2023.0001',
 'UHSS-RF-2023.0047',
 'UHSA-RF-2023.0015',
 'UHSS-RF-2023.0044',
 'UHPP-RF-2023.0011',
 'UHMA-RF-2023.0090',
 'UHSA-RF-2023.0029',
 'UHSO-RF-

In [78]:
teste =OccurrenceRecord.objects.filter(number='UHIT-RF-2023.0025')[0].geometry
teste

In [80]:
teste[0].__dict__

{'_constructor_args': ((<django.contrib.gis.geos.libgeos.LP_GEOSGeom_t at 0x7f20aa785950>,),
  {'srid': 4326}),
 '_ptr': <django.contrib.gis.geos.libgeos.LP_GEOSGeom_t at 0x7f20aa785950>,
 '_cs': None}

In [15]:
pr=Procedure.objects.first()

In [16]:
pr._meta.get_fields()

(<OneToOneRel: service_orders.procedure>,
 <ManyToManyRel: service_orders.procedurefile>,
 <ManyToOneRel: service_orders.procedureresource>,
 <ManyToManyRel: work_plans.workplan>,
 <ManyToManyRel: monitorings.monitoringcampaign>,
 <ManyToManyRel: monitorings.monitoringrecord>,
 <ManyToManyRel: monitorings.operationalcontrolrecord>,
 <django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: action>,
 <django.db.models.fields.TextField: occurrence_kind>,
 <django.db.models.fields.related.ForeignKey: occurrence_type>,
 <django.db.models.fields.related.ForeignKey: service_order_action_status>,
 <django.db.models.fields.related.ForeignKey: firm>,
 <django.db.models.fields.related.ForeignKey: responsible>,
 <django.db.models.fields.related.OneToOneField: procedure_previous>,
 <django.contrib.postgres.fields.jsonb.JSONField: form_data>,
 <django.db.models.fields.TextField: to_do>,
 <django.db.models.fields.related.ForeignKey: created_by>,
 <django.db.models.fiel

In [24]:
ServiceOrderAction._meta.get_fields()

(<ManyToOneRel: occurrence_records.occurrencerecord>,
 <ManyToOneRel: service_orders.procedure>,
 <django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: service_order>,
 <django.db.models.fields.TextField: name>,
 <django.db.models.fields.related.ForeignKey: created_by>,
 <django.db.models.fields.related.ForeignKey: service_order_action_status>,
 <django.db.models.fields.related.ForeignKey: firm>,
 <django.db.models.fields.related.ForeignKey: responsible>,
 <django.db.models.fields.DateTimeField: opened_at>,
 <django.db.models.fields.DateTimeField: closed_at>,
 <django.db.models.fields.DateTimeField: estimated_end_date>,
 <django.db.models.fields.related.ForeignKey: parent_record>)

# Entrega

In [40]:
soa=ServiceOrderAction.objects.all()[:10]

In [74]:
pr=Procedure.objects.filter(action__in=soa)

In [77]:
pr[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f7afd7e5c10>,
 'uuid': UUID('62cd9216-0cad-4b04-b69c-fc9e96803948'),
 'action_id': UUID('58056087-4349-42d3-bbbe-b48fccc4c50b'),
 'occurrence_kind': '',
 'occurrence_type_id': None,
 'service_order_action_status_id': UUID('33ccc2d4-a195-49db-a6e5-ad5edfce0e09'),
 'firm_id': UUID('c5937fda-32a3-4a74-a30d-0e08ed573002'),
 'responsible_id': UUID('31c72676-8481-4d13-b7aa-bc52b5f77abb'),
 'procedure_previous_id': None,
 'form_data': {},
 'to_do': 'Efetuar denuncia, através de Boletim de Ocorrência.',
 'created_by_id': UUID('31c72676-8481-4d13-b7aa-bc52b5f77abb'),
 'created_at': datetime.datetime(2023, 10, 23, 19, 28, 15, 630447, tzinfo=<UTC>),
 'deadline': datetime.datetime(2023, 11, 24, 19, 27, tzinfo=<UTC>),
 'done_at': datetime.datetime(2023, 11, 10, 19, 26, tzinfo=<UTC>),
 '_fieldsignals_originals': {(140166655505104,
   140166323859184): {'uuid': UUID('62cd9216-0cad-4b04-b69c-fc9e96803948'), 'action': UUID('58056087-4349-42d3-bbbe-b48f

In [85]:
lista2=[]
for a in pr:
    lista2.append([str(a.uuid),str(a.action_id)])

In [84]:
lista2

[]

In [60]:
lista=[]
for a in soa:
    lista.append([str(a.service_order_id),str(a.uuid)])

In [61]:
lista

[['2d25c6ab-4d5b-43e8-9179-ebe796faa084',
  '58056087-4349-42d3-bbbe-b48fccc4c50b'],
 ['92cbeca4-fc9a-4f94-ab3e-518ada15d1fd',
  '85d0016e-61fc-40e7-b7ad-d5e57b747b8e'],
 ['92cbeca4-fc9a-4f94-ab3e-518ada15d1fd',
  'c4bc22a9-0e39-4def-9f66-25aa4e921fb9'],
 ['1c3240af-0b72-4d90-a2cd-6ad59c88a6d8',
  '9d1e75cd-b93c-47a9-b165-6d0f413523d8'],
 ['1c3240af-0b72-4d90-a2cd-6ad59c88a6d8',
  '6510f829-ff1f-4dfc-a29c-c238665441bb'],
 ['918e5ec3-2e4f-41fc-aafd-f438a700e371',
  'e0deebfe-893c-472e-914d-c4c8c3d12b37'],
 ['d2a1fa5c-20a2-41c8-bc37-3582801a1917',
  '936fd1fc-d68a-4179-a7aa-b8c61040a9cb'],
 ['9379f3a5-9181-4158-9c82-c919ab255461',
  '26d1044c-86d5-4e14-b5da-86a908b57b4e'],
 ['9379f3a5-9181-4158-9c82-c919ab255461',
  '974168ff-66ce-4701-9662-7eb6ce6363b5'],
 ['9379f3a5-9181-4158-9c82-c919ab255461',
  'cc8d82f8-57d3-4538-a24b-0f04a2d7ab08']]

In [62]:
import pandas as pd

In [68]:
df=pd.DataFrame(data=lista,columns=['ServiceOrder uuid','ServiceOrderAction uuid'])

In [69]:
df

,ServiceOrder uuid,ServiceOrderAction uuid
0,2d25c6ab-4d5b-43e8-9179-ebe796faa084,58056087-4349-42d3-bbbe-b48fccc4c50b
1,92cbeca4-fc9a-4f94-ab3e-518ada15d1fd,85d0016e-61fc-40e7-b7ad-d5e57b747b8e
2,92cbeca4-fc9a-4f94-ab3e-518ada15d1fd,c4bc22a9-0e39-4def-9f66-25aa4e921fb9
3,1c3240af-0b72-4d90-a2cd-6ad59c88a6d8,9d1e75cd-b93c-47a9-b165-6d0f413523d8
4,1c3240af-0b72-4d90-a2cd-6ad59c88a6d8,6510f829-ff1f-4dfc-a29c-c238665441bb
5,918e5ec3-2e4f-41fc-aafd-f438a700e371,e0deebfe-893c-472e-914d-c4c8c3d12b37
6,d2a1fa5c-20a2-41c8-bc37-3582801a1917,936fd1fc-d68a-4179-a7aa-b8c61040a9cb
7,9379f3a5-9181-4158-9c82-c919ab255461,26d1044c-86d5-4e14-b5da-86a908b57b4e
8,9379f3a5-9181-4158-9c82-c919ab255461,974168ff-66ce-4701-9662-7eb6ce6363b5
9,9379f3a5-9181-4158-9c82-c919ab255461,cc8d82f8-57d3-4538-a24b-0f04a2d7ab08


In [86]:
df2=pd.DataFrame(data=lista2,columns=['Procedure uuid','ServiceOrderAction uuid'])

In [87]:
df2

,Procedure uuid,ServiceOrderAction uuid
0,62cd9216-0cad-4b04-b69c-fc9e96803948,58056087-4349-42d3-bbbe-b48fccc4c50b
1,10a246f1-a57d-4a16-875e-94598a78c68a,6510f829-ff1f-4dfc-a29c-c238665441bb
2,6fbeef49-b07b-4c53-bf64-97cf666c90c0,85d0016e-61fc-40e7-b7ad-d5e57b747b8e
3,4caab57e-fd63-467f-a94e-c3415feeb3e5,c4bc22a9-0e39-4def-9f66-25aa4e921fb9
4,8cf6fa96-1822-428e-befd-6a8410235eda,9d1e75cd-b93c-47a9-b165-6d0f413523d8
5,5e25aa44-d998-4050-894b-a25fb7a57930,6510f829-ff1f-4dfc-a29c-c238665441bb
6,b35507dc-6d86-463c-b75b-64fea33e6cdf,85d0016e-61fc-40e7-b7ad-d5e57b747b8e
7,06a2997f-b6c7-4652-be6c-ea5ff4c9302a,c4bc22a9-0e39-4def-9f66-25aa4e921fb9
8,d9b162f6-fd14-4aab-a6ed-4059a8af5416,9d1e75cd-b93c-47a9-b165-6d0f413523d8
9,e3d21370-bd79-4550-911e-6296e677d3e3,6510f829-ff1f-4dfc-a29c-c238665441bb


In [4]:
company=Company.objects.get(name="GES - UHE-JAGUARA")
engie_user=User.objects.get(username='lucas.reis')

In [31]:
oc=OccurrenceRecord.objects.filter(company=company,created_by=engie_user,created_at__gte='2023-10-28')
so=ServiceOrder.objects.filter(created_by=engie_user,company=company)
soa=ServiceOrderAction.objects.filter(created_by=engie_user,service_order__in=so)
pro=Procedure.objects.filter(action__in=soa,responsible=engie_user,created_by=engie_user)
oc.count(),so.count(),soa.count(),pro.count()

(284, 284, 2246, 284)

In [32]:
pro.delete()
soa.delete()
so.delete()
oc.delete()

(284, {'occurrence_records.OccurrenceRecord': 284})

In [59]:
City.objects.filter(name='Sulina')

<QuerySet [<City: [41] Sulina>]>

In [60]:
City.objects.filter(name='Saudade do Iguaçu')

<QuerySet [<City: [41] Saudade do Iguaçu>]>

In [53]:
for a in city:
    print(a.uuid)

12485fe2-a296-448d-949c-67a1d7c53b4f


In [33]:
city[1].uuid

UUID('fa52950c-e137-42af-968c-f086762590dd')

In [38]:
OccurrenceRecord.objects.filter(city=city[1])

<QuerySet []>

In [50]:
for a in range(8):
    if OccurrenceRecord.objects.filter(city=city[a]).exists():
        print('existe',city[a].uuid)
    else:
#         City.objects.filter(uuid=city[a].uuid).delete()
        city[a].delete()

existe 12485fe2-a296-448d-949c-67a1d7c53b4f


In [96]:
queryset = City.objects.values('name').annotate(count=Count('name'))

In [97]:
repeated_values = queryset.filter(count__gt=1)

In [98]:
repeated_values

<QuerySet [{'name': 'Palmas', 'count': 2}, {'name': 'Caseiros', 'count': 2}, {'name': 'Manfrinópolis', 'count': 2}]>

In [67]:
for a in repeated_values:
    for b in City.objects.filter(name=a['name']):
        print(b.uf_code,b.name)

42 Paial
42 Paial
41 Palmas
17 Palmas
43 Viadutos
43 Viadutos
42 Caseiros
43 Caseiros
43 Trindade do Sul
43 Trindade do Sul
43 Trindade do Sul
43 Trindade do Sul
42 Chapecó
42 Chapecó
41 Rio Bonito do Iguaçu
41 Rio Bonito do Iguaçu
41 Rio Bonito do Iguaçu
41 Rio Bonito do Iguaçu
41 Rio Bonito do Iguaçu
41 Rio Bonito do Iguaçu
41 Rio Bonito do Iguaçu
41 Rio Bonito do Iguaçu
41 Rio Bonito do Iguaçu
41 Rio Bonito do Iguaçu
41 Rio Bonito do Iguaçu
41 Rio Bonito do Iguaçu
41 Rio Bonito do Iguaçu
41 Rio Bonito do Iguaçu
41 Rio Bonito do Iguaçu
41 Rio Bonito do Iguaçu
41 Rio Bonito do Iguaçu
41 Rio Bonito do Iguaçu
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
42 Itá
41 Quedas do Iguaçu
41 Quedas do Iguaçu
41 Quedas do Iguaçu
43 Aratiba
43 Aratiba
43 Aratiba
43 Aratiba
43 Aratiba
43 Aratiba
43 Aratiba
43 Aratiba
43 Aratiba
43 Candiota
43 Candiota
43 Candiota
43 Candiota
43 Erval Grande


In [69]:
city_delete=[]
for a in repeated_values:
    ocr=OccurrenceRecord.objects.filter(city__name=a['name'])
    if ocr.exclude(city__name__in=['Palmas','Caseiros']).exists():
        city_prime=City.objects.filter(name=a['name']).first()
        ocr.update(city=city_prime)
        city_delete.append(City.objects.filter(name=a['name']).exclude(uuid=city_prime.uuid))
    else:
        print(a)

{'name': 'Palmas', 'count': 2}
{'name': 'Caseiros', 'count': 2}
{'name': 'Manfrinópolis', 'count': 2}


In [87]:
len(city_delete),city_delete[0]

(27, <QuerySet [<City: [42] Paial>]>)

In [95]:
for a in city_delete:
    for b in a:
        b.delete()

In [72]:
city

<QuerySet []>

In [115]:
step1 = ApprovalStep.objects.get(approval_flow__company__name='Engie Umburanas',name='Aguardando Aprovação do Supervisor',
                                approval_flow__target_model__icontains='access')

In [116]:
step2 = ApprovalStep.objects.get(approval_flow__company__name='Engie Umburanas',
                                 name='Aguardando Aprovação do Usuário-Chave do Sistema',
                                approval_flow__target_model__icontains='access')

In [117]:
aprovado = ApprovalStep.objects.get(approval_flow__company__name='Engie Umburanas',
                                    name='Aprovado',
                                   approval_flow__target_model__icontains='access')

In [129]:
at=ApprovalTransition.objects.filter(origin__approval_flow__company__name='Engie Umburanas',
                                 origin__approval_flow__target_model__icontains='access')
at

<QuerySet [<ApprovalTransition: [Solicitar Aprovação Usuário Chave]: Aguardando Aprovação do Supervisor - Aguardando Aprovação do Usuário-Chave do Sistema>, <ApprovalTransition: [Aprovar]: Aguardando Aprovação do Usuário-Chave do Sistema - Aprovado>]>

In [125]:
at.origin = step2
at.save()

In [119]:
at[0].origin,at[0].destination

(<ApprovalStep: [Engie Umburanas]: Aguardando Aprovação do Supervisor>,
 <ApprovalStep: [Engie Umburanas]: Aprovado>)

In [127]:
transition1 = ApprovalTransition(
    name="Solicitar Aprovação Usuário Chave",
    origin=step1,
    destination=step2,
    condition={"===": [{"var": "request.action"}, "Solicitar Aprovação Usuário Chave"]},
    callback={"change_fields": [{"name": "approved", "value": True}, {"name": "done", "value": True}], "create_user_in_company": True},
)
transition1.save()

In [1]:
ocr=OccurrenceRecord.objects.get(uuid='efcdeb68-cd5d-450c-b5de-cc6211efd96a')

In [8]:
ocr.geometry

In [7]:
company_origin = Company.objects.get(name='GES - UHE-ITA')
company = Company.objects.get(name='Energia Compartilhado')
company_origin,company

(<Company: d436bd3c-e145-44f2-a5ee-797a67110efd: GES - UHE-ITA>,
 <Company: 45b6a769-2235-4a13-a7b3-8b4d242be56b: Energia Compartilhado>)

In [8]:
for a in ShapeFile.objects.filter(companies=company_origin):
    a.companies.add(company)

In [9]:
for a in TileLayer.objects.filter(companies=company_origin):
    a.companies.add(company)

In [5]:
for a in OccurrenceTypeSpecs.objects.filter(company=Company.objects.get(name='GSO - UHE-ITA')):
    a.uuid=None
    a.company = company
#     a.save()

In [6]:
for a in ApprovalFlow.objects.filter(company=company_origin):
    a.uuid = None
    a.company=company
#     a.save()

In [7]:
for a in ApprovalStep.objects.filter(approval_flow__company=company_origin):
    a.uuid=None
    a.approval_flow = ApprovalFlow.objects.get(name=a.approval_flow.name,company=company)
#     a.save()

In [47]:
for a in ApprovalTransition.objects.filter(origin__approval_flow__company=company_origin):
    
    if a.origin.approval_flow.target_model != 'occurrence_records.OccurrenceRecord':
        a.uuid=None
        a.origin = ApprovalStep.objects.get(approval_flow__company=company,
                                        name=a.origin.name,
                                        approval_flow__target_model=a.origin.approval_flow.target_model)
        a.destination = ApprovalStep.objects.get(approval_flow__company=company,
                                        name=a.destination.name,
                                        approval_flow__target_model=a.destination.approval_flow.target_model)
#         a.save()
    else:
        a.uuid=None
        a.origin = ApprovalStep.objects.get(approval_flow__company=company,
                                        name=a.origin.name,
                                        approval_flow__target_model='reportings.Reporting')
        a.destination = ApprovalStep.objects.get(approval_flow__company=company,
                                        name=a.destination.name,
                                        approval_flow__target_model='reportings.Reporting')
#         a.save()
#         print(a.origin,a.destination)

In [65]:
for a in ApprovalTransition.objects.filter(origin__approval_flow__company=company_origin):
#     print(a.origin.name,' > ',a.destination.name)
    if a.destination.name == 'Cancelado':
        a.uuid=None
        a.origin = ApprovalStep.objects.get(approval_flow__company=company,
                                        name=a.origin.name,
                                        approval_flow__target_model='reportings.Reporting')
        a.destination = ApprovalStep.objects.get(approval_flow__company=company,
                                        name=a.destination.name,
                                        approval_flow__target_model='reportings.Reporting')
        a.save()
    

In [45]:
ApprovalFlow.objects.filter(company=company)

<QuerySet [<ApprovalFlow: [Energia Compartilhado]: Fluxo Usuários - companies.AccessRequest>, <ApprovalFlow: [Energia Compartilhado]: Fluxo Boletins de Medição - service_orders.MeasurementBulletin>, <ApprovalFlow: [Energia Compartilhado]: Fluxo Registros de Monitoramento - occurrence_records.OccurrenceRecord>, <ApprovalFlow: [Energia Compartilhado]: Fluxo Registros - reportings.Reporting>]>

In [19]:
OccurrenceRecord.objects.filter(occurrence_type__name='Agendamento')[0].uuid

UUID('b06fd92f-2ace-493f-a74b-22150a8585f4')

In [10]:
from uuid import uuid4

In [11]:
uuid4()

UUID('23e6f197-d4e1-43b7-974a-bce33bbc98f5')

UUID('23e6f197-d4e1-43b7-974a-bce33bbc98f5')

In [59]:
ocrs=OccurrenceRecord.history.filter(occurrence_type__name='Ponto de monitoramento',history_type='-').order_by('-history_date')
ocrs.count()

13

In [60]:
for a in ocrs:
    a.instance.save()

In [38]:
old=OccurrenceRecord.history.filter(occurrence_type__name='Ponto de monitoramento',history_type='+').order_by('-history_date')

In [52]:
old[0].operational_control

UUID('a9e78ec7-bc58-4f43-802b-4281184a1df8')

In [57]:
ServiceOrderActionStatus.history.filter(uuid='a9e78ec7-bc58-4f43-802b-4281184a1df8')[0].instance.save()

In [31]:
ocrs[0].operational_control.history.filter(history_type='-')

DoesNotExist: OperationalControl matching query does not exist.

In [25]:
ocrs=OccurrenceRecord.objects.filter(occurrence_type__name='Ponto de monitoramento')
ocrs.count()

0

In [23]:
ocrs2=OccurrenceRecord.objects.filter(occurrence_type__name='Pontos de monitoramento de águas superficiais')

In [24]:
ocrs2.count()

0

In [18]:
ocrs2[0].number='CDT-2023.0007'
# ocrs2[0].city.delete()
# del ocrs2[0].river_id
# ocrs2[0].status.delete()
# ocrs2[0].operational_control.delete()
# del ocrs2[0].firm
ocrs2[0].save()

IndexError: list index out of range

In [11]:
ocr=OccurrenceRecord.objects.get(uuid='12d3682a-2f7c-4d63-a1ca-47fb75669550')
ocr.__dict__

{'_state': <django.db.models.base.ModelState at 0x7ff37b99b4d0>,
 'uuid': UUID('12d3682a-2f7c-4d63-a1ca-47fb75669550'),
 'datetime': datetime.datetime(2023, 11, 22, 13, 55, 21, 128000, tzinfo=<UTC>),
 'number': 'UHIT-RG-2023.6551',
 'created_at': datetime.datetime(2023, 11, 22, 19, 3, 36, 436789, tzinfo=<UTC>),
 'updated_at': datetime.datetime(2023, 11, 23, 1, 16, 51, 498632, tzinfo=<UTC>),
 'reviews': 0,
 'company_id': UUID('d436bd3c-e145-44f2-a5ee-797a67110efd'),
 'editable': True,
 'is_approved': False,
 'uf_code': 43,
 'city_id': UUID('5a1c8c6a-8987-4c6c-b625-85483a49a7df'),
 'location_id': None,
 'place_on_dam': '',
 'river_id': None,
 'point': None,
 'geometry': <GeometryCollection object at 0x7ff377a7bf40>,
 'properties': [{}],
 'distance_from_dam': 0.0,
 'other_reference': '',
 'origin': '5',
 'origin_media': '',
 'informer': {},
 'territorial_administration': '',
 'created_by_id': UUID('eb44f871-cd7b-4f55-94bd-6af25fadfa70'),
 'search_tag_description': 'Talude marginal',
 'occ

In [17]:
ocr.geometry[0]

In [53]:
mon_super = OccurrenceType.objects.get(uuid='71702b2c-0d0c-41c1-b5c4-243a40785967')
company=Company.objects.get(name='Energia Compartilhado')
user=User.objects.get(username='engie')
firm=Firm.objects.get(company=company,name='Suporte')
status = ServiceOrderActionStatus.objects.get(name='Identificado',companies=company)
road=Road.objects.get(company=company,name__icontains='nome')
aps=ApprovalStep.objects.get(approval_flow__company=company,name='Registro em Elaboração')
mon_super.name,company,user,firm,status,road,aps

('Cadastro',
 <Company: 45b6a769-2235-4a13-a7b3-8b4d242be56b: Energia Compartilhado>,
 <User: engie - Suporte Kartado>,
 <Firm: [45b6a769-2235-4a13-a7b3-8b4d242be56b: Energia Compartilhado] 4a981c32-19cc-4f32-b2f2-7e9c57cf3ef2: Suporte>,
 <ServiceOrderActionStatus: Identificado - REPORTING_STATUS>,
 <Road: 649: Sem Nome 1>,
 <ApprovalStep: [Energia Compartilhado]: Registro em Elaboração>)

In [54]:
ocr=OccurrenceRecord.objects.filter(company__name='GES - UHE-ITA',occurrence_type=mon_super)
ocr.count()

1430

In [20]:
ocr_institucional = []
for a in ocr:
    if a.form_data.get('institutional_representaties'):
        ocr_institucional.append(a)

In [52]:
len(ocr_institucional),ocr[20].geometry[0]

(194, <Point object at 0x7ff3788ff490>)

In [86]:
dict_local={"ZAS":"1","ZSS":"2","NA":"3"}
dict_local['ZAS']

'1'

In [56]:
from tqdm.notebook import tqdm
from uuid import uuid4
for a in tqdm(ocr[:20]):
#     a.form_data['ufs'] = a.uf_code
#     a.form_data['local'] = dict_local.get(a.place_on_dam,"3")
    rep=Reporting(
        uuid=uuid4(),
        km=0,
        road=road,
        occurrence_type=OccurrenceType.objects.get(name='Instituições e parceiros',company=company),
        company=company,
        point=a.geometry[0] if a.geometry is not None else None,
        created_by=user,
        firm=firm,
        status=status,
        form_data=a.form_data,
        approval_step=aps
    )
    
    rep.save()
    

  0%|          | 0/20 [00:00<?, ?it/s]

In [96]:
rep

<Reporting: [Energia Compartilhado GSO] CDT-2023.0060 - 2023-11-24 14:26:59.730663+00:00>

In [15]:
monitorings=MonitoringPoint.objects.filter(monitoring_plan__uuid='f1d0a53f-b591-454d-9c54-2e81dc2cbefa')

In [20]:
monitorings.count(),monitorings[1].__dict__,monitorings[1].coordinates

(3,
 {'_state': <django.db.models.base.ModelState at 0x7f4646108390>,
  'uuid': UUID('ee78e408-f019-4a77-aacb-bdbb40ab1e83'),
  'created_at': datetime.datetime(2019, 12, 10, 18, 50, 17, 433774, tzinfo=<UTC>),
  'updated_at': datetime.datetime(2019, 12, 10, 18, 50, 17, 433800, tzinfo=<UTC>),
  'created_by_id': UUID('df3e437a-8a19-4a0d-8632-99bdd573e7fb'),
  'active': True,
  'code': 'MVU2',
  'uf_code': 42,
  'city_id': None,
  'location_id': UUID('e3beb841-435b-43e7-8988-79455fef6cd4'),
  'river_id': UUID('96757977-4aa6-4104-8568-4df8f6c09acc'),
  'place_on_dam': '2',
  'coordinates': <Point object at 0x7f46468950a0>,
  'monitoring_plan_id': UUID('f1d0a53f-b591-454d-9c54-2e81dc2cbefa'),
  'coverage_area': None,
  'segment': '4',
  'description': 'Rio Uruguai, sob a ponte da SC 466',
  'depth': '1',
  'position': '2',
  'stratification': '2',
  'zone': '2'},
 <Point object at 0x7f4646177910>)

In [7]:
from uuid import uuid4
for a in monitorings:
    rep=Reporting(
        uuid=uuid4(),
        km=0,
        road=road,
        occurrence_type=mon_super,
        company=company,
        created_by=user,
        firm=firm,
        status=status,
        form_data={
            'point_code':a.code,
            'description':a.description,
            'depth':a.depth,
            'position':a.position,
            'stratification': a.stratification,
            'zone':a.zone,
            'brazilian_state':'2',
            'city':'1',
            'neighbor':'1',
            'place':'1',
            'river':'1'
        }
    )
    
    rep.save()

In [25]:
point=monitorings[0].coordinates

<QuerySet [<OccurrenceType: Atendimento Interno - ['Energia Compartilhado']>, <OccurrenceType: Instituições e parceiros - ['Energia Compartilhado']>, <OccurrenceType: Pontos de monitoramento de águas superficiais - ['Energia Compartilhado']>, <OccurrenceType: Atendimento Externo - ['Energia Compartilhado']>, <OccurrenceType: Agendamento - ['Energia Compartilhado']>, <OccurrenceType: Ocorrência Ambiental - ['Energia Compartilhado']>, <OccurrenceType: Parâmetros físico-químicos - ['Energia Compartilhado']>]>

In [33]:
for a in OccurrenceType.objects.filter(company=company):
    for b in a.form_fields['fields']:
        b['showInColumn'] = True
    a.save()

In [43]:
UserPermission.objects.filter(companies=company)[0].permissions

{'Job': {'can_edit': True,
  'can_view': True,
  'queryset': 'all',
  'can_create': True,
  'can_delete': True,
  'can_reschedule': True,
  'can_view_progress_in_app': True},
 'File': {'can_edit': True,
  'can_view': True,
  'queryset': 'all',
  'can_create': True,
  'can_delete': True},
 'Firm': {'can_edit': True,
  'can_view': True,
  'queryset': 'all',
  'can_create': True,
  'can_delete': True},
 'Road': {'can_edit': True,
  'can_view': True,
  'queryset': 'all',
  'can_create': False,
  'can_delete': False},
 'User': {'can_edit': True,
  'can_view': True,
  'queryset': 'all',
  'can_create': True,
  'can_delete': False,
  'can_edit_all_fields': False},
 'Company': {'can_edit': True,
  'can_view': True,
  'queryset': 'self',
  'can_create': True,
  'can_delete': False,
  'can_change_custom_options': True},
 'Service': {'can_edit': True,
  'can_view': False,
  'queryset': 'all',
  'can_create': True,
  'can_delete': True},
 'Contract': {'can_edit': True,
  'can_view': True,
  'query

In [47]:
for a in UserPermission.objects.filter(companies=company):
    a.permissions['Reporting']['can_approve'] = True
    a.save()

In [48]:
a.permissions['Reporting']

{'can_edit': True,
 'can_view': True,
 'queryset': 'all',
 'can_create': True,
 'can_delete': True,
 'can_view_price_accumulator': True,
 'can_approve': True}

In [55]:
reps=Reporting.objects.filter(company=company).exclude(occurrence_type__occurrence_kind='2')
app = ApprovalStep.objects.get(uuid='d1a1a2e5-92aa-4794-9c41-aeadae249746')
reps.count(),app

(10, <ApprovalStep: [Energia Compartilhado]: Registro em Elaboração>)

In [56]:
for a in reps:
    a.approval_step = app
    a.save()

Searching KM on direction...
Searching KM without direction...


In [6]:
permissions= UserPermission.objects.filter(companies__name='Energia Compartilhado')

In [10]:
from tqdm.notebook import tqdm
for a in tqdm(UserPermission.objects.filter(companies__name='Energia Compartilhado GSO')):
    try:
        del a.permissions['MultipleDailyReport']
        a.save()
    except Exception as e:
        print(a.name, e)

  0%|          | 0/13 [00:00<?, ?it/s]

In [17]:
for a in RecordPanel.objects.filter(company__name='GSO - UHE-ITA'):
    a.uuid=None
    a.company = Company.objects.get(name='Energia Compartilhado GSO')
    a.save()

'reporting'
'reporting'
'reporting'
'reporting'
'reporting'


In [18]:
for a in OccurrenceType.objects.filter(company=Company.objects.get(name='GSO - UHE-ITA')):
    a.uuid=None
#     a.save()
    specs = OccurrenceTypeSpecs.objects.get(company=Company.objects.get(name='Energia Compartilhado GSO'),
                                            occurrence_type__name=a.name)
    specs.occurrence_type=a
#     specs.save()

In [23]:
for a in OccurrenceType.objects.filter(company=Company.objects.get(name='Energia Compartilhado GSO')):
    a.occurrence_kind = 2
    a.save()
    print(a.occurrence_kind,a.name)

2 Ficha descrita da estação remota
2 Identificação em Área Vulnerável
2 Inventário de danos
2 Sinalização da rota de fuga
2 Ponto de encontro


In [50]:
River.objects.filter(company=Company.objects.get(name='GES - UHE-ITA'))

<QuerySet [<River: [GES - UHE-ITA] - Rio dos Queimados>, <River: [GES - UHE-ITA] - Rio Fragosos>, <River: [GES - UHE-ITA] - Arroio Lambedor>, <River: [GES - UHE-ITA] - Rio Palomas>, <River: [GES - UHE-ITA] - Rio Ligeiro>, <River: [GES - UHE-ITA] - Rio Novo>, <River: [GES - UHE-ITA] - Rio Engano>, <River: [GES - UHE-ITA] - Rio Dourado>, <River: [GES - UHE-ITA] - Rio Jacutinga>, <River: [GES - UHE-ITA] - Rio Rancho Grande>, <River: [GES - UHE-ITA] - Rio Suzana>, <River: [GES - UHE-ITA] - Lajeado Teixeira Soares>, <River: [GES - UHE-ITA] - Rio Suruvi>, <River: [GES - UHE-ITA] - Rio do Peixe>, <River: [GES - UHE-ITA] - Rio Uruguai>, <River: [GES - UHE-ITA] - Rio Mirim>]>

In [38]:
dados=[]
for a in City.objects.all():
    dados.append({'name':a.name,'ufCode':[str(a.uf_code)],'value':str(a.uuid)})

In [52]:
dados=[]
for i,a in enumerate(River.objects.all(),start=1):
    dados.append({'name':a.name,'value':i})

In [54]:
OccurrenceType.objects.get(uuid='113abbf5-453f-41dd-b939-a623e7113a17')

<OccurrenceType: Ficha descrita da estação remota - ['GSO - UHE-ITA', 'GSO - UHE-SÃO SALVADOR', 'GSO - UHE-MACHADINHO', 'GSO - PCH-RONDONÓPOLIS', 'GSO - UHE-SALTO SANTIAGO', 'GSO - UHE-MIRANDA', 'GSO - UHE-SALTO OSORIO', 'GSO - UHE-PASSO FUNDO', 'GSO - UHE-CANA BRAVA', 'GSO - UHE-JAGUARA', 'GSO - UTPS', 'GSO - UHE-PONTE DE PEDRA', 'GSO - PCH-JOSÉ GELÁZIO DA ROCHA', 'GSO - UHE-ESTREITO']>

In [35]:
estados = [
    {"name": "Rondônia", "value": "11"},
    {"name": "Acre", "value": "12"},
    {"name": "Amazonas", "value": "13"},
    {"name": "Roraima", "value": "14"},
    {"name": "Pará", "value": "15"},
    {"name": "Amapá", "value": "16"},
    {"name": "Tocantins", "value": "17"},
    {"name": "Maranhão", "value": "21"},
    {"name": "Piauí", "value": "22"},
    {"name": "Ceará", "value": "23"},
    {"name": "Rio Grande do Norte", "value": "24"},
    {"name": "Paraíba", "value": "25"},
    {"name": "Pernambuco", "value": "26"},
    {"name": "Alagoas", "value": "27"},
    {"name": "Sergipe", "value": "28"},
    {"name": "Bahia", "value": "29"},
    {"name": "Minas Gerais", "value": "31"},
    {"name": "Espírito Santo", "value": "32"},
    {"name": "Rio de Janeiro", "value": "33"},
    {"name": "São Paulo", "value": "35"},
    {"name": "Paraná", "value": "41"},
    {"name": "Santa Catarina", "value": "42"},
    {"name": "Rio Grande do Sul", "value": "43"},
    {"name": "Mato Grosso do Sul", "value": "50"},
    {"name": "Mato Grosso", "value": "51"},
    {"name": "Goiás", "value": "52"},
    {"name": "Distrito Federal", "value": "53"}
]

In [60]:
ocr=OccurrenceRecord.objects.filter(occurrence_type__uuid='c16b1d6b-856c-4ff7-ae2f-f68d5223a077',
                                    company__name='GES - UHE-CANA BRAVA')

In [61]:
ocr.count()

816

In [62]:
from tqdm.notebook import tqdm
alterar=[]
for a in tqdm(ocr):
    if str(a.occurrence_type_id) != 'c16b1d6b-856c-4ff7-ae2f-f68d5223a077':
#         print(a.occurrence_type_id)
        alterar.append(a)

  0%|          | 0/816 [00:00<?, ?it/s]

In [64]:
len(alterar)

0

In [ ]:
# miranda operationalcontrol 20d2ffb0-d5dd-4158-af8f-81c7c46874a2

In [70]:
ocr=OccurrenceRecord.objects.filter(operational_control__uuid='969efb2a-fda0-4251-b9d1-01424c24ea3e')

In [71]:
ocr.count(),ocr[0].company

(1531, <Company: cff59760-69dd-4d07-bba5-61aa8b6faeea: GES - UHE-CANA BRAVA>)

In [72]:
from tqdm.notebook import tqdm
alterar=[]
for a in tqdm(ocr):
    if str(a.company.name) != 'GES - UHE-CANA BRAVA':
#         print(a.occurrence_type_id)
        alterar.append(a)

  0%|          | 0/1531 [00:00<?, ?it/s]

In [73]:
len(alterar),alterar[0].company

(715, <Company: 96bcaf1a-593b-4c6e-bdca-9b0dad0db0f1: GES - UHE-SÃO SALVADOR>)

In [74]:
for a in tqdm(alterar):
    a.company = ocr[0].company
    a.save()

  0%|          | 0/715 [00:00<?, ?it/s]

In [1]:
company=Company.objects.get(name='GES - UHE-ESTREITO')
operational_control=OperationalControl.objects.get(firm__company=company, kind=next(a['value'] for a in company.custom_options['operationalControl']['fields']['kind']['selectOptions']['options'] if 'horto' in a['name'].lower()))
cadastro = OccurrenceType.objects.get(uuid='c16b1d6b-856c-4ff7-ae2f-f68d5223a077',company=company)
cadastro_matriz_vegetal = OccurrenceType.objects.get(uuid='f200613b-8862-4a24-8c16-d6babd215fd7',company=company)
company,operational_control,cadastro,cadastro_matriz_vegetal

(<Company: 42ddc389-3258-4daf-868a-097235427e0f: GES - UHE-ESTREITO>,
 <OperationalControl: [GES - UHE-ESTREITO]: 701 - [Serviços de desenvolvimento de plataforma para gestão de equipes e serviços ambientais]>,
 <OccurrenceType: Cadastro - ['GES - UHE-MIRANDA', 'GES - UHE-PASSO FUNDO', 'GES - UHE-SALTO OSORIO', 'GES - UHE-PONTE DE PEDRA', 'GES - UHE-SALTO SANTIAGO', 'GES - UHE-JAGUARA', 'GES - UHE-ITA', 'GES - UHE-ESTREITO', 'GES - UHE-SÃO SALVADOR', 'GES - UHE-MACHADINHO', 'GES - UHE-CANA BRAVA']>,
 <OccurrenceType: Cadastro de Matriz Vegetal - ['GES - UHE-ESTREITO', 'GES - UHE-JAGUARA', 'GES - UHE-CANA BRAVA', 'GES - UHE-ITA', 'ATIVO-TREINAMENTO', 'GES - UHE-MIRANDA', 'GES - UHE-SALTO OSORIO', 'GES - UHE-SÃO SALVADOR', 'GES - UHE-PASSO FUNDO', 'GES - UHE-MACHADINHO', 'GES - UHE-PONTE DE PEDRA', 'GES - UHE-SALTO SANTIAGO', 'GES - PCH-JOSÉ GELÁZIO DA ROCHA', 'GES - PCH-RONDONÓPOLIS']>)

In [49]:
from django.db.models import CharField
from django.db.models.functions import Cast
especies=list(OccurrenceRecord.objects.filter(occurrence_type=cadastro,
                                              company=company,form_data__kingdom='2').annotate(uuid_str=Cast("uuid",output_field=CharField())
                                                                                     ).values_list('uuid_str',flat=True).distinct())

In [50]:
len(especies)

846

In [51]:
excluir=[]
for a in especies:
    ocr=OccurrenceRecord.objects.filter(company=company,
                                form_data__records=a)
    if not ocr.exists():
        excluir.append(a)

In [52]:
len(excluir),excluir[0]

(714, '026e2cae-a54c-4017-9714-24a14b7f7bc3')

In [53]:
for a in excluir:
    ocr=OccurrenceRecord.objects.filter(company=company,
                                uuid=a)
    ocr.delete()

In [45]:
ocr=OccurrenceRecord.objects.filter(company=company,
                                occurrence_type=cadastro_matriz_vegetal,
                                form_data__records=excluir[1])
ocr

<QuerySet []>

In [18]:
ocr=OccurrenceRecord.objects.filter(company=company,
                                occurrence_type=cadastro_matriz_vegetal,
                                form_data__records__in=especies).distinct()
ocr.count()

773

In [34]:
ocr=OccurrenceRecord.objects.filter(company=company,
                                occurrence_type=cadastro_matriz_vegetal).exclude(
                                form_data__records__in=especies).distinct()
ocr.count()

67

In [24]:
ocr=OccurrenceRecord.objects.filter(company=company,
                                occurrence_type=cadastro_matriz_vegetal)

In [25]:
excluir=[]
for a in ocr:
    if a.form_data.get('records') not in especies:
        excluir.append(a)

In [29]:
len(excluir),excluir[0].occurrence_type.name,excluir[0].__dict__

(68,
 'Cadastro de Matriz Vegetal',
 {'_state': <django.db.models.base.ModelState at 0x7f0274974bd0>,
  'uuid': UUID('24649e5a-52b9-4a91-9652-809c98b172a2'),
  'datetime': datetime.datetime(2023, 11, 9, 14, 15, 13, 379000, tzinfo=<UTC>),
  'number': 'UHET-RG-2023.3408',
  'created_at': datetime.datetime(2023, 11, 9, 14, 25, 50, 848761, tzinfo=<UTC>),
  'updated_at': datetime.datetime(2023, 11, 9, 14, 25, 51, 80505, tzinfo=<UTC>),
  'reviews': 0,
  'company_id': UUID('42ddc389-3258-4daf-868a-097235427e0f'),
  'editable': True,
  'is_approved': True,
  'uf_code': 21,
  'city_id': UUID('38410323-3368-4815-9ee7-7ee05ccad8b2'),
  'location_id': UUID('b78830fb-234a-4c33-8912-c1a77c410629'),
  'place_on_dam': '2',
  'river_id': UUID('892ab731-2738-4898-8d02-318fb2a24086'),
  'point': None,
  'geometry': <GeometryCollection object at 0x7f026edd5b50>,
  'properties': [{}],
  'distance_from_dam': 0.0,
  'other_reference': '',
  'origin': '',
  'origin_media': '',
  'informer': {'mail': '',
   'p

In [6]:
ocrs=OccurrenceRecord.objects.filter(company__name="GES - UHE-ITA",
                                operational_control__uuid='79063a2a-49fc-46e1-b9d9-da2d5069db0f'
                               )

In [29]:
companies=list(OccurrenceTypeSpecs.objects.filter(occurrence_type=cadastro_matriz_vegetal
                                                 ).values_list('company__name',flat=True))
companies

['GES - UHE-ESTREITO',
 'GES - UHE-JAGUARA',
 'GES - UHE-CANA BRAVA',
 'GES - UHE-ITA',
 'ATIVO-TREINAMENTO',
 'GES - UHE-MIRANDA',
 'GES - UHE-SALTO OSORIO',
 'GES - UHE-SÃO SALVADOR',
 'GES - UHE-PASSO FUNDO',
 'GES - UHE-MACHADINHO',
 'GES - UHE-PONTE DE PEDRA',
 'GES - UHE-SALTO SANTIAGO',
 'GES - PCH-JOSÉ GELÁZIO DA ROCHA',
 'GES - PCH-RONDONÓPOLIS']

In [58]:
data=[]
for a in companies:
    especies=list(OccurrenceRecord.objects.filter(occurrence_type=cadastro,
                                              company__name=a).annotate(uuid_str=Cast("uuid",output_field=CharField())
                                                                                     ).values_list('uuid_str',flat=True))
    especies_com_cadastro_ativo=OccurrenceRecord.objects.filter(company__name=a,
                                occurrence_type=cadastro_matriz_vegetal,
                                form_data__records__in=especies).distinct()
    data.append({'unidade':a,'qtd_especies_em_cadastro_vegetal':especies_com_cadastro_ativo.count()})

In [59]:
data

[{'unidade': 'GES - UHE-ESTREITO', 'qtd_especies_em_cadastro_vegetal': 773},
 {'unidade': 'GES - UHE-JAGUARA', 'qtd_especies_em_cadastro_vegetal': 0},
 {'unidade': 'GES - UHE-CANA BRAVA', 'qtd_especies_em_cadastro_vegetal': 0},
 {'unidade': 'GES - UHE-ITA', 'qtd_especies_em_cadastro_vegetal': 992},
 {'unidade': 'ATIVO-TREINAMENTO', 'qtd_especies_em_cadastro_vegetal': 0},
 {'unidade': 'GES - UHE-MIRANDA', 'qtd_especies_em_cadastro_vegetal': 0},
 {'unidade': 'GES - UHE-SALTO OSORIO', 'qtd_especies_em_cadastro_vegetal': 0},
 {'unidade': 'GES - UHE-SÃO SALVADOR', 'qtd_especies_em_cadastro_vegetal': 0},
 {'unidade': 'GES - UHE-PASSO FUNDO', 'qtd_especies_em_cadastro_vegetal': 0},
 {'unidade': 'GES - UHE-MACHADINHO', 'qtd_especies_em_cadastro_vegetal': 0},
 {'unidade': 'GES - UHE-PONTE DE PEDRA',
  'qtd_especies_em_cadastro_vegetal': 0},
 {'unidade': 'GES - UHE-SALTO SANTIAGO',
  'qtd_especies_em_cadastro_vegetal': 0},
 {'unidade': 'GES - PCH-JOSÉ GELÁZIO DA ROCHA',
  'qtd_especies_em_cadast

In [34]:
occ=OccurrenceType.objects.get(uuid='eec1c966-a20c-4dd8-b369-b9ed163b914c')
company=Company.objects.get(name="Energia Compartilhado")
company,occ

(<Company: 45b6a769-2235-4a13-a7b3-8b4d242be56b: Energia Compartilhado>,
 <OccurrenceType: Ocorrência Ambiental - ['Energia Compartilhado']>)

In [25]:
# OccurrenceType.objects.filter(occurrence_kind=1,name__startswith='Ocorrência',company=company).exclude(uuid=occ.uuid).delete()

(22,
 {'occurrence_records.OccurrenceTypeSpecs': 11,
  'occurrence_records.OccurrenceType': 11})

In [35]:
lista =[a['name'] for a in occ.form_fields['fields'][0]['selectOptions']['options']]
len(lista),lista

(12,
 ['Aprisionamento',
  'Caça/Pesca',
  'Contaminação',
  'Deslizamento',
  'Despejo',
  'Erosão',
  'Incêndio',
  'Perecimento',
  'Proliferação',
  'Supressão',
  'Vazamento',
  'Abertura de Acesso'])

In [36]:
from tqdm.notebook import tqdm
from copy import deepcopy
for a in tqdm(lista[1:]):
    new_occ=deepcopy(occ)
    new_occ.name=''
    new_occ.uuid=None
    new_occ.name=f'{occ.name} {a}'
    new_occ.form_fields['name'] = f'{occ.name} {a}'
    new_occ.form_fields['displayName'] = f'{occ.name} {a}'
#     print(new_occ.name)
    new_occ.save()
    
    spec=OccurrenceTypeSpecs(occurrence_type=new_occ, company=company)
    spec.save()

  0%|          | 0/11 [00:00<?, ?it/s]

In [44]:
company_origin=Company.objects.get(name='GSO - UHE-ITA')
company=Company.objects.get(name="Energia Compartilhado")
company_origin,company

(<Company: e9fce210-f652-4d7d-befe-32e02a8f0f60: GSO - UHE-ITA>,
 <Company: 45b6a769-2235-4a13-a7b3-8b4d242be56b: Energia Compartilhado>)

In [45]:
for a in OccurrenceType.objects.filter(company=company_origin):
    a.uuid=None
    a.save()
    
    spec=OccurrenceTypeSpecs(occurrence_type=a, company=company)
    spec.save()

In [47]:
occ=OccurrenceType.objects.filter(company=company,name__startswith='Ocorrência',occurrence_kind=1)
occ

<QuerySet [<OccurrenceType: Ocorrência Ambiental Incêndio - ['Energia Compartilhado']>, <OccurrenceType: Ocorrência Ambiental Deslizamento - ['Energia Compartilhado']>, <OccurrenceType: Ocorrência Ambiental Caça/Pesca - ['Energia Compartilhado']>, <OccurrenceType: Ocorrência Ambiental Contaminação - ['Energia Compartilhado']>, <OccurrenceType: Ocorrência Ambiental Despejo - ['Energia Compartilhado']>, <OccurrenceType: Ocorrência Ambiental Perecimento - ['Energia Compartilhado']>, <OccurrenceType: Ocorrência Ambiental Erosão - ['Energia Compartilhado']>, <OccurrenceType: Ocorrência Ambiental Proliferação - ['Energia Compartilhado']>, <OccurrenceType: Ocorrência Ambiental Vazamento - ['Energia Compartilhado']>, <OccurrenceType: Ocorrência Ambiental Supressão - ['Energia Compartilhado']>, <OccurrenceType: Ocorrência Ambiental Abertura de Acesso - ['Energia Compartilhado']>, <OccurrenceType: Ocorrência Ambiental Aprisionamento - ['Energia Compartilhado']>]>

In [48]:
for a in occ:
    a.name = a.name.replace('Ocorrência Ambiental ','')
    a.save()

In [13]:
user=User.objects.get(saml_nameid='ZT5753')
user_origin = User.objects.get(username='engie')
user,user_origin

(<User: ZT5753 - Daniel Silva Alves>, <User: engie - Suporte Kartado>)

In [15]:
for a in UserInCompany.objects.filter(company__name__startswith='GSB',user=user_origin):
    a.uuid=None
    a.user = user
    a.save()

ValueError: The QuerySet value for an exact lookup must be limited to one result using slicing.

In [17]:
OccurrenceRecord.objects.get(number='UHMI-RG-2023.1136').point

In [15]:
from openpyxl import load_workbook
from django.contrib.gis.geos import Point, GeometryCollection
import locale
from tqdm.notebook import tqdm

In [14]:
def transform_coordinates(x, y, srid=31982):

    pnt = Point(x, y, srid=srid)
    pnt.transform(4326)

    lng = pnt.coords[0]
    lat = pnt.coords[1]
    return Point(lng, lat)

In [11]:
wb = load_workbook('mirandaReg' + '.xlsx')
sheetname = wb.sheetnames[1]
# sheetname
ws = wb[sheetname]

In [12]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
        
    values.append(obj)

In [36]:
point=transform_coordinates(values[0]['Coordenada X'], values[0]['Coordenada Y']) if values[0]['Coordenada X'] is not None else None
geometry=GeometryCollection()
geometry.append(point) if values[0]['Coordenada X'] is not None else None
ocr=OccurrenceRecord.objects.filter(created_at__gte='2023-10-11',
                                 created_at__lte='2023-10-12',
                                     company__name='GES - UHE-MIRANDA',
                                    geometry=geometry)

In [38]:
for a in ocr:
    print(a.geometry[0].x,a.geometry[0].y)

-48.023506441429525 -18.900805929539306
-48.023506441429525 -18.900805929539306
-48.023506441429525 -18.900805929539306
-48.023506441429525 -18.900805929539306
-48.023506441429525 -18.900805929539306
-48.023506441429525 -18.900805929539306
-48.023506441429525 -18.900805929539306
-48.023506441429525 -18.900805929539306
-48.023506441429525 -18.900805929539306
-48.023506441429525 -18.900805929539306


In [31]:
rep_edit,erros=[],[]
for a in tqdm(values):
    try:
        point=transform_coordinates(a['Coordenada X'], a['Coordenada Y']) if a['Coordenada X'] is not None else None
        geometry=GeometryCollection()
        geometry.append(point) if a['Coordenada X'] is not None else None
        ocr=OccurrenceRecord.objects.get(created_at__gte='2023-10-11',
                                 created_at__lte='2023-10-12',
                                     company__name='GES - UHE-MIRANDA',
                                    geometry=geometry)
        if a['Coordenada X'] is not None and '23' in str(a['Fuso']).strip() :
            p=transform_coordinates(a['Coordenada X'], a['Coordenada Y'],srid=31983) if a['Coordenada X'] is not None else None
            g=GeometryCollection()
            g.append(p) if a['Coordenada X'] is not None else None
            ocr.geometry = g
        
        ocr.properties=[{"label":a['N. da Invasão']}]

        rep_edit.append(ocr)
    except Exception as e:
        print(e)
        erros.append(a)

  0%|          | 0/1025 [00:00<?, ?it/s]

get() returned more than one OccurrenceRecord -- it returned 10!
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
get() returned more than one OccurrenceRecord -- it returned 2!
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
get() returned more than one OccurrenceRecord -- it returned 2!
get() returned more than one OccurrenceRecord -- it returned 2!
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matching query does not exist.
OccurrenceRecord matchi

In [44]:
len(rep_edit),len(erros2)

(2, 37)

In [43]:
for b in tqdm(rep_edit):
    b.save()

  0%|          | 0/2 [00:00<?, ?it/s]

In [41]:
rep_edit,erros2=[],[]
for i,a in tqdm(enumerate(erros2)):
    try:
        point=transform_coordinates(a['Coordenada X'], a['Coordenada Y']) if a['Coordenada X'] is not None else None
        geometry=GeometryCollection()
        geometry.append(point) if a['Coordenada X'] is not None else None
        ocr=OccurrenceRecord.objects.get(created_at__gte='2023-10-11',
                                 created_at__lte='2023-10-12',
                                     company__name='GES - UHE-MIRANDA',
                                    geometry=geometry,
                                        city=City.objects.get(name=a['Município'].split('/')[0]))
        if a['Coordenada X'] is not None and '23' in str(a['Fuso']).strip() :
            p=transform_coordinates(a['Coordenada X'], a['Coordenada Y'],srid=31983) if a['Coordenada X'] is not None else None
            g=GeometryCollection()
            g.append(p) if a['Coordenada X'] is not None else None
            ocr.geometry = g
        
        ocr.properties=[{"label":a['N. da Invasão']}]

        rep_edit.append(ocr)
    except Exception as e:
        print(e,a['Coordenada X'], a['Coordenada Y'],a['Fuso'],i+2)
        erros2.append(a)

0it [00:00, ?it/s]

get() returned more than one OccurrenceRecord -- it returned 10! 813573 7907508 22 2
OccurrenceRecord matching query does not exist. 184706 7908048 23 3
OccurrenceRecord matching query does not exist. 814842 7906829 22 4
OccurrenceRecord matching query does not exist. 814577 7906094 22 5
OccurrenceRecord matching query does not exist. 814806 7905268 22 6
OccurrenceRecord matching query does not exist. 815615 7904968 22 7
OccurrenceRecord matching query does not exist. 184149 7904970 23 8
OccurrenceRecord matching query does not exist. 184256 7904946 23 9
OccurrenceRecord matching query does not exist. 190404 7890749 23 10
OccurrenceRecord matching query does not exist. 812696 7886816 23 12
OccurrenceRecord matching query does not exist. 814305 7903007 22 13
OccurrenceRecord matching query does not exist. 813438 7902821 22 14
OccurrenceRecord matching query does not exist. 8134118 7902818 22 15
get() returned more than one OccurrenceRecord -- it returned 2! 812972 7902171 22 16
get() re

In [79]:
ocr=OccurrenceRecord.objects.get(number='UHMI-RG-2023.1064')

In [80]:
point=transform_coordinates(185226, 7818142,srid=31983)
geometry=GeometryCollection()
geometry.append(point)
ocr.geometry = geometry
ocr.save()

In [72]:
ocr.geometry == geometry

True

In [54]:
# ocr.properties=[{"label":'UHMI-350E'}]
# ocr.save()

In [15]:
import copy
import json

In [3]:
occ=OccurrenceType.objects.get(name='Leitura de profundidade')

In [22]:
def get_difference(dict1, dict2):
    result = {}
    for key in dict1.keys():
        value = dict1[key]
        if isinstance(value, dict):
            difference = get_difference(value, dict2.get(key, {}))
            if difference:
                result[key] = difference
        elif isinstance(value, list):
            if value != dict2.get(key):
                result[key] = dict2.get(key, None)
            else:
                for i in range(len(value)):
                    if isinstance(value[i], dict):
                        difference = get_difference(value[i], dict2.get(key, [])[i])
                        if difference:
                            if key not in result:
                                result[key] = []
                            result[key].append(difference)
                    elif value[i] != dict2.get(key, [])[i]:
                        if key not in result:
                            result[key] = []
                        result[key].append(dict2.get(key, [])[i])
        elif value != dict2.get(key):
            result[key] = dict2.get(key, None)
    return result



In [24]:
fields=[]
results=[]
for a in occ.history.all().order_by('-history_date'):
    if fields is not None and fields != a.form_fields:
#         results.append({a.history_date:get_difference(fields,a.form_fields['fields'])})
        print('________',a.history_date,'________')
        print(json.dumps(a.form_fields['fields']))
    fields=a.form_fields
    

________ 2022-02-04 17:20:15.359261+00:00 ________
[{"id": 1, "unit": "NA", "apiName": "quota", "autofill": {"if": [{"==": [{"var": "relationships.monitoringPoint.data.id"}, "84b423ae-8c96-42fa-b817-5575d7ed8ffc"]}, 165.049, {"==": [{"var": "relationships.monitoringPoint.data.id"}, "ecd2ad1e-3bb9-4f61-bba1-63e5151372e0"]}, 170.146, {"==": [{"var": "relationships.monitoringPoint.data.id"}, "fa323b34-ebec-4a41-a1c0-16490e027b01"]}, 177.037, {"==": [{"var": "relationships.monitoringPoint.data.id"}, "cf5cc2e5-aec4-4444-a4db-cce5a8337d4a"]}, 170.072, {"==": [{"var": "relationships.monitoringPoint.data.id"}, "8f7aa934-b11b-433f-9d27-b860c08fb60e"]}, 172.658, {"==": [{"var": "relationships.monitoringPoint.data.id"}, "b7d3d154-fb06-4f10-a466-28f534b0653b"]}, 169.489, {"==": [{"var": "relationships.monitoringPoint.data.id"}, "d2609e4d-c723-40e2-be6b-15c0324d295c"]}, 172.866, {"==": [{"var": "relationships.monitoringPoint.data.id"}, "da99f69b-c838-4913-a12a-ad09d07b2475"]}, 167.143, {"==": [{"va

________ 2022-02-04 17:20:15.359261+00:00 ________
[{"id": 1, "unit": "NA", "apiName": "quota", "autofill": {"if": [{"==": [{"var": "relationships.monitoringPoint.data.id"}, "84b423ae-8c96-42fa-b817-5575d7ed8ffc"]}, 165.049, {"==": [{"var": "relationships.monitoringPoint.data.id"}, "ecd2ad1e-3bb9-4f61-bba1-63e5151372e0"]}, 170.146, {"==": [{"var": "relationships.monitoringPoint.data.id"}, "fa323b34-ebec-4a41-a1c0-16490e027b01"]}, 177.037, {"==": [{"var": "relationships.monitoringPoint.data.id"}, "cf5cc2e5-aec4-4444-a4db-cce5a8337d4a"]}, 170.072, {"==": [{"var": "relationships.monitoringPoint.data.id"}, "8f7aa934-b11b-433f-9d27-b860c08fb60e"]}, 172.658, {"==": [{"var": "relationships.monitoringPoint.data.id"}, "b7d3d154-fb06-4f10-a466-28f534b0653b"]}, 169.489, {"==": [{"var": "relationships.monitoringPoint.data.id"}, "d2609e4d-c723-40e2-be6b-15c0324d295c"]}, 172.866, {"==": [{"var": "relationships.monitoringPoint.data.id"}, "da99f69b-c838-4913-a12a-ad09d07b2475"]}, 167.143, {"==": [{"va

In [18]:
MonitoringPoint.objects.get(uuid='84b423ae-8c96-42fa-b817-5575d7ed8ffc')

<MonitoringPoint: [GES - UHE-ESTREITO]: PZ 07>

<MonitoringPoint: [GES - UHE-ESTREITO]: PZ 07>

In [15]:
for a in OccurrenceType.objects.filter(company__name='Energia Compartilhado GSO'):
    for field in a.form_fields['fields']:
        if field.get('dataType') == 'arrayOfObjects' or field.get('data_type') == 'arrayOfObjects':
            if field.get('inner_fields'):
                for inner in field.get('inner_fields'):
                    if field.get('canFilter'):
                        print(field.get('apiName'),a.name)
                    else:
                        print(field.get('apiName'),a.name)
            else:
                for inner in field.get('innerFields'):
                    if field.get('canFilter'):
                        print(field.get('apiName'),a.name)
                    else:
                        print(field.get('apiName'),a.name)
        if field.get('canFilter'):
            print(a)
            

restriction Edificação sociofamiliar
restriction Edificação sociofamiliar
restriction Edificação sociofamiliar
restriction Edificação sociofamiliar
propCultures Edificação sociofamiliar
propCultures Edificação sociofamiliar
propCultures Edificação sociofamiliar
propAnimal Edificação sociofamiliar
propAnimal Edificação sociofamiliar
building Edificação sociofamiliar
building Edificação sociofamiliar
building Edificação sociofamiliar
building Edificação sociofamiliar
building Edificação sociofamiliar
building Edificação sociofamiliar
building Edificação sociofamiliar
building Edificação sociofamiliar
people Edificação sociofamiliar
people Edificação sociofamiliar
people Edificação sociofamiliar
people Edificação sociofamiliar
people Edificação sociofamiliar
people Edificação sociofamiliar
people Edificação sociofamiliar
people Edificação sociofamiliar
people Edificação sociofamiliar
notes Edificação sociofamiliar
additionalContacts Edificação sociofamiliar
additionalContacts Edificação s

In [5]:
OccurrenceType.objects.filter(company__name='Energia Compartilhado GSO')

<QuerySet [<OccurrenceType: Edificação sociofamiliar - ['Energia Compartilhado GSO']>, <OccurrenceType: Conjunto Sonoro - ['Energia Compartilhado GSO']>, <OccurrenceType: Identificação em Área Vulnerável - ['Energia Compartilhado GSO']>, <OccurrenceType: Inventário de danos - ['Energia Compartilhado GSO']>, <OccurrenceType: Edificação de uso coletivo - ['Energia Compartilhado GSO']>, <OccurrenceType: Edificação sem habitação - ['Energia Compartilhado GSO']>, <OccurrenceType: Edificação desmobilizada - ['Energia Compartilhado GSO']>, <OccurrenceType: Rota de fuga - ['Energia Compartilhado GSO']>, <OccurrenceType: Ponto de encontro - ['Energia Compartilhado GSO']>]>

In [3]:
aps=ApprovalStep.objects.get(approval_flow__company__name='Energia Compartilhado',name='Registro em Elaboração')

In [4]:
for a in Reporting.objects.filter(approval_step__isnull=True,company__name='Energia Compartilhado'):
    a.approval_step=aps
    a.save()

Searching KM on direction...
Searching KM without direction...


In [1]:
Contract.objects.get(uuid='e6dd6564-1a51-4f21-89f5-eb0fbd04ee4e')

<Contract: [GES - GEH] e6dd6564-1a51-4f21-89f5-eb0fbd04ee4e: RERU | RERI - Serviços de topografia, levantamento cadastral e elaboração de cartografias (2022-09-20 - 2024-09-21)>

In [3]:
reg=OccurrenceRecord.objects.get(number='UHSS-RG-2023.3780')

In [5]:
reg.delete()

(6,
 {'occurrence_records.OccurrenceRecord_search_tags': 3,
  'files.File': 2,
  'occurrence_records.OccurrenceRecord': 1})

In [9]:
for a in OperationalControl.objects.get(uuid='efc6c6e3-88e6-489b-bc29-ab8609eea7af').history.all():
    print(a.firm)

[d436bd3c-e145-44f2-a5ee-797a67110efd: GES - UHE-ITA] 82b8578c-26d3-405f-8c43-94029fdfa8ed: Homologadores
[d436bd3c-e145-44f2-a5ee-797a67110efd: GES - UHE-ITA] 82b8578c-26d3-405f-8c43-94029fdfa8ed: Homologadores
[d436bd3c-e145-44f2-a5ee-797a67110efd: GES - UHE-ITA] 82b8578c-26d3-405f-8c43-94029fdfa8ed: Homologadores
[d436bd3c-e145-44f2-a5ee-797a67110efd: GES - UHE-ITA] 82b8578c-26d3-405f-8c43-94029fdfa8ed: Homologadores


In [2]:
ui=UserInCompany.objects.get(user__username='mariana.lima')

In [3]:
ui

<UserInCompany: [d436bd3c-e145-44f2-a5ee-797a67110efd: GES - UHE-ITA] mariana.lima>

In [6]:
for a in ui.permissions.history.all():
    print(a.permissions['OccurrenceRecord'])

{'can_edit': True, 'can_view': True, 'queryset': 'self', 'can_create': True, 'can_delete': False, 'can_approve': True, 'can_vg_sync': True, 'disableSubMenus': False, 'canViewInvolvedParts': True, 'can_view_geolocation': True, 'can_create_operational': True, 'canViewHidrologySection': True, 'can_view_involved_parts': True, 'canViewProceduresSection': True, 'can_view_hidrology_section': True, 'can_view_procedures_section': True}
{'can_edit': True, 'can_view': True, 'queryset': 'self', 'can_create': True, 'can_delete': False, 'can_approve': True, 'can_vg_sync': True, 'disableSubMenus': False, 'canViewInvolvedParts': True, 'can_view_geolocation': True, 'canViewHidrologySection': True, 'can_view_involved_parts': True, 'canViewProceduresSection': True, 'can_view_hidrology_section': True, 'can_view_procedures_section': True}
{'can_edit': True, 'can_view': True, 'queryset': 'self', 'can_create': True, 'can_delete': False, 'can_approve': True, 'can_vg_sync': True, 'disableSubMenus': False, 'can

In [16]:
user = User.objects.get(first_name__icontains="dhonata")
user_new=User.objects.get(username='lucas.reis')

In [17]:
ServiceOrderAction.objects.filter(service_order__company__name='GES - UHE-MIRANDA',responsible=user).update(responsible=user_new)

991

In [15]:
soa.count()

991

In [2]:
OccurrenceRecord.objects.get(number='UHIT-RG-2023.6797').delete()

(2,
 {'occurrence_records.OccurrenceRecord_search_tags': 1,
  'occurrence_records.OccurrenceRecord': 1})

In [2]:
import pandas as pd
from tqdm.notebook import tqdm
import re
from django.db.models import Q

In [3]:
filename = 'Dados Engie'
pd.DataFrame().to_excel(filename+'.xlsx')

In [4]:
ocrs=OccurrenceRecord.objects.all().values('company__name','created_at__month','created_at__year').annotate(qtd_apt=Count('uuid')).order_by('company__name')

In [5]:
ocrs

<QuerySet [{'company__name': 'ATIVO-TREINAMENTO', 'created_at__month': 1, 'created_at__year': 2019, 'qtd_apt': 4}, {'company__name': 'ATIVO-TREINAMENTO', 'created_at__month': 1, 'created_at__year': 2020, 'qtd_apt': 9}, {'company__name': 'ATIVO-TREINAMENTO', 'created_at__month': 2, 'created_at__year': 2019, 'qtd_apt': 6}, {'company__name': 'ATIVO-TREINAMENTO', 'created_at__month': 2, 'created_at__year': 2020, 'qtd_apt': 1}, {'company__name': 'ATIVO-TREINAMENTO', 'created_at__month': 3, 'created_at__year': 2019, 'qtd_apt': 1}, {'company__name': 'ATIVO-TREINAMENTO', 'created_at__month': 3, 'created_at__year': 2020, 'qtd_apt': 8}, {'company__name': 'ATIVO-TREINAMENTO', 'created_at__month': 4, 'created_at__year': 2019, 'qtd_apt': 2}, {'company__name': 'ATIVO-TREINAMENTO', 'created_at__month': 4, 'created_at__year': 2020, 'qtd_apt': 6}, {'company__name': 'ATIVO-TREINAMENTO', 'created_at__month': 5, 'created_at__year': 2019, 'qtd_apt': 3}, {'company__name': 'ATIVO-TREINAMENTO', 'created_at__m

In [6]:
df = pd.DataFrame(data=ocrs,columns=['company__name','created_at__month','created_at__year','qtd_apt'])
df.columns=['Usina','Mês','Ano','Quantidade de Registros']
df

,Usina,Mês,Ano,Quantidade de Registros
0,ATIVO-TREINAMENTO,1,2019,4
1,ATIVO-TREINAMENTO,1,2020,9
2,ATIVO-TREINAMENTO,2,2019,6
3,ATIVO-TREINAMENTO,2,2020,1
4,ATIVO-TREINAMENTO,3,2019,1
...,...,...,...,...
536,GSO - UTPS,7,2023,5
537,Limbo - UTPS,4,2023,90
538,POC-Projeto APR,5,2020,6
539,POC-Projeto APR,6,2020,1


In [7]:
df['Usina_inicio'] = df['Usina'].str.slice(stop=3)
dfSomaMesUsina = df.groupby(["Ano",'Mês','Usina_inicio'])['Quantidade de Registros'].sum()
df,dfSomaMesUsina

(                 Usina  Mês   Ano  Quantidade de Registros Usina_inicio
 0    ATIVO-TREINAMENTO    1  2019                        4          ATI
 1    ATIVO-TREINAMENTO    1  2020                        9          ATI
 2    ATIVO-TREINAMENTO    2  2019                        6          ATI
 3    ATIVO-TREINAMENTO    2  2020                        1          ATI
 4    ATIVO-TREINAMENTO    3  2019                        1          ATI
 ..                 ...  ...   ...                      ...          ...
 536         GSO - UTPS    7  2023                        5          GSO
 537       Limbo - UTPS    4  2023                       90          Lim
 538    POC-Projeto APR    5  2020                        6          POC
 539    POC-Projeto APR    6  2020                        1          POC
 540  RECURSOS HIDRICOS   11  2023                       13          REC
 
 [541 rows x 5 columns],
 Ano   Mês  Usina_inicio
 2018  10   ATI                 7
       12   ATI                 2
    

In [8]:
df_soma = df.groupby('Usina_inicio')['Quantidade de Registros'].sum()
df_soma

Usina_inicio
ATI       166
GES     42379
GSB    372369
GSO     11742
Lim        90
POC         7
REC        13
Name: Quantidade de Registros, dtype: int64

In [9]:
df2 = pd.DataFrame(data=df_soma)
df_final_qtd_registros = df2.transpose()[['GES','GSB','GSO','REC']]
df_final_qtd_registros = df_final_qtd_registros.transpose()
df_final_qtd_registros

,Quantidade de Registros
Usina_inicio,
GES,42379
GSB,372369
GSO,11742
REC,13


In [10]:
df_mes_soma = pd.DataFrame(data=dfSomaMesUsina)
df_mes_soma = df_mes_soma.reset_index()
df_mes_soma_v2 = df_mes_soma[df_mes_soma['Usina_inicio'].isin(['GES','GSB','GSO','REC'])].set_index(['Ano','Usina_inicio'])
df_mes_soma_v2

Mês  Quantidade de Registros
Ano  Usina_inicio                              
2018 GES            12                        1
2019 GES             1                       26
     GES             2                       18
     GES             3                        5
     GES             4                        5
...                ...                      ...
2023 REC            11                       13
     GES            12                      355
     GSB            12                    23727
2024 GES             1                       13
     GSB             1                     5649

[82 rows x 2 columns]

In [11]:
aclog=ActionLog.objects.filter(
    Q(company__name__startswith='GES')|Q(company__name__startswith='GSB')|Q(company__name__startswith='GSO')|Q(company__name__startswith='REC')
).values_list('company__name','created_at__month','created_at__year','action').annotate(qtd_atividade=Count('uuid')).order_by('company__name')

In [12]:
ActionLog._meta.fields,aclog

((<django.db.models.fields.UUIDField: uuid>,
  <django.db.models.fields.related.ForeignKey: company>,
  <django.db.models.fields.related.ForeignKey: company_group>,
  <django.db.models.fields.related.ForeignKey: user>,
  <django.db.models.fields.DateTimeField: created_at>,
  <django.db.models.fields.TextField: action>,
  <django.db.models.fields.TextField: user_ip>,
  <django.db.models.fields.TextField: user_port>,
  <django.db.models.fields.TextField: user_agent>,
  <django.db.models.fields.related.ForeignKey: content_type>,
  <django.db.models.fields.UUIDField: object_id>),
 <QuerySet [('GES - GEH', 1, 2021, 'Create', 1), ('GES - GEH', 1, 2022, 'Update', 1), ('GES - GEH', 1, 2023, 'Create', 2), ('GES - GEH', 1, 2023, 'Update', 5), ('GES - GEH', 2, 2023, 'Create', 66), ('GES - GEH', 2, 2023, 'Update', 57), ('GES - GEH', 3, 2022, 'Create', 41), ('GES - GEH', 3, 2022, 'Delete', 5), ('GES - GEH', 3, 2022, 'Update', 62), ('GES - GEH', 3, 2023, 'Create', 19), ('GES - GEH', 3, 2023, 'Delete

In [13]:
df_action=pd.DataFrame(data=aclog)
df_action.rename(columns={0: 'Usina',1:'Mês',2:'Ano',3: 'Ação',4: 'Qtd por Ação'}, inplace=True)
df_action['Ação']=df_action['Ação'].str.replace('Create','Criação')
df_action['Ação']=df_action['Ação'].str.replace('Delete','Deleção')
df_action['Ação']=df_action['Ação'].str.replace('Update','Atualização')
df_action

,Usina,Mês,Ano,Ação,Qtd por Ação
0,GES - GEH,1,2021,Criação,1
1,GES - GEH,1,2022,Atualização,1
2,GES - GEH,1,2023,Criação,2
3,GES - GEH,1,2023,Atualização,5
4,GES - GEH,2,2023,Criação,66
...,...,...,...,...,...
1387,GSO - UTPS,7,2023,Atualização,1
1388,GSO - UTPS,8,2023,Atualização,2
1389,GSO - UTPS,10,2023,Atualização,3
1390,RECURSOS HIDRICOS,1,2024,Atualização,1


In [14]:
df_action['Usina_inicio'] = df_action['Usina'].str.slice(stop=3)
df_action_soma = df_action.groupby('Usina_inicio')['Qtd por Ação'].sum()
df_action_soma_anomes = df_action.groupby(['Ano','Mês','Usina_inicio'])['Qtd por Ação'].sum()
df_action_soma_anomes

Ano   Mês  Usina_inicio
2020  8    GES              969
      9    GES             3181
      10   GES             3066
      11   GES             2402
           GSO                4
                           ... 
2023  12   GES             8147
           GSB              118
           GSO               14
2024  1    GES             1036
           REC                1
Name: Qtd por Ação, Length: 71, dtype: int64

In [15]:
df2_action_soma_anomes = pd.DataFrame(data=df_action_soma_anomes)
df2_action_soma_anomes

Qtd por Ação
Ano  Mês Usina_inicio              
2020 8   GES                    969
     9   GES                   3181
     10  GES                   3066
     11  GES                   2402
         GSO                      4
...                             ...
2023 12  GES                   8147
         GSB                    118
         GSO                     14
2024 1   GES                   1036
         REC                      1

[71 rows x 1 columns]

In [16]:
df2_action_soma = pd.DataFrame(data=df_action_soma)
df_final_action_soma = df2_action_soma.transpose()[['GES','GSB','GSO','REC']]
df2_action_soma.rename(columns={'Qtd por Ação':'Total de ações no sistema por usina'},inplace=True)
df2_action_soma

,Total de ações no sistema por usina
Usina_inicio,
GES,170895
GSB,988
GSO,1082
REC,2


In [17]:
occs = OccurrenceType.objects.all().values('company__name').annotate(qtd_forms=Count('uuid')).distinct()
df_forms=pd.DataFrame(data=occs)
df_forms['Usina_inicio'] = df_forms['company__name'].str.slice(stop=3)
df_forms_soma = df_forms.groupby('Usina_inicio')['qtd_forms'].sum()
df2_forms_soma = pd.DataFrame(data=df_forms_soma)
df2_final_forms_soma = df2_forms_soma.transpose()[['GES','GSB','GSO','REC']]
df2_final_forms_soma=df2_final_forms_soma.transpose()
df2_final_forms_soma.rename(columns={'qtd_forms':'Quantidade de Formularios por Usina'},inplace=True)
df2_final_forms_soma

,Quantidade de Formularios por Usina
Usina_inicio,
GES,733
GSB,499
GSO,125
REC,2


In [18]:
with pd.ExcelWriter(filename+'t.xlsx', mode='w',engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name='Qtd Registro por Usina', index=False)
    df_final_qtd_registros.to_excel(writer, sheet_name='Qtd Reg por Subsis', index=True)
    df_mes_soma_v2.to_excel(writer, sheet_name='Qtd-Reg-Ano-Mes-Subsis', index=True)
    df_action.to_excel(writer, sheet_name='Atividades por Usina', index=False)
    df2_action_soma.to_excel(writer, sheet_name='Total Ações por Subsis', index=True)
    df2_action_soma_anomes.to_excel(writer, sheet_name='Total-Ações-Ano-Mes-Subsis', index=True)
    df2_final_forms_soma.to_excel(writer, sheet_name='Total Form por Subsis', index=True)

In [1]:
(OccurrenceType.objects.filter(company__name__startswith='GES').distinct().count(),
OccurrenceType.objects.filter(company__name__startswith='GSB').distinct().count(),
OccurrenceType.objects.filter(company__name__startswith='GSO').distinct().count(),
OccurrenceType.objects.filter(company__name__startswith='REC').distinct().count())

(109, 164, 9, 2)

In [1]:
user=User.objects.get(uuid='eb44f871-cd7b-4f55-94bd-6af25fadfa70')

In [5]:
for a in user.history.all():
    print(a.saml_nameid)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [5]:
# user=User.objects.get(username='SR5378')
contract=Contract.objects.get(uuid='1ec6ac28-0c4b-454f-ba85-ce56f3251fdc')
contract

<Contract: [] 1ec6ac28-0c4b-454f-ba85-ce56f3251fdc: Programa de Educação Ambiental e Visitas (2023-12-04 - 2025-12-04)>

In [44]:
pr=ProcedureResource.objects.filter(
#     service_order=ServiceOrder.objects.get(number='UHIT-OS-2021.0012'),
#                                     created_by=user,
                                   service_order_resource__contract=contract,
                                   creation_date__month='1',creation_date__day='25',
    resource__name__icontains='Execução dos Serviços em Educação Ambiental'
).order_by('approval_date')
pr.count(),pr[0]

(1,
 <ProcedureResource: [GES - UHE-PASSO FUNDO] - Execução dos Serviços em Educação Ambiental: 0.0 Un. - 1.0 Un.>)

In [45]:
pr_get=pr.first()
pr_get.approval_status = "WAITING_APPROVAL"
pr_get.save()

'NoneType' object has no attribute 'company'


In [26]:
pr_get.measurement_bulletin

In [14]:
pr_get.resource.__dict__

{'_state': <django.db.models.base.ModelState at 0x7efd835c6c90>,
 'uuid': UUID('b3a69684-e8f8-4676-82f8-65797d05e26a'),
 'company_id': UUID('574400b8-28b9-4a59-b29f-9d542e42fbdc'),
 'name': 'Execução dos Serviços em Educação Ambiental',
 'total_amount': 0.0,
 'unit': 'Un.',
 'is_extra': True,
 'created_by_id': None}

In [35]:
for a in pr:
    print(a,a.creation_date,a.approval_date,a.approved_by.username,a.created_by.username)

[GES - UHE-ITA] - 1.2 IQAR: 0.0 un - 1.0 un 2023-12-06 18:12:54.940798+00:00 2023-12-06 18:20:35.991632+00:00 gilnei.minella SR5378


In [2]:
ProcedureResource._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: procedure>,
 <django.db.models.fields.related.ForeignKey: service_order>,
 <django.db.models.fields.related.ForeignKey: resource>,
 <django.db.models.fields.related.ForeignKey: service_order_resource>,
 <django.db.models.fields.related.ForeignKey: created_by>,
 <django.db.models.fields.FloatField: amount>,
 <django.db.models.fields.FloatField: unit_price>,
 <django.db.models.fields.FloatField: total_price>,
 <django.db.models.fields.DateTimeField: creation_date>,
 <django.db.models.fields.CharField: approval_status>,
 <django.db.models.fields.DateTimeField: approval_date>,
 <django.db.models.fields.related.ForeignKey: approved_by>,
 <django.db.models.fields.related.ForeignKey: measurement_bulletin>,
 <django.db.models.fields.related.ForeignKey: firm>,
 <django.db.models.fields.related.ForeignKey: reporting>)

In [4]:
Resource._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: company>,
 <django.db.models.fields.TextField: name>,
 <django.db.models.fields.FloatField: total_amount>,
 <django.db.models.fields.CharField: unit>,
 <django.db.models.fields.BooleanField: is_extra>,
 <django.db.models.fields.related.ForeignKey: created_by>)

In [7]:
ServiceOrderResource._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: contract>,
 <django.db.models.fields.related.ForeignKey: resource>,
 <django.db.models.fields.FloatField: amount>,
 <django.db.models.fields.FloatField: unit_price>,
 <django.db.models.fields.FloatField: used_price>,
 <django.db.models.fields.FloatField: remaining_amount>,
 <django.db.models.fields.DateTimeField: creation_date>,
 <django.db.models.fields.DateTimeField: effective_date>,
 <django.db.models.fields.related.ForeignKey: created_by>,
 <django.db.models.fields.CharField: resource_kind>,
 <django.db.models.fields.related.ForeignKey: entity>,
 <django.db.models.fields.related.ForeignKey: additional_control_model>,
 <django.db.models.fields.CharField: additional_control>)

In [2]:
mb=MeasurementBulletin.objects.get(number='UHPF-BM-2024.0002')

In [27]:
mb.delete()

(1, {'service_orders.MeasurementBulletin': 1})

In [25]:
mb.save()

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-01-28 22:41:34.937078) while time zone support is active.
  RuntimeWarning)


In [20]:
mb.approval_step

<ApprovalStep: [GES - UHE-PASSO FUNDO]: Aprovado>

In [22]:
mb.__dict__

{'_state': <django.db.models.base.ModelState at 0x7efd83d1bd50>,
 'uuid': None,
 'number': 'UHPF-BM-2024.0002',
 'firm_id': None,
 'firm_manager_id': None,
 'creation_date': datetime.datetime(2024, 1, 29, 13, 19, 1, 58084, tzinfo=<UTC>),
 'measurement_date': datetime.datetime(2024, 1, 25, 14, 42, 20, 607000, tzinfo=<UTC>),
 'created_by_id': UUID('3b3bae79-efd6-4f8b-a70b-b87fa72fc274'),
 'total_price': 30500.0,
 'contract_id': UUID('1ec6ac28-0c4b-454f-ba85-ce56f3251fdc'),
 'extra_info': {'accounting_classification': '010-615011121013-01054061-2514010-1103-0-0'},
 'description': 'UHPF-BM-2024.0001',
 'approval_step_id': UUID('d3d7a17f-7eb1-4fb2-9345-a32c55933c00'),
 'editable': False,
 'period_starts_at': None,
 'period_ends_at': None,
 'work_day': None,
 '_fieldsignals_originals': {(139627780650320,
   139627308363216): {'uuid': UUID('d24d502c-1cef-4138-bbec-40a10812f1b2'), 'number': 'UHPF-BM-2024.0002', 'creation_date': datetime.datetime(2024, 1, 29, 13, 19, 1, 58084, tzinfo=<UTC>), 'm

In [28]:
ContractItemUnitPrice._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.DateTimeField: created_at>,
 <django.db.models.fields.TextField: sort_string>,
 <django.db.models.fields.related.ForeignKey: entity>,
 <django.db.models.fields.related.ForeignKey: resource>)

In [46]:
ci=ContractItemUnitPrice.objects.get(uuid='637bbe27-e2bb-496f-ad85-67412fc29fa6')

In [48]:
ci.resource.__dict__

{'_state': <django.db.models.base.ModelState at 0x7efd7eb41d90>,
 'uuid': UUID('0d686ebb-9f59-49a2-bd6e-f82d94743668'),
 'contract_id': UUID('1ec6ac28-0c4b-454f-ba85-ce56f3251fdc'),
 'resource_id': UUID('b3a69684-e8f8-4676-82f8-65797d05e26a'),
 'amount': 24.0,
 'unit_price': 15500.0,
 'used_price': 0.0,
 'remaining_amount': 24.0,
 'creation_date': datetime.datetime(2024, 1, 25, 14, 35, 38, 867208, tzinfo=<UTC>),
 'effective_date': datetime.datetime(2024, 1, 25, 14, 35, 38, 865917, tzinfo=<UTC>),
 'created_by_id': None,
 'resource_kind': '',
 'entity_id': UUID('628b7a4b-d172-4ec7-97f7-2e28a2dce395'),
 'additional_control_model_id': None,
 'additional_control': '',
 '_ServiceOrderResource__original_unit_price': 15500.0}

In [38]:
ci.resource.remaining_amount = 24

In [40]:
ci.resource.used_price = 0

In [42]:
ci.resource.__dict__

{'_state': <django.db.models.base.ModelState at 0x7efd7f3b3ed0>,
 'uuid': UUID('0d686ebb-9f59-49a2-bd6e-f82d94743668'),
 'contract_id': UUID('1ec6ac28-0c4b-454f-ba85-ce56f3251fdc'),
 'resource_id': UUID('b3a69684-e8f8-4676-82f8-65797d05e26a'),
 'amount': 24.0,
 'unit_price': 15500.0,
 'used_price': 0,
 'remaining_amount': 24,
 'creation_date': datetime.datetime(2024, 1, 25, 14, 35, 38, 867208, tzinfo=<UTC>),
 'effective_date': datetime.datetime(2024, 1, 25, 14, 35, 38, 865917, tzinfo=<UTC>),
 'created_by_id': None,
 'resource_kind': '',
 'entity_id': UUID('628b7a4b-d172-4ec7-97f7-2e28a2dce395'),
 'additional_control_model_id': None,
 'additional_control': '',
 '_ServiceOrderResource__original_unit_price': 15500.0}

In [43]:
# ci.save()

In [4]:
ServiceOrder._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: company>,
 <django.db.models.fields.CharField: number>,
 <django.db.models.fields.TextField: description>,
 <django.db.models.fields.related.ForeignKey: created_by>,
 <django.db.models.fields.CharField: kind>,
 <django.db.models.fields.CharField: process_type>,
 <django.db.models.fields.CharField: shape_file_property>,
 <django.contrib.postgres.fields.array.ArrayField: uf_code>,
 <django.contrib.postgres.fields.array.ArrayField: place_on_dam>,
 <django.db.models.fields.TextField: other_reference>,
 <django.db.models.fields.DateTimeField: opened_at>,
 <django.db.models.fields.DateTimeField: updated_at>,
 <django.db.models.fields.DateTimeField: closed_at>,
 <django.db.models.fields.BooleanField: is_closed>,
 <django.db.models.fields.TextField: closed_description>,
 <django.db.models.fields.related.ForeignKey: closed_by>,
 <django.contrib.postgres.fields.jsonb.JSONField: priority>,
 <django.contrib.p

In [4]:
user=User.objects.get(uuid='fdd8761a-0f32-47da-8122-e23db29c0ae2')
companies=Company.objects.filter(name__in=['GES - UHE-MIRANDA','GES - UHE-JAGUARA'])
so=ServiceOrder.objects.filter(company__in=companies)
user,companies,so.count()

(<User: ronivaldo.silva - Ronivaldo José da Silva>,
 <QuerySet [<Company: 17663a30-5f71-4e7c-8ac2-adf472342b42: GES - UHE-JAGUARA>, <Company: 2fd0ca76-b741-47d7-af9c-607e6ed1eca2: GES - UHE-MIRANDA>]>,
 1485)

In [5]:
from tqdm.notebook import tqdm
for a in tqdm(so):
    if a.kind == 'LAND': # Natureza Patrimonial
        a.managers.add(user) # Gerir entregas
        a.responsibles.add(user) # Responsáveis

  0%|          | 0/1485 [00:00<?, ?it/s]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:11:58.433980) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:11:59.098766) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:11:59.168789) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTi

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:12:03.172177) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:12:03.413235) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:12:03.477290) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTi

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:12:07.227396) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:12:07.293622) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:12:07.474691) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTi

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:12:11.121370) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:12:11.192641) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:12:11.326572) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTi

...

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:17:14.263915) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:17:14.422476) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:17:14.506163) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTi

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:17:17.760856) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:17:17.832977) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:17:17.995327) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTi

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:17:21.117527) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:17:21.201422) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:17:21.344083) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTi

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:17:24.388508) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:17:24.532673) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:17:24.607731) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTi

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:17:27.864975) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:17:27.946590) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField ToDo.created_at received a naive datetime (2024-02-21 02:17:28.097192) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTi

In [1]:
AccessRequest._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: user>,
 <django.db.models.fields.related.ForeignKey: company>,
 <django.db.models.fields.DateField: expiration_date>,
 <django.db.models.fields.DateTimeField: created_at>,
 <django.db.models.fields.TextField: description>,
 <django.db.models.fields.BooleanField: approved>,
 <django.db.models.fields.BooleanField: done>,
 <django.db.models.fields.related.ForeignKey: permissions>,
 <django.db.models.fields.related.ForeignKey: created_by>,
 <django.db.models.fields.related.ForeignKey: approval_step>)

In [63]:
AccessRequest.objects.get(uuid='81ff3349-e786-49d6-a336-392f66cbe71a').created_by.get_full_name()

'Tulio Ribeiral'

In [4]:
AccessRequest.objects.get(uuid='7264d930-d3e2-4f6e-be35-1141ebaad78d').__dict__

{'_state': <django.db.models.base.ModelState at 0x7f0610b37890>,
 'uuid': UUID('7264d930-d3e2-4f6e-be35-1141ebaad78d'),
 'user_id': UUID('1754675f-d5de-4586-9556-858b52ac93f4'),
 'company_id': UUID('d436bd3c-e145-44f2-a5ee-797a67110efd'),
 'expiration_date': None,
 'created_at': datetime.datetime(2023, 12, 29, 16, 57, 40, 388914, tzinfo=<UTC>),
 'description': 'Gerente regional, acesso gerencial',
 'approved': True,
 'done': True,
 'permissions_id': UUID('18435388-c318-487c-bb9b-37c8bf80dfdf'),
 'created_by_id': UUID('df3e437a-8a19-4a0d-8632-99bdd573e7fb'),
 'approval_step_id': UUID('73d0c3a8-b819-49e8-bd54-433fd342581f'),
 '_fieldsignals_originals': {(139664364946576,
   139664059349904): {'uuid': UUID('7264d930-d3e2-4f6e-be35-1141ebaad78d'), 'user': UUID('1754675f-d5de-4586-9556-858b52ac93f4'), 'company': UUID('d436bd3c-e145-44f2-a5ee-797a67110efd'), 'created_at': datetime.datetime(2023, 12, 29, 16, 57, 40, 388914, tzinfo=<UTC>), 'description': 'Gerente regional, acesso gerencial', '

In [58]:
from tqdm.notebook import tqdm
obj={}
for a in tqdm(AccessRequest.objects.filter(done=False).exclude(created_by__username='engie').prefetch_related('permissions','company','user')):
    if not UserInCompany.objects.filter(user=a.user,company=a.company).exists():
#         if (a.permissions is None or a.company not in a.permissions.companies.all()):
        if a.user.username not in obj:
            obj[a.user.username]={}

        obj[a.user.username]['Nome Completo'] = a.user.get_full_name()
        obj[a.user.username]['Criado Por'] = a.created_by.get_full_name()
        obj[a.user.username]['Criado Em'] = a.created_at.strftime('%d/%m/%Y')
        obj[a.user.username]['Permissão'] = a.permissions.name if a.permissions is not None else 'Sem Permissão'
        obj[a.user.username]['Concluida'] = "Sim" if a.done == True else "Não"
        obj[a.user.username]['Aprovada'] = "Sim" if a.approved == True else "Não"
        obj[a.user.username]['Usina'] = a.company.name
#             print(a.user.username, a.permissions.name if a.permissions is not None else 'Sem Permissão',a.company.name)

  0%|          | 0/263 [00:00<?, ?it/s]

In [60]:
obj

{'tulio.ribeiral': {'Nome Completo': 'Tulio Ribeiral',
  'Criado Por': 'Cid Ionceck',
  'Criado Em': '18/06/2020',
  'Permissão': 'GES - HOMOLOGADOR',
  'Concluida': 'Não',
  'Aprovada': 'Não',
  'Usina': 'ATIVO-TREINAMENTO'},
 'DF5705': {'Nome Completo': 'Carlos Klimaszewski',
  'Criado Por': 'Cid Ionceck',
  'Criado Em': '19/08/2020',
  'Permissão': 'GES - COLABORADOR INTERNO',
  'Concluida': 'Não',
  'Aprovada': 'Não',
  'Usina': 'GES - UHE-MACHADINHO'},
 'ZF5232': {'Nome Completo': 'Cristiano Pessoa Dos Santos',
  'Criado Por': 'Cid Ionceck',
  'Criado Em': '19/08/2020',
  'Permissão': 'GES - COLABORADOR INTERNO',
  'Concluida': 'Não',
  'Aprovada': 'Não',
  'Usina': 'GES - UHE-ITA'},
 'acl414': {'Nome Completo': 'Johnny Rafael Lang',
  'Criado Por': 'Cid Ionceck',
  'Criado Em': '05/11/2022',
  'Permissão': 'GES - LÍDER TÉCNICO E COMERCIAL EXTERNO',
  'Concluida': 'Não',
  'Aprovada': 'Não',
  'Usina': 'GES - GEH'},
 'micheledepaula': {'Nome Completo': 'Michele de Paula',
  'Criad

In [61]:
import pandas as pd
df= pd.DataFrame(data=obj)

In [62]:
df = df.transpose()

In [63]:
df.to_excel('solicitacoesAcessoEngie.xlsx')
df

,Nome Completo,Criado Por,Criado Em,Permissão,Concluida,Aprovada,Usina
tulio.ribeiral,Tulio Ribeiral,Cid Ionceck,18/06/2020,GES - HOMOLOGADOR,Não,Não,ATIVO-TREINAMENTO
DF5705,Carlos Klimaszewski,Cid Ionceck,19/08/2020,GES - COLABORADOR INTERNO,Não,Não,GES - UHE-MACHADINHO
ZF5232,Cristiano Pessoa Dos Santos,Cid Ionceck,19/08/2020,GES - COLABORADOR INTERNO,Não,Não,GES - UHE-ITA
acl414,Johnny Rafael Lang,Cid Ionceck,05/11/2022,GES - LÍDER TÉCNICO E COMERCIAL EXTERNO,Não,Não,GES - GEH
micheledepaula,Michele de Paula,Carlos Eduardo Alves Dos Santos,11/02/2021,GES - LÍDER TÉCNICO E COMERCIAL EXTERNO,Não,Não,GES - PCH-RONDONÓPOLIS
...,...,...,...,...,...,...,...
kellysoncosta,Kellyson Matheus Santos Costa,Shara Pereira Bartkow,28/10/2022,GES - COLABORADOR EXTERNO,Não,Não,GES - UHE-ESTREITO
rasdonysousa,Rasdony Klaiver Figueredo Sousa,Shara Pereira Bartkow,28/10/2022,GES - COLABORADOR EXTERNO,Não,Não,GES - UHE-ESTREITO
lucasoliveira,Lucas Cruz Oliveira,Shara Pereira Bartkow,28/10/2022,GES - COLABORADOR EXTERNO,Não,Não,GES - UHE-ESTREITO
LM6155,Erica Da Cruz De Oliveira,Cid Ionceck,07/07/2023,GES - LIDER TÉCNICO INTERNO,Não,Não,GES - UHE-PONTE DE PEDRA


In [53]:
per=UserPermission.objects.all().values('name','companies__name').order_by('companies__name')
per

<QuerySet [{'name': 'GES - COLABORADOR INTERNO', 'companies__name': 'ATIVO-TREINAMENTO'}, {'name': 'GES - ANALISTA DE PROCESSOS', 'companies__name': 'ATIVO-TREINAMENTO'}, {'name': 'GES - LIDER TÉCNICO EXTERNO', 'companies__name': 'ATIVO-TREINAMENTO'}, {'name': 'GES - LÍDER TÉCNICO E COMERCIAL CONSÓRCIO', 'companies__name': 'ATIVO-TREINAMENTO'}, {'name': 'GES - HOMOLOGADOR', 'companies__name': 'ATIVO-TREINAMENTO'}, {'name': 'GES - GERENCIAL', 'companies__name': 'ATIVO-TREINAMENTO'}, {'name': 'GES - LIDER COMERCIAL EXTERNO', 'companies__name': 'ATIVO-TREINAMENTO'}, {'name': 'GES - COLABORADOR EXTERNO', 'companies__name': 'ATIVO-TREINAMENTO'}, {'name': 'GES - LÍDER TÉCNICO E COMERCIAL EXTERNO', 'companies__name': 'ATIVO-TREINAMENTO'}, {'name': 'INATIVO', 'companies__name': 'ATIVO-TREINAMENTO'}, {'name': 'GES - LIDER COMERCIAL INTERNO', 'companies__name': 'ATIVO-TREINAMENTO'}, {'name': 'GES - ADMINISTRADOR', 'companies__name': 'ATIVO-TREINAMENTO'}, {'name': 'GES - LIDER TÉCNICO INTERNO', '

In [55]:
df2=pd.DataFrame(data=per)
df2.to_excel('listaSuspensa.xlsx')

In [62]:
df2.set_index('companies__name')

,name
companies__name,
ATIVO-TREINAMENTO,INATIVO
ATIVO-TREINAMENTO,GES - LÍDER TÉCNICO E COMERCIAL CONSÓRCIO
ATIVO-TREINAMENTO,GES - ADMINISTRADOR
ATIVO-TREINAMENTO,GES - HOMOLOGADOR
ATIVO-TREINAMENTO,GES - GERENCIAL
...,...
POC-Projeto APR,LIDER COMERCIAL EXTERNO
POC-Projeto APR,LIDER COMERCIAL INTERNO
POC-Projeto APR,LIDER TÉCNICO INTERNO


In [3]:
ar=AccessRequest.objects.get(uuid='66f47720-5bb4-4791-86da-c377810e70e3')

In [6]:
ar.created_at,ar.__dict__

(datetime.datetime(2019, 12, 20, 18, 32, 7, 795748, tzinfo=<UTC>),
 {'_state': <django.db.models.base.ModelState at 0x7f1ae9f6ad90>,
  'uuid': UUID('66f47720-5bb4-4791-86da-c377810e70e3'),
  'user_id': UUID('1104c685-2deb-41f0-8b0d-e0da4722fad6'),
  'company_id': UUID('d436bd3c-e145-44f2-a5ee-797a67110efd'),
  'expiration_date': datetime.date(2020, 7, 1),
  'created_at': datetime.datetime(2019, 12, 20, 18, 32, 7, 795748, tzinfo=<UTC>),
  'description': 'Realizar atendimento de tarefas em serviços',
  'approved': True,
  'done': True,
  'permissions_id': None,
  'created_by_id': UUID('df3e437a-8a19-4a0d-8632-99bdd573e7fb'),
  'approval_step_id': None,
  '_fieldsignals_originals': {(139753905734672,
    139753600138128): {'uuid': UUID('66f47720-5bb4-4791-86da-c377810e70e3'), 'user': UUID('1104c685-2deb-41f0-8b0d-e0da4722fad6'), 'company': UUID('d436bd3c-e145-44f2-a5ee-797a67110efd'), 'expiration_date': datetime.date(2020, 7, 1), 'created_at': datetime.datetime(2019, 12, 20, 18, 32, 7, 79

In [1]:
for a in UserPermission.history.filter(uuid='15541baa-6750-4851-bf65-4f8722bcdfa5'):
    print(a.history_date,a.name,a.history_user,a.history_type)

2023-09-19 20:41:52.194579+00:00 GSO - ADMINISTRADOR engie - Suporte Kartado ~
2023-09-05 20:10:41.164127+00:00 GSO - ADMINISTRADOR None ~
2023-08-21 17:35:38.540119+00:00 ADMINISTRADOR engie - Suporte Kartado ~
2023-08-11 17:52:33.568543+00:00 ADMINISTRADOR engie - Suporte Kartado ~
2022-11-24 09:37:21.071684+00:00 ADMINISTRATOR None ~
2022-11-23 11:47:43.086035+00:00 ADMINISTRATOR None ~
2021-03-09 18:05:25.002491+00:00 ADMINISTRATOR None ~
2021-03-09 18:05:24.984209+00:00 ADMINISTRATOR None ~
2021-03-09 12:40:41.636304+00:00 ADMINISTRATOR None ~
2021-03-09 12:40:41.615840+00:00 ADMINISTRATOR None ~
2020-12-07 04:04:55.040936+00:00 ADMINISTRATOR None ~
2020-12-07 04:04:09.810795+00:00 ADMINISTRATOR None ~
2020-12-07 03:59:52.081753+00:00 ADMINISTRATOR None ~
2020-12-07 03:59:10.362480+00:00 ADMINISTRATOR None ~
2020-12-07 03:55:15.236116+00:00 ADMINISTRATOR None ~
2020-06-23 02:18:31.156785+00:00 ADMINISTRATOR None ~
2020-06-23 02:10:38.398658+00:00 ADMINISTRATOR None ~
2020-06-23 01

In [16]:
for a in AccessRequest.history.filter(uuid='66f47720-5bb4-4791-86da-c377810e70e3').only('permissions'):
    print(a.permissions_id,a.history_date)

d7d23043-a6f9-43ba-aaa0-276816341e02 2019-12-20 18:51:05.945140+00:00
d7d23043-a6f9-43ba-aaa0-276816341e02 2019-12-20 18:51:01.470106+00:00
None 2019-12-20 18:32:07.800212+00:00


In [17]:
UserPermission.history.filter(uuid='d7d23043-a6f9-43ba-aaa0-276816341e02')

<QuerySet []>

In [14]:
pe=UserPermission.objects.filter(name__icontains="Administrador",companies__name__startswith="GSO").distinct()
user=User.objects.get(username='lais.centenaro')
perm = pe[0]
perm,user

(<UserPermission: ['GSO - UHE-CANA BRAVA', 'GSO - UHE-SALTO OSORIO', 'GSO - UTPS', 'GSO - UHE-SALTO SANTIAGO', 'GSO - UHE-PONTE DE PEDRA', 'GSO - UHE-JAGUARA', 'GSO - UHE-MIRANDA', 'GSO - PCH-JOSÉ GELÁZIO DA ROCHA', 'GSO - PCH-RONDONÓPOLIS', 'GSO - UHE-ESTREITO', 'GSO - UHE-SÃO SALVADOR', 'GSO - UHE-PASSO FUNDO', 'GSO - UHE-ITA', 'GSO - UHE-MACHADINHO'] - GSO - ADMINISTRADOR>,
 <User: lais.centenaro - Laís Caroline Centenaro>)

In [19]:
for a in Company.objects.filter(name__in=['GSO - UHE-ITA','GSO - UHE-PASSO FUNDO','GSO - UHE-MACHADINHO']):
    uic=UserInCompany.objects.filter(user=user,company=a)
    if uic.exists():
        u=uic.first()
        u.permissions=perm
        u.save()
    else:
        up=UserInCompany(user=user,company=a,permissions=perm)
        up.save()

In [16]:
UserInCompany._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: user>,
 <django.db.models.fields.related.ForeignKey: company>,
 <django.db.models.fields.DateField: expiration_date>,
 <django.db.models.fields.IntegerField: level>,
 <django.db.models.fields.related.ForeignKey: permissions>,
 <django.contrib.postgres.fields.array.ArrayField: added_permissions>)

In [1]:
for a in Company.objects.filter(metadata__company_prefix='UHSS'):
    print(a.name,a.metadata['company_prefix'])

GSO - UHE-SALTO SANTIAGO UHSS
GSB - UHE-SALTO SANTIAGO UHSS
GES - UHE-SALTO SANTIAGO UHSS


In [4]:
old_responsible= User.objects.get(username='marcos.ecoagro')
new_responsible = User.objects.get(username='joelio.lima')
# status=ServiceOrderActionStatus.objects.get(companies__name='GES - UHE-JAGUARA')
ocrs=OccurrenceRecord.objects.filter(company__name='GES - UHE-JAGUARA',status=status,created_by=old_responsible)
status,old_responsible,new_responsible.uuid,occs.count()

(<ServiceOrderActionStatus: Aguardando aprovação técnica - OCCURRENCE_RECORD_STATUS>,
 <User: marcos.ecoagro - Marcos Pereira de Carvalho>,
 UUID('7f915e99-263b-4f8a-aa82-0de19f18f25a'),
 4)

In [7]:
ocrs=OccurrenceRecord.objects.filter(occurrence_type__name='Ocorrências patrimoniais')
occs.count(),occ

2775

In [4]:
cons_detalhe=[{"name":"Pórtico","value":"1"},{"name":"Trapiche fixo","value":"2"},{"name":"Quiosque","value":"3"},{"name":"Moradia","value":"4"},{"name":"Abrigo","value":"5"},{"name":"Rampa","value":"6"},{"name":"Churrasqueira","value":"7"},{"name":"Mesa","value":"8"},{"name":"Banco","value":"9"},{"name":"Calçada","value":"10"},{"name":"Poste","value":"11"},{"name":"Cerca","value":"12"},{"name":"Garagem","value":"13"},{"name":"Muro","value":"14"},{"name":"Contenção","value":"15"},{"name":"Arruamento pavimentado","value":"16"},{"name":"Pier","value":"17"},{"name":"Atracadouro","value":"18"},{"name":"Drenagem identificada","value":"19"},{"name":"Edificação para animais","value":"20"},{"name":"Tanque-rede","value":"21"},{"name":"Portão","value":"22"},{"name":"Alambrado","value":"23"},{"name":"Dragagem","value":"24"},{"name":"Trapiche","value":"25"},{"name":"Praia artificial","value":"26"},{"name":"Jardim","value":"27"},{"name":"Trapiche flutuante","value":"28"},{"name":"Campo de futebol","value":"29"},{"name":"Casa","value":"30"},{"name":"Enrocamento","value":"31"},{"name":"Galinheiro","value":"32"},{"name":"Mesa com bancos","value":"32"},{"name":"Muro","value":"33"},{"name":"Poste com luminária","value":"34"},{"name":"Chiqueiro","value":"35"},{"name":"Cultivo","value":"36"},{"name":"Acesso","value":"37"},{"name":"Estrutura","value":"38"},{"name":"Caixa de água","value":"38"},{"name":"Plataforma flutuante","value":"39"},{"name":"Movimentação do solo","value":"40"},{"name":"Supressão da vegetação","value":"41"},{"name":"Escada","value":"42"},{"name":"Pia","value":"43"},{"name":"Aterro","value":"44"},{"name":"Cabana","value":"45"},{"name":"Vala","value":"46"},{"name":"Antena","value":"47"},{"name":"Captação de água","value":"48"},{"name":"Fossa","value":"49"},{"name":"Chuveiro","value":"50"},{"name":"Passarela","value":"51"},{"name":"Entulho","value":"52"},{"name":"Pilares","value":"53"},{"name":"Balanço","value":"54"},{"name":"Açude","value":"55"},{"name":"Poste com antena de internet","value":"56"},{"name":"Cerca lateral","value":"57"},{"name":"Parquinho infantil","value":"58"},{"name":"Casinha de madeira","value":"59"},{"name":"Tanque","value":"60"},{"name":"Paiol","value":"61"},{"name":"Barraco","value":"62"},{"name":"Banheiro","value":"63"},{"name":"Big-bags","value":"64"},{"name":"Pecuária","value":"65"},{"name":"Estábulo","value":"66"},{"name":"Casa flutuante","value":"67"},{"name":"Galpão","value":"68"},{"name":"Piscina","value":"69"},{"name":"Cascalho","value":"70"},{"name":"Pastagem","value":"71"},{"name":"Lavoura","value":"72"},{"name":"Pia","value":"73"},{"name":"Pocilga","value":"74"},{"name":"Trilho","value":"75"},{"name":"Pomar","value":"76"},{"name":"Horta","value":"77"},{"name":"Saleiros","value":"78"},{"name":"Mureta","value":"79"},{"name":"Dispositivo flutuante","value":"80"},{"name":"Escavação","value":"81"},{"name":"Luminária","value":"82"},{"name":"Banco","value":"83"},{"name":"Aterro","value":"84"},{"name":"Cerca","value":"85"},{"name":"Escada","value":"86"},{"name":"Pedras britas","value":"87"},{"name":"Casa de madeira","value":"88"},{"name":"Pontos de ancoragem (blocos de concreto)","value":"89"},{"name":"Rampa","value":"90"},{"name":"Mesa","value":"91"},{"name":"Estrutura (Casa na Arvore)","value":"92"},{"name":"Estrutura de concreto (santa)","value":"93"},{"name":"Calçada","value":"94"},{"name":"Trapiche flutuante","value":"95"},{"name":"Garagem","value":"96"},{"name":"Cerca lateral","value":"97"},{"name":"Casa flutuante","value":"98"},{"name":"Casa","value":"99"},{"name":"Canteiro","value":"100"}]
irregularidades= [{"name":"Tanque-rede","value":"1"},{"name":"Trapiche flutuante","value":"2"},{"name":"Depósito de combustível","value":"3"},{"name":"Embarcação sem documentação","value":"4"},{"name":"Casa flutuante sem documentação","value":"5"},{"name":"Rede elétrica","value":"6"},{"name":"Ajardinamento","value":"7"},{"name":"Criação de bovinos","value":"8"},{"name":"Criação de equinos","value":"9"},{"name":"Criação de ovinos","value":"10"},{"name":"Criação de caprinos","value":"11"},{"name":"Criação de asininos","value":"12"},{"name":"Criação de aves","value":"13"},{"name":"Criação de muares","value":"14"},{"name":"Criação de animais exóticos","value":"15"},{"name":"Criação de peixes","value":"16"},{"name":"Enfeites ornamentais","value":"17"},{"name":"Pastagem","value":"18"},{"name":"Açude/lagoa","value":"19"},{"name":"Lavoura","value":"20"},{"name":"Plantio de árvores exóticas","value":"21"},{"name":"Movimentação de solo","value":"22"},{"name":"Depósito de terra","value":"23"},{"name":"Depósito de materiais","value":"24"},{"name":"Estrada/acesso sem pavimentação","value":"25"},{"name":"Supressão","value":"26"},{"name":"Uso de agrotóxicos","value":"27"},{"name":"Outros","value":"28"},{"name":"Pier","value":"29"},{"name":"Mureta","value":"30"},{"name":"Canaleta","value":"31"},{"name":"Escada","value":"32"},{"name":"Luminária","value":"33"},{"name":"Plataforma flutuante","value":"34"},{"name":"Canteiro","value":"35"},{"name":"Calçada","value":"36"},{"name":"Dispositivo flutuante","value":"37"},{"name":"Pergolado","value":"38"},{"name":"Cerca","value":"39"},{"name":"Churrasqueira","value":"40"},{"name":"Barraco","value":"41"},{"name":"Poço","value":"42"},{"name":"Pocilga","value":"43"},{"name":"Enrocamento","value":"44"},{"name":"Jardim","value":"45"},{"name":"Pia","value":"46"},{"name":"Poste","value":"47"},{"name":"Mesa","value":"48"}]

In [9]:
from tqdm.notebook import tqdm
rep_edit=[]
for a in tqdm(ocrs):
    if a.form_data.get('irregularities'):
        for inner in a.form_data.get('irregularities'):
            try:
                if inner.get('unauthorized_construction_irregular'):# Irregularidades
                    inner['unauthorized_construction_irregular'] = next(v['name'] for v in irregularidades if v['value'] == inner.get('unauthorized_construction_irregular'))
                if inner.get('unauthorized_construction'):# Constatação Detalhe
                    inner['unauthorized_construction'] = next(v['name'] for v in irregularidades if v['value'] == inner.get('unauthorized_construction'))
            except:
                pass
        rep_edit.append(a)

  0%|          | 0/2775 [00:00<?, ?it/s]

In [17]:
for i,a in enumerate(tqdm(ocrs)):
    if a.form_data.get('irregularities'):
        for inner in a.form_data.get('irregularities'):
            if inner.get('unauthorized_construction'):
                print(inner.get('unauthorized_construction'),i)

  0%|          | 0/2775 [00:00<?, ?it/s]

Criação de muares 1
Criação de animais exóticos 1
Criação de equinos 5
Enfeites ornamentais 5
Trapiche flutuante 5
Casa flutuante sem documentação 7
Criação de muares 8
Criação de muares 9
Criação de ovinos 9
Rede elétrica 9
Trapiche flutuante 9
Tanque-rede 10
Ajardinamento 11
Rede elétrica 11
Trapiche flutuante 11
Açude/lagoa 11
Estrada/acesso sem pavimentação 11
Depósito de terra 11
Ajardinamento 13
Casa flutuante sem documentação 13
Trapiche flutuante 14
Criação de muares 17
Rede elétrica 17
Trapiche flutuante 17
Criação de caprinos 17
Lavoura 18
Criação de asininos 18
Embarcação sem documentação 21
Criação de asininos 23
Criação de asininos 23
Ajardinamento 29
Criação de bovinos 29
Criação de equinos 29
Depósito de materiais 32
Criação de ovinos 34
Trapiche flutuante 35
Criação de asininos 36
Movimentação de solo 37
Embarcação sem documentação 39
Movimentação de solo 40
Trapiche flutuante 43
Criação de asininos 44
Criação de caprinos 44
Rede elétrica 44
Embarcação sem documentação 

In [20]:
len(rep_edit),rep_edit[2].form_data,rep_edit[2].number

(2648,
 {'deadline': '2022-03-30T17:07:00.000Z',
  'agreement': 'guilherme',
  'observation': [{'observation_description': 'em vistoria realizada no dia 10/01/2022, identificamos uma área com ajardinamento.\nidentificamos como o infrator Guilherme de Lima.\nem conversa com o mesmo, chegamos num acordo amigavel para abandono da area para a regeneração da vegetação.'}],
  'agreement_type': True,
  'include_origin': True,
  'irregularities': [{'width': 25,
    'length': 25,
    'finding': '2',
    'display_width': True,
    'display_length': True,
    'place_identification': '1',
    'unauthorized_construction_irregular': 'Ajardinamento'}],
  'engie_hidrologia': {'usina': 'UHIT',
   'data_hora': '2022-01-10 15:00:00.0',
   'perc_vol_util': '7.83238095238095238095238095238095238095E01',
   'vazao_vertida': '0',
   'vazao_afluente': '511',
   'vazao_defluente': '511',
   'vazao_turbinada': '511',
   'nivel_canal_fuga': '264.41',
   'nivel_reservatorio': '368.77'},
  'procedure_objects': [],

In [23]:
from helpers.histories import bulk_update_with_history

In [24]:
user=User.objects.get(username='engie')

In [42]:
records_edit=bulk_update_with_history(rep_edit, OccurrenceRecord, user=user, use_django_bulk=True, batch_size=600)

In [41]:
len(rep_edit)

390

In [38]:
ocr=OccurrenceRecord.objects.filter(company__name='GES - UHE-JAGUARA',created_at__gte='2023-10-1')
ocr.count(),ocr[0].search_tag_description

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.created_at received a naive datetime (2023-10-01 00:00:00) while time zone support is active.
  RuntimeWarning)


(840, '')

In [40]:
rep_edit=[]
for a in ocr:
    if (a.search_tag_description == '' or a.search_tag_description == ' ') and len(a.properties) > 0 and a.properties != [{}] and a.properties is not None:
#         print(a.number,a.properties[0])
        try:
            if "JA" in a.properties[0].get('label'):
                print(a.number,a.properties[0])
                a.search_tag_description = a.properties[0].get('label')
                rep_edit.append(a)
        except:
            pass

UHJA-RG-2023.1688 {'label': 'UHJA-002AD'}
UHJA-RG-2023.1667 {'label': 'UHJA-089AAE'}
UHJA-RG-2023.1493 {'label': 'UHJA-087D'}
UHJA-RG-2023.1618 {'label': 'UHJA-070E'}
UHJA-RG-2023.1635 {'label': 'UHJA-093E'}
UHJA-RG-2023.1377 {'label': 'UHJA-144D'}
UHJA-RG-2023.1388 {'label': 'UHJA-192E'}
UHJA-RG-2023.1636 {'label': 'UHJA-094E'}
UHJA-RG-2023.1477 {'label': 'UHJA-148AE'}
UHJA-RG-2023.1475 {'label': 'UHJA-149E'}
UHJA-RG-2023.1424 {'label': 'UHJA-110D'}
UHJA-RG-2023.1549 {'label': 'UHJA-036D'}
UHJA-RG-2023.1427 {'label': 'UHJA-178E'}
UHJA-RG-2023.1559 {'label': 'UHJA-126E'}
UHJA-RG-2023.1369 {'label': 'UHJA-199E'}
UHJA-RG-2023.1657 {'label': 'UHJA-010D'}
UHJA-RG-2023.1517 {'label': 'UHJA-135E'}
UHJA-RG-2023.1499 {'label': 'UHJA-140E'}
UHJA-RG-2023.1503 {'label': 'UHJA-050D'}
UHJA-RG-2023.1506 {'label': 'UHJA-046D'}
UHJA-RG-2023.1697 {'label': 'UHJA-048E'}
UHJA-RG-2023.1455 {'label': 'UHJA-163E'}
UHJA-RG-2023.1576 {'label': 'UHJA-118E'}
UHJA-RG-2023.1472 {'label': 'UHJA-152E'}
UHJA-RG-2023

In [19]:
occ = OccurrenceType.objects.get(uuid="f200613b-8862-4a24-8c16-d6babd215fd7")
ocr=OccurrenceRecord.objects.filter(company__name='GES - UHE-ESTREITO',occurrence_type=occ,created_by=User.objects.get(username='engie'))
occ,ocr.count()

(<OccurrenceType: Cadastro de Matriz Vegetal - ['GES - UHE-ESTREITO', 'GES - UHE-JAGUARA', 'GES - UHE-CANA BRAVA', 'GES - UHE-ITA', 'ATIVO-TREINAMENTO', 'GES - UHE-MIRANDA', 'GES - UHE-SALTO OSORIO', 'GES - UHE-SÃO SALVADOR', 'GES - UHE-PASSO FUNDO', 'GES - UHE-MACHADINHO', 'GES - UHE-PONTE DE PEDRA', 'GES - UHE-SALTO SANTIAGO', 'GES - PCH-JOSÉ GELÁZIO DA ROCHA', 'GES - PCH-RONDONÓPOLIS']>,
 840)

In [5]:
def get_uf(uf_code):
    if type(uf_code) != 'str':
        uf_code=str(uf_code)
    
    uf=[{'id': '12', 'name': 'AC'}, {'id': '27', 'name': 'AL' }, {'id': '13', 'name': 'AM'}, {'id': '16', 'name': 'AP'}, 
        {'id': '29', 'name': 'BA'}, {'id': '23', 'name': 'CE' }, {'id': '53', 'name': 'DF'}, {'id': '32', 'name': 'ES'}, 
        {'id': '52', 'name': 'GO'}, {'id': '21', 'name': 'MA' }, {'id': '31', 'name': 'MG'}, {'id': '50', 'name': 'MS'}, 
        {'id': '51', 'name': 'MT'}, {'id': '15', 'name': 'PA' }, {'id': '25', 'name': 'PB'}, {'id': '26', 'name': 'PE'}, 
        {'id': '22', 'name': 'PI'}, {'id': '41', 'name': 'PR' }, {'id': '33', 'name': 'RJ'}, {'id': '24', 'name': 'RN'},
        {'id': '11', 'name': 'RO'}, {'id': '14', 'name': 'RR' }, {'id': '43', 'name': 'RS'}, {'id': '42', 'name': 'SC'},
        {'id': '28', 'name': 'SE'}, {'id': '35', 'name': 'SP' }, {'id': '17', 'name': 'TO'}]
    uf_translation={item['name']: item['id'] for item in uf}
#     print(uf_translation)    
    
    return uf_translation[uf_code]

In [8]:
city=City.objects.get(name='Estreito')

In [17]:
location=Location.objects.get(uuid='b78830fb-234a-4c33-8912-c1a77c410629')
st = SearchTag.objects.get(uuid='12a7dc21-7485-46f8-8ac8-6d16cb5aae0f')
location,st

(<Location: [GES - UHE-ESTREITO]- [Estreito] UHE Estreito>,
 <SearchTag: GES - UHE-ESTREITO - 1 - select_option - Controle Operacional>)

In [21]:
rep_edit=[]
for a in ocr:
#     city = City.objects.get(name='Estreito')
    a.search_tags.add(*[st])
#     rep_edit.append(a)

In [10]:
len(rep_edit)

840

In [11]:
from helpers.histories import bulk_update_with_history

In [12]:
user=User.objects.get(username='engie')

In [13]:
records_edit=bulk_update_with_history(rep_edit, OccurrenceRecord, user=user, use_django_bulk=True, batch_size=600)

In [2]:
!cat .env

STAGE=ENGIE_STAGING


In [5]:
permissions = UserPermission.objects.get(uuid='df68170e-5d0d-4c81-9b19-89e28885354e')
user = User.objects.get(username='evair.dalmazo')

permissions,user

(<UserPermission: ['PRE-GSO - UHE-ESTREITO', 'PRE-GSO - UHE-CANA BRAVA', 'PRE-Limbo - UTPS', 'PRE-GSO - UHE-ITA', 'PRE-GSO - PCH-RONDONÓPOLIS', 'PRE-GSO - UHE-MIRANDA', 'PRE-GSO - UHE-SALTO SANTIAGO', 'PRE-GSO - UHE-PONTE DE PEDRA', 'PRE-GSO - PCH-JOSÉ GELÁZIO DA ROCHA', 'PRE-GSO - UHE-SALTO OSORIO', 'PRE-GSO - UHE-JAGUARA', 'PRE-GES - UHE-CANA BRAVA', 'PRE-GES - UHE-ESTREITO', 'PRE-GSB - UHE-MACHADINHO', 'PRE-GSO - UTPS', 'PRE-GES - UHE-SALTO SANTIAGO', 'PRE-GES - UHE-MIRANDA', 'PRE-GES - UHE-PONTE DE PEDRA', 'PRE-GES - UHE-JAGUARA', 'PRE-GES - UHE-ITA', 'PRE-GSO - UHE-MACHADINHO', 'PRE-GSO - UHE-SÃO SALVADOR', 'PRE-GSO - UHE-PASSO FUNDO', 'PRE-GEH', 'PRE-GES - PCH-RONDONÓPOLIS', 'PRE-GES - UHE-MACHADINHO', 'PRE-GES - PCH-JOSÉ GELÁZIO DA ROCHA', 'PRE-GES - UHE-SÃO SALVADOR', 'PRE-GES - UHE-PASSO FUNDO', 'PRE-GES - UHE-SALTO OSORIO', 'PRE-ATIVO-TREINAMENTO', 'PRE-Engie Umburanas', 'PRE-RECURSOS HIDRICOS', 'PRE-GSB - UHE-SALTO SANTIAGO'] - ADMINISTRADOR>,
 <User: evair.dalmazo - Evair P

In [9]:
for a in Company.objects.filter(name__startswith='PRE-GES'):
    if UserInCompany.objects.filter(company=a,user=user).exists():
        continue
    ui = UserInCompany(company=a,permissions=permissions,user=user)
    ui.save()

In [6]:
UserInCompany._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: user>,
 <django.db.models.fields.related.ForeignKey: company>,
 <django.db.models.fields.DateField: expiration_date>,
 <django.db.models.fields.IntegerField: level>,
 <django.db.models.fields.related.ForeignKey: permissions>,
 <django.contrib.postgres.fields.array.ArrayField: added_permissions>,
 <django.db.models.fields.BooleanField: is_active>)

In [22]:
occ=OccurrenceType.objects.get(uuid='faa62cbd-c2ec-49fc-bc4a-9b1ab9edcca4')

In [35]:
occ.name

'Ocorrências patrimoniais'

In [36]:
select=[]
for a in occ.history.all().order_by('-history_date'):
    for field in a.form_fields['fields']:
        if field['api_name']== 'irregularities':
            for inner in field['inner_fields']:
                if inner['display_name']== 'Irregularidades':
                    print(inner['api_name'])
                    if inner['data_type'] == 'select' and select == []:
                        select = inner['select_options']['options']
                        print(select)
                        break
                

unauthorizedConstructionIrregular
unauthorizedConstructionIrregular
[{'name': 'Tanque-rede', 'value': '1'}, {'name': 'Trapiche flutuante', 'value': '2'}, {'name': 'Depósito de combustível', 'value': '3'}, {'name': 'Embarcação sem documentação', 'value': '4'}, {'name': 'Casa flutuante sem documentação', 'value': '5'}, {'name': 'Rede elétrica', 'value': '6'}, {'name': 'Ajardinamento', 'value': '7'}, {'name': 'Criação de bovinos', 'value': '8'}, {'name': 'Criação de equinos', 'value': '9'}, {'name': 'Criação de ovinos', 'value': '10'}, {'name': 'Criação de caprinos', 'value': '11'}, {'name': 'Criação de asininos', 'value': '12'}, {'name': 'Criação de aves', 'value': '13'}, {'name': 'Criação de muares', 'value': '14'}, {'name': 'Criação de animais exóticos', 'value': '15'}, {'name': 'Criação de peixes', 'value': '16'}, {'name': 'Enfeites ornamentais', 'value': '17'}, {'name': 'Pastagem', 'value': '18'}, {'name': 'Açude/lagoa', 'value': '19'}, {'name': 'Lavoura', 'value': '20'}, {'name': 'P

KeyError: 'api_name'

In [34]:
select_const_detalhe=[{'name': 'Pórtico', 'value': '1'}, {'name': 'Trapiche fixo', 'value': '2'}, {'name': 'Quiosque', 'value': '3'}, {'name': 'Moradia', 'value': '4'}, {'name': 'Abrigo', 'value': '5'}, {'name': 'Rampa', 'value': '6'}, {'name': 'Churrasqueira', 'value': '7'}, {'name': 'Mesa', 'value': '8'}, {'name': 'Banco', 'value': '9'}, {'name': 'Calçada', 'value': '10'}, {'name': 'Poste', 'value': '11'}, {'name': 'Cerca', 'value': '12'}, {'name': 'Garagem', 'value': '13'}, {'name': 'Muro', 'value': '14'}, {'name': 'Contenção', 'value': '15'}, {'name': 'Arruamento pavimentado', 'value': '16'}, {'name': 'Pier', 'value': '17'}, {'name': 'Atracadouro', 'value': '18'}, {'name': 'Drenagem identificada', 'value': '19'}, {'name': 'Edificação para animais', 'value': '20'}, {'name': 'Tanque-rede', 'value': '21'}, {'name': 'Portão', 'value': '22'}, {'name': 'Alambrado', 'value': '23'}, {'name': 'Dragagem', 'value': '24'}, {'name': 'Trapiche', 'value': '25'}, {'name': 'Praia artificial', 'value': '26'}, {'name': 'Jardim', 'value': '27'}, {'name': 'Trapiche flutuante', 'value': '28'}, {'name': 'Campo de futebol', 'value': '29'}, {'name': 'Casa', 'value': '30'}, {'name': 'Enrocamento', 'value': '31'}, {'name': 'Galinheiro', 'value': '32'}, {'name': 'Mesa com bancos', 'value': '32'}, {'name': 'Muro', 'value': '33'}, {'name': 'Poste com luminária', 'value': '34'}, {'name': 'Chiqueiro', 'value': '35'}, {'name': 'Cultivo', 'value': '36'}, {'name': 'Acesso', 'value': '37'}, {'name': 'Estrutura', 'value': '38'}, {'name': 'Caixa de água', 'value': '38'}, {'name': 'Plataforma flutuante', 'value': '39'}, {'name': 'Movimentação do solo', 'value': '40'}, {'name': 'Supressão da vegetação', 'value': '41'}, {'name': 'Escada', 'value': '42'}, {'name': 'Pia', 'value': '43'}, {'name': 'Aterro', 'value': '44'}, {'name': 'Cabana', 'value': '45'}, {'name': 'Vala', 'value': '46'}, {'name': 'Antena', 'value': '47'}, {'name': 'Captação de água', 'value': '48'}, {'name': 'Fossa', 'value': '49'}, {'name': 'Chuveiro', 'value': '50'}, {'name': 'Passarela', 'value': '51'}, {'name': 'Entulho', 'value': '52'}, {'name': 'Pilares', 'value': '53'}, {'name': 'Balanço', 'value': '54'}, {'name': 'Açude', 'value': '55'}, {'name': 'Poste com antena de internet', 'value': '56'}, {'name': 'Cerca lateral', 'value': '57'}, {'name': 'Parquinho infantil', 'value': '58'}, {'name': 'Casinha de madeira', 'value': '59'}, {'name': 'Tanque', 'value': '60'}, {'name': 'Paiol', 'value': '61'}, {'name': 'Barraco', 'value': '62'}, {'name': 'Banheiro', 'value': '63'}, {'name': 'Big-bags', 'value': '64'}, {'name': 'Pecuária', 'value': '65'}, {'name': 'Estábulo', 'value': '66'}, {'name': 'Casa flutuante', 'value': '67'}, {'name': 'Galpão', 'value': '68'}, {'name': 'Piscina', 'value': '69'}, {'name': 'Cascalho', 'value': '70'}, {'name': 'Pastagem', 'value': '71'}, {'name': 'Lavoura', 'value': '72'}, {'name': 'Pia', 'value': '73'}, {'name': 'Pocilga', 'value': '74'}, {'name': 'Trilho', 'value': '75'}, {'name': 'Pomar', 'value': '76'}, {'name': 'Horta', 'value': '77'}, {'name': 'Saleiros', 'value': '78'}, {'name': 'Mureta', 'value': '79'}, {'name': 'Dispositivo flutuante', 'value': '80'}, {'name': 'Escavação', 'value': '81'}, {'name': 'Luminária', 'value': '82'}]
select_irregularidades=[{'name': 'Tanque-rede', 'value': '1'}, {'name': 'Trapiche flutuante', 'value': '2'}, {'name': 'Depósito de combustível', 'value': '3'}, {'name': 'Embarcação sem documentação', 'value': '4'}, {'name': 'Casa flutuante sem documentação', 'value': '5'}, {'name': 'Rede elétrica', 'value': '6'}, {'name': 'Ajardinamento', 'value': '7'}, {'name': 'Criação de bovinos', 'value': '8'}, {'name': 'Criação de equinos', 'value': '9'}, {'name': 'Criação de ovinos', 'value': '10'}, {'name': 'Criação de caprinos', 'value': '11'}, {'name': 'Criação de asininos', 'value': '12'}, {'name': 'Criação de aves', 'value': '13'}, {'name': 'Criação de muares', 'value': '14'}, {'name': 'Criação de animais exóticos', 'value': '15'}, {'name': 'Criação de peixes', 'value': '16'}, {'name': 'Enfeites ornamentais', 'value': '17'}, {'name': 'Pastagem', 'value': '18'}, {'name': 'Açude/lagoa', 'value': '19'}, {'name': 'Lavoura', 'value': '20'}, {'name': 'Plantio de árvores exóticas', 'value': '21'}, {'name': 'Movimentação de solo', 'value': '22'}, {'name': 'Depósito de terra', 'value': '23'}, {'name': 'Depósito de materiais', 'value': '24'}, {'name': 'Estrada/acesso sem pavimentação', 'value': '25'}, {'name': 'Supressão', 'value': '26'}, {'name': 'Uso de agrotóxicos', 'value': '27'}, {'name': 'Outros', 'value': '28'}, {'name': 'Pier', 'value': '29'}, {'name': 'Mureta', 'value': '30'}, {'name': 'Canaleta', 'value': '31'}, {'name': 'Escada', 'value': '32'}, {'name': 'Luminária', 'value': '33'}, {'name': 'Plataforma flutuante', 'value': '34'}, {'name': 'Canteiro', 'value': '35'}, {'name': 'Calçada', 'value': '36'}, {'name': 'Dispositivo flutuante', 'value': '37'}, {'name': 'Pergolado', 'value': '38'}, {'name': 'Cerca', 'value': '39'}, {'name': 'Churrasqueira', 'value': '40'}, {'name': 'Barraco', 'value': '41'}, {'name': 'Poço', 'value': '42'}, {'name': 'Pocilga', 'value': '43'}, {'name': 'Enrocamento', 'value': '44'}, {'name': 'Jardim', 'value': '45'}, {'name': 'Pia', 'value': '46'}, {'name': 'Poste', 'value': '47'}, {'name': 'Mesa', 'value': '48'}]

In [14]:
occ.history.all().order_by('-history_date')[0].form_fields

{'id': 20,
 'name': 'patrimonialOccurrence',
 'fields': [{'id': 11,
   'api_name': 'irregularities',
   'data_type': 'arrayOfObjects',
   'display_name': '',
   'inner_fields': [{'order': 1,
     'api_name': 'placeIdentification',
     'print_if': {'!!': {'var': {'cat': ['data.formData.irregularities',
         {'var': 'index.0'},
         '.placeIdentification']}}},
     'data_type': 'select',
     'display_name': 'Identificação do local',
     'select_options': {'options': [{'name': 'Área de preservação permanente',
        'value': '1'},
       {'name': 'Área remanescente', 'value': '2'},
       {'name': 'Área adquirida', 'value': '3'},
       {'name': 'Área de inundação', 'value': '4'},
       {'name': 'Área industrial', 'value': '5'}]}},
    {'order': 2,
     'api_name': 'finding',
     'print_if': {'!!': {'var': {'cat': ['data.formData.irregularities',
         {'var': 'index.0'},
         '.finding']}}},
     'data_type': 'select',
     'display_name': 'Constatação',
     'selec

In [39]:
ocrs=OccurrenceRecord.objects.filter(occurrence_type__name='Ocorrências patrimoniais', form_data__irregularities__isnull=False)
ocrs.count()

2871

In [50]:
from tqdm.notebook import tqdm
irregularidades=set()
constatacao_detalhe=set()
for a in tqdm(ocrs):
    if a.form_data.get('irregularities'):
        for inner in a.form_data.get('irregularities'):
            if inner.get('unauthorized_construction_irregular') == '25':
                print(a.number,a.company.name)
            if inner.get('unauthorized_construction_irregular'):# Irregularidades
                irregularidades.add(inner.get('unauthorized_construction_irregular'))
            if inner.get('unauthorized_construction'):# Constatação Detalhe
                constatacao_detalhe.add(inner.get('unauthorized_construction'))

  0%|          | 0/2871 [00:00<?, ?it/s]

UHPP-RG-2024.0009 GES - UHE-PONTE DE PEDRA


In [52]:
len(irregularidades),irregularidades

(48,
 {'25',
  'Ajardinamento',
  'Açude/lagoa',
  'Balanço',
  'Campo de areia',
  'Casa flutuante sem documentação',
  'Cerca',
  'Churrasqueira',
  'Cisterna para captação de água',
  'Criação de animais exóticos',
  'Criação de aves',
  'Criação de bovinos',
  'Criação de equinos',
  'Criação de peixes',
  'Delimitação com estacas',
  'Depósito de materiais',
  'Depósito de terra',
  'Dispositivo flutuante',
  'Embarcação sem documentação',
  'Enfeites ornamentais',
  'Enrocamento',
  'Escada',
  'Estrada/acesso sem pavimentação',
  'Fossa negra ',
  'Gangorra',
  'Incinerador',
  'Lavoura',
  'Mesa',
  'Movimentação de solo',
  'Outros',
  'Pastagem',
  'Pergolado',
  'Pier',
  'Plantio de árvores exóticas',
  'Poste',
  'Poço',
  'Rede elétrica',
  'Supressão',
  'Supressão de vegetação nativa para instalação de captação de água.',
  'Tanque-rede',
  'Trapiche flutuante',
  'Uso de agrotóxicos',
  'campo gramado com demarcação externa de pneus',
  'cisterna para capitação de água

In [51]:
len(constatacao_detalhe),constatacao_detalhe

(58,
 {'Abrigo',
  'Alambrado',
  'Arruamento pavimentado',
  'Atracadouro',
  'Banco',
  'Banheiro',
  'Biodigestor',
  'Bomba para captação de água do reservatório',
  'Caixa de água',
  'Calçada',
  'Calçada de concreto.',
  'Canaleta de drenagem pluvial',
  'Casa',
  'Cerca',
  'Cerca de arame com mureta de alvenaria e postes de concreto.',
  'Cerca de tela com postes de madeira.',
  'Cerca lateral',
  'Churrasqueira',
  'Contenção',
  'Contenção feita com postes de madeira.',
  'Deck de madeira',
  'Dragagem',
  'Drenagem identificada',
  'Edificação para animais',
  'Garagem',
  'Mesa',
  'Moradia',
  'Mureta',
  'Mureta de alvenaria com alambrado.',
  'Muro',
  'Muro de contenção feito de madeira.',
  'Muro de divisa feito de madeira.',
  'Outdoor',
  'Parquinho infantil',
  'Passarela',
  'Pergolado de madeira coberto com chapas de policarbonato',
  'Pia',
  'Pier',
  'Piscina',
  'Portão',
  'Poste',
  'Poste com luminária',
  'Postes de madeira com luminárias.',
  'Pér de mad

In [53]:
data={"Irregularidades":irregularidades,"Constatação (Detalhe)": constatacao_detalhe}

In [54]:
import pandas as pd

In [57]:
df = pd.DataFrame.from_dict(data, orient='index')
df.transpose().to_excel('Irregularidades-ConstatacoesDetalhe.xlsx')

In [1]:
RecordPanel._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.TextField: name>,
 <django.db.models.fields.CharField: panel_type>,
 <django.contrib.postgres.fields.jsonb.JSONField: conditions>,
 <django.db.models.fields.related.ForeignKey: company>,
 <django.contrib.postgres.fields.jsonb.JSONField: list_columns>,
 <django.contrib.postgres.fields.jsonb.JSONField: list_order_by>,
 <django.contrib.postgres.fields.jsonb.JSONField: kanban_columns>,
 <django.contrib.postgres.fields.jsonb.JSONField: kanban_group_by>,
 <django.db.models.fields.TextField: icon>,
 <django.db.models.fields.IntegerField: icon_size>,
 <django.db.models.fields.CharField: color>,
 <django.db.models.fields.DateTimeField: created_at>,
 <django.db.models.fields.related.ForeignKey: created_by>,
 <django.db.models.fields.related.ForeignKey: content_type>)

In [ ]:
showInAppMap,showInWebMap

In [1]:
rp=RecordPanel.objects.get(uuid='fed86fa3-2ff6-4c9c-ae08-4a8eb932c20d')

In [4]:
rp.__dict__

{'_state': <django.db.models.base.ModelState at 0x7f091fbd88e0>,
 'uuid': UUID('fed86fa3-2ff6-4c9c-ae08-4a8eb932c20d'),
 'name': 'ZAS',
 'panel_type': 'LIST',
 'conditions': {'data': {'search_tag': None, 'place_on_dam': None},
  'logic': {'and': [{'==': [{'var': 'search_tag'},
      'e87e45ed-72ac-4b1b-899c-9a91faf0ddb9']},
    {'==': [{'var': 'place_on_dam'}, 'ZAS']}]},
  'errors': []},
 'company_id': UUID('e9fce210-f652-4d7d-befe-32e02a8f0f60'),
 'list_columns': ['datetime',
  'number',
  'type',
  'placeOnDam',
  'form_data__altitude',
  'form_data__mainContactName',
  'form_data__fixedResidents',
  'form_data__reducedMobility',
  'form_data__mainContactPhone',
  'city',
  'status'],
 'list_order_by': [{'field': 'datetime', 'order': 'ASC'}],
 'kanban_columns': [],
 'kanban_group_by': [],
 'icon': 'home',
 'icon_size': 15,
 'color': '#FFAA00',
 'created_at': datetime.datetime(2023, 4, 20, 16, 53, 28, 132593, tzinfo=<UTC>),
 'created_by_id': UUID('0aa50773-b368-4a50-9f12-4a7d8dfaf256'

In [2]:
RecordPanelShowList.objects.filter(panel=rp).count()

57

In [3]:
RecordPanelShowMobileMap.objects.filter(panel=rp)

<QuerySet [<RecordPanelShowMobileMap: [GSO - UHE-ITA] 946e151d-de54-4dce-bc4a-aa281337e336 - ZAS>, <RecordPanelShowMobileMap: [GSO - UHE-ITA] f8d8bc12-aba6-46d3-a50c-4adeaea0e5ae - ZAS>]>

In [5]:
user=User.objects.get(username='engie')
user

<User: engie - Suporte Kartado>

In [ ]:
from simple_history.utils import bulk_create_with_history
from tqdm.notebook import tqdm

In [ ]:
paineis = ['Uso e ocupação irregular', 'Deferidos', 'Inadimplência de PU']

In [7]:
rp_show_list=[]
rp_show_mobile=[]
rp_show_web=[]
for rpjaguara in tqdm(RecordPanel.objects.filter(name__in=paineis,
                                            company__name='GES - UHE-JAGUARA')):
    for company in Company.objects.filter(name__startswith="GES").exclude(name='GES - UHE-JAGUARA'):
        rp = RecordPanel.objects.filter(name__icontains=rpjaguara.name,company=company)
        if rp.exists():
            continue
        rpjaguara.uuid = None
        rpjaguara.company = company
        rpjaguara.save()
        rpjaguara.viewer_permissions.add(*UserPermissions.objects.filter(companies=company))
    

GES - UHE-CANA BRAVA ['Registros patrimoniais']
GES - UHE-ESTREITO ['Horto Florestal']
GES - UHE-ITA ['Homologados', 'Em tratamento', 'Atendidos', 'Em Elaboração']
GES - UHE-JAGUARA ['Uso e ocupação irregular', 'Deferidos', 'Inadimplência de PU']
GES - UHE-MIRANDA ['Teste Marcos', 'Uso e Ocupação Irregular', 'Teste Luan']


In [26]:
rp_show_list=[]
rp_show_mobile=[]
rp_show_web=[]
for rp in tqdm(RecordPanel.objects.filter(name__in=paineis).exclude(company__name='GES - UHE-JAGUARA')):
    for ui in tqdm(UserInCompany.objects.filter(name__startswith="GES").exclude(name='GES - UHE-JAGUARA')):
        last_order=RecordPanelShowList.objects.filter(panel__company=ui.company,user=ui.user).order_by('order').last().order
        
        if not RecordPanelShowList.objects.filter(panel=rp,user=ui.user).exists():
            rpsl = RecordPanelShowList(panel=rp,user=ui.user,order=last_oder+1)
            rp_show_list.append(rpsl)
        
        if not RecordPanelShowMobileMap.objects.filter(panel=rp,user=ui.user).exists():
            rpsmm = RecordPanelShowMobileMap(panel=rp,user=ui.user)
            rp_show_mobile.append(rpsmm)
        
        if not RecordPanelShowWebMap.objects.filter(panel=rp,user=ui.user).exists():
            rpswm = RecordPanelShowWebMap(panel=rp,user=ui.user)
            rp_show_web.append(rpswm)
    

In [ ]:
user=User.objects.get(username='engie')

In [ ]:
result1 = bulk_create_with_history(rp_show_list, RecordPanelShowList, default_user=user)
result2 = bulk_create_with_history(rp_show_mobile, RecordPanelShowMobileMap, default_user=user) 
result3 = bulk_create_with_history(rp_show_web, RecordPanelShowWebMap, default_user=user)

In [36]:
RecordPanelShowList.objects.filter(panel__company__name='CCR - SPVias',user__username='isabela.silveira').order_by('order').last().order

3

In [1]:
RecordPanelShowList._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: panel>,
 <django.db.models.fields.related.ForeignKey: user>,
 <django.db.models.fields.IntegerField: order>)

In [2]:
RecordPanelShowMobileMap._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: panel>,
 <django.db.models.fields.related.ForeignKey: user>)

In [11]:
RecordPanelShowWebMap._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: panel>,
 <django.db.models.fields.related.ForeignKey: user>)

In [3]:
RecordPanel._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.TextField: name>,
 <django.db.models.fields.CharField: panel_type>,
 <django.contrib.postgres.fields.jsonb.JSONField: conditions>,
 <django.db.models.fields.related.ForeignKey: company>,
 <django.contrib.postgres.fields.jsonb.JSONField: list_columns>,
 <django.contrib.postgres.fields.jsonb.JSONField: list_order_by>,
 <django.contrib.postgres.fields.jsonb.JSONField: kanban_columns>,
 <django.contrib.postgres.fields.jsonb.JSONField: kanban_group_by>,
 <django.db.models.fields.TextField: icon>,
 <django.db.models.fields.IntegerField: icon_size>,
 <django.db.models.fields.CharField: color>,
 <django.db.models.fields.DateTimeField: created_at>,
 <django.db.models.fields.related.ForeignKey: created_by>,
 <django.db.models.fields.related.ForeignKey: content_type>)

In [6]:
RecordPanel.objects.first().viewer_permissions

<django.db.models.fields.related_descriptors.create_forward_many_to_many_manager.<locals>.ManyRelatedManager at 0x7f0731205b80>

In [10]:
UserPermission.objects.first().companies

<django.db.models.fields.related_descriptors.create_forward_many_to_many_manager.<locals>.ManyRelatedManager at 0x7f0731205ac0>

In [2]:
for a in Company.objects.all():
    rps=RecordPanel.objects.filter(company=a)
    if not rps.exists():
        continue
    print(a.name,[b.name for b in rps])

CCR - SPVias ['PRE 02', 'PRE 03', 'MF 01']
CCR - Rio Sp ['ITS', 'Painel teste', 'Drenagem', 'Monitoração Edificações 2023', 'OAC ANTT', 'Monit. Sinalização Vertical', 'Monitoração Vertical']
CCR - RodoAnel ['Monitoração Drenagem superficial 2024']


In [24]:
Firm.objects.filter(name='Campo/Escritório').first().reportings.count()

3839

In [37]:
occRota = OccurrenceType.objects.get(uuid='69513ccd-b8f9-439c-a418-158e95524d85')
occEncontro = OccurrenceType.objects.get(uuid='01dfad93-30e9-45fa-a7d0-4b59973a7223')
occRota,occEncontro

(<OccurrenceType: Sinalização da rota de fuga - ['GSO - UHE-ITA', 'GSO - UHE-SÃO SALVADOR', 'GSO - UHE-MACHADINHO', 'GSO - PCH-RONDONÓPOLIS', 'GSO - UHE-SALTO SANTIAGO', 'GSO - UHE-MIRANDA', 'GSO - UHE-SALTO OSORIO', 'GSO - UHE-PASSO FUNDO', 'GSO - UHE-CANA BRAVA', 'GSO - UHE-JAGUARA', 'GSO - UTPS', 'GSO - UHE-PONTE DE PEDRA', 'GSO - PCH-JOSÉ GELÁZIO DA ROCHA', 'GSO - UHE-ESTREITO']>,
 <OccurrenceType: Ponto de encontro - ['GSO - UHE-ESTREITO', 'GSO - PCH-RONDONÓPOLIS', 'GSO - PCH-JOSÉ GELÁZIO DA ROCHA', 'GSO - UHE-PONTE DE PEDRA', 'GSO - UHE-SÃO SALVADOR', 'GSO - UHE-CANA BRAVA', 'GSO - UHE-SALTO OSORIO', 'GSO - UHE-SALTO SANTIAGO', 'GSO - UHE-MIRANDA', 'GSO - UHE-JAGUARA', 'GSO - UHE-ITA', 'GSO - UHE-MACHADINHO', 'GSO - UHE-PASSO FUNDO']>)

In [15]:
from tqdm.notebook import tqdm

In [31]:
registros_sem_cidade=[]
to_fix=[]
proper=[]
for a in tqdm(OccurrenceRecord.objects.filter(occurrence_type=occEncontro,company__name__startswith='GSO')):
    if a.city == None:
        history_city = a.history.all().order_by('-history_date')[1].city_id
        name_city = City.history.filter(uuid=history_city).first().name
        city_atual = City.objects.filter(name=name_city)
        if city_atual.count()>1:
            print(city_atual)
        a.city = city_atual.first()
        registros_sem_cidade.append(a)
    try:
        firm = Firm.objects.get(company=a.company,name__icontains=a.city.name)
    except:
#         print(a.company,a.firm.name,a.city.name)
        firm = Firm.objects.get(company=a.company,name='Equipe Estreito (Defesa Civil)')
    if a.city.name not in a.firm.name:
#         print(firm.name,a.city.name)
        a.firm = firm
        to_fix.append(a)
    else:
        proper.append(a)

  0%|          | 0/294 [00:00<?, ?it/s]

<QuerySet [<City: [41] Sulina>, <City: [41] Sulina>]>
<QuerySet [<City: [41] Sulina>, <City: [41] Sulina>]>
<QuerySet [<City: [41] Sulina>, <City: [41] Sulina>]>
<QuerySet [<City: [41] Sulina>, <City: [41] Sulina>]>
<QuerySet [<City: [41] Sulina>, <City: [41] Sulina>]>
<QuerySet [<City: [41] Sulina>, <City: [41] Sulina>]>
<QuerySet [<City: [41] Sulina>, <City: [41] Sulina>]>
<QuerySet [<City: [41] Sulina>, <City: [41] Sulina>]>
<QuerySet [<City: [41] Sulina>, <City: [41] Sulina>]>
<QuerySet [<City: [41] Sulina>, <City: [41] Sulina>]>
<QuerySet [<City: [41] Sulina>, <City: [41] Sulina>]>


In [32]:
len(to_fix)

294

In [33]:
len(proper)

0

In [33]:
to_fix[0].firm,to_fix[0].company,to_fix[0].number

(<Firm: [4b4bfdad-f8b8-403b-93a6-26e846a6770b: GSO - UHE-MACHADINHO] cee9925b-72d8-450a-bb1d-224685af1102: Equipe Piratuba>,
 <Company: 4b4bfdad-f8b8-403b-93a6-26e846a6770b: GSO - UHE-MACHADINHO>,
 'UHMA-RF-2023.0093')

In [34]:
from helpers.histories import bulk_update_with_history

In [35]:
user=User.objects.get(username='engie')

In [36]:
records_edit=bulk_update_with_history(to_fix, OccurrenceRecord, user=user, use_django_bulk=True, batch_size=150)

In [44]:
for a in to_fix:
    history = a.history.all().order_by('-history_date')[1]
    history.city = a.city
    history.instance.save()

In [34]:
OccurrenceRecord.history.filter(occurrence_type=occRota,
                                company__name__startswith='GSO',
                               history_date__gte='2024-03-13').count()

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalOccurrenceRecord.history_date received a naive datetime (2024-03-13 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


541

In [24]:
len(registros_sem_cidade),registros_sem_cidade[0].city

(102, <City: [43] Aratiba>)

In [13]:
for a in tqdm(registros_sem_cidade):
    history_city = a.history.all().order_by('-history_date')[1].city_id
    name_city = City.history.filter(uuid=history_city).first().name
    city_atual = City.objects.get(name=name_city)
    


In [9]:
history.__dict__

{'_state': <django.db.models.base.ModelState at 0x7f931e7454c0>,
 'geometry_hash': 'cee39f03c684efc930a824fb4a574955',
 'uuid': UUID('9d3c1f28-b3fa-4dcb-9ebd-955c90ed3137'),
 'datetime': datetime.datetime(2023, 9, 18, 21, 21, 36, 117097, tzinfo=<UTC>),
 'number': 'UHIT-RF-2023.0036',
 'created_at': datetime.datetime(2023, 9, 18, 21, 25, 13, 395789, tzinfo=<UTC>),
 'updated_at': datetime.datetime(2023, 10, 16, 23, 49, 29, 132496, tzinfo=<UTC>),
 'reviews': 0,
 'editable': True,
 'is_approved': False,
 'uf_code': 43,
 'place_on_dam': '',
 'point': None,
 'geometry': None,
 'properties': [{'UF': 'RS',
   'ZAS': 'ITÁ',
   'NM_MUN': 'Aratiba',
   'ALTWGS84': 350,
   'NUMERO_ID': 36,
   'COMPRIMENT': 236,
   'CONVERGENC': 11,
   'NUM_PT_ENC': 126}],
 'distance_from_dam': 0.0,
 'other_reference': '',
 'origin': None,
 'origin_media': None,
 'informer': {},
 'territorial_administration': '',
 'search_tag_description': '',
 'form_data': {'xcoord': [-52.38890236624917, -27.303125765617494],
  'y

In [58]:
OccurrenceRecord.objects.get(number='UHIT-RF-2023.0059').__dict__

{'_state': <django.db.models.base.ModelState at 0x7f930b204280>,
 'uuid': UUID('0c82d37e-3778-4def-a866-f117f56b8387'),
 'datetime': datetime.datetime(2023, 9, 18, 21, 21, 36, 117097, tzinfo=<UTC>),
 'number': 'UHIT-RF-2023.0059',
 'created_at': datetime.datetime(2023, 9, 18, 21, 27, 37, 96884, tzinfo=<UTC>),
 'updated_at': datetime.datetime(2024, 3, 13, 13, 23, 50, 758053, tzinfo=<UTC>),
 'reviews': 0,
 'company_id': UUID('e9fce210-f652-4d7d-befe-32e02a8f0f60'),
 'editable': True,
 'is_approved': False,
 'uf_code': 42,
 'city_id': UUID('0bc4b730-abd4-49fa-b2dd-9ccab30d30d3'),
 'location_id': None,
 'place_on_dam': '',
 'river_id': None,
 'point': None,
 'geometry': None,
 'properties': [{'UF': 'SC',
   'ZAS': 'ITÁ',
   'NM_MUN': 'Itá',
   'ALTWGS84': 335,
   'NUMERO_ID': 59,
   'COMPRIMENT': 64,
   'CONVERGENC': 33,
   'NUM_PT_ENC': 138}],
 'distance_from_dam': 0.0,
 'other_reference': '',
 'origin': None,
 'origin_media': None,
 'informer': {},
 'territorial_administration': '',
 'cr

In [64]:
OccurrenceRecord.objects.filter(occurrence_type__in=[occRota,occEncontro]).count()

937

In [38]:
occRota = OccurrenceType.objects.get(uuid='69513ccd-b8f9-439c-a418-158e95524d85')
occEncontro = OccurrenceType.objects.get(uuid='01dfad93-30e9-45fa-a7d0-4b59973a7223')
occRota.name,occEncontro.name

('Sinalização da rota de fuga', 'Ponto de encontro')

In [46]:
to_fix=[]
for a in tqdm(OccurrenceRecord.objects.filter(geometry__isnull=True,
                                              occurrence_type__in=[occRota,occEncontro]
                                             ).order_by('company__name','number')):
    
    historico = a.history.filter(geometry__isnull=False).order_by('-history_date')
    
    if historico.exists():
        a.geometry = historico.first().geometry
        to_fix.append(a)
    else:
        print(a.company.name,a.number)

  0%|          | 0/541 [00:00<?, ?it/s]

In [61]:
len(to_fix),

541

In [34]:
from helpers.histories import bulk_update_with_history

In [35]:
user=User.objects.get(username='engie')

In [36]:
records_edit=bulk_update_with_history(to_fix, OccurrenceRecord, user=user, use_django_bulk=True, batch_size=150)

In [1]:
OccurrenceRecord.objects.get(number='UHJA-RG-2023.1767').is_approved

False

In [2]:
OccurrenceRecord.objects.filter(number='UHJA-RG-2023.1767').update(is_approved=True)

1

In [2]:
aps = ApprovalStep.objects.get(uuid='e86c42b1-7b9a-4ef7-8f1f-aa3192251b77')
status=ServiceOrderActionStatus.objects.get(uuid='6f6cdc39-cbcf-4649-84ca-9807f2a61645')
ocrs=OccurrenceRecord.objects.filter(number__in=['UHMI-RG-2023.1224','UHMI-RG-2023.1801','UHMI-RG-2023.1797'])
aps,status,ocrs.count()

(<ApprovalStep: [GES - UHE-MIRANDA]: Registro em Elaboração>,
 <ServiceOrderActionStatus: Registro em Elaboração - OCCURRENCE_RECORD_STATUS>,
 3)

In [3]:
for a in ocrs:
    a.approval_step=aps
    a.status=status
    a.save()

In [12]:
aps = ApprovalStep.objects.filter(uuid='e86c42b1-7b9a-4ef7-8f1f-aa3192251b77')
subcompany = SubCompany.objects.filter(name="LR Ambiental",company__name='GES - UHE-MIRANDA')
aps,subcompany

(<QuerySet [<ApprovalStep: [GES - UHE-MIRANDA]: Aguardando Homologação>]>,
 <QuerySet [<SubCompany: [2fd0ca76-b741-47d7-af9c-607e6ed1eca2: GES - UHE-MIRANDA] 47b3c5a6-7498-4089-8a17-2d608fbb65e9: LR Ambiental>]>)

In [13]:
subcompany[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x7bf6875326a0>,
 'uuid': UUID('47b3c5a6-7498-4089-8a17-2d608fbb65e9'),
 'subcompany_type': 'HIRED',
 'company_id': UUID('2fd0ca76-b741-47d7-af9c-607e6ed1eca2'),
 'name': 'LR Ambiental',
 'cnpj': '37.208.076/0001-17',
 'responsible_id': UUID('bdb70b20-4e78-4d6b-91dd-714438f3a5a8'),
 'contract': '',
 'contract_start_date': None,
 'contract_end_date': None,
 'office': 'Empresa de Fiscalização Ambiental e Patrimonial',
 'construction_name': '',
 'logo': '',
 'hired_by_subcompany_id': UUID('ab437ee4-afdb-435a-9c27-514d4f6d028b')}

In [18]:
for a in subcompany:
    print(a.history.all().values('responsible__username'))

<QuerySet [{'responsible__username': 'lucas.reis'}, {'responsible__username': 'ronivaldo.silva'}, {'responsible__username': 'lucas.reis'}, {'responsible__username': 'lucas.reis'}]>


In [17]:
a.history.all()[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x7bf687532880>,
 'uuid': UUID('47b3c5a6-7498-4089-8a17-2d608fbb65e9'),
 'subcompany_type': 'HIRED',
 'name': 'LR Ambiental',
 'cnpj': '37.208.076/0001-17',
 'contract': '',
 'contract_start_date': None,
 'contract_end_date': None,
 'office': 'Empresa de Fiscalização Ambiental e Patrimonial',
 'construction_name': '',
 'logo': '',
 'company_id': UUID('2fd0ca76-b741-47d7-af9c-607e6ed1eca2'),
 'responsible_id': UUID('bdb70b20-4e78-4d6b-91dd-714438f3a5a8'),
 'hired_by_subcompany_id': UUID('ab437ee4-afdb-435a-9c27-514d4f6d028b'),
 'history_id': UUID('fcb3ae10-26dc-4bf1-abfd-a845e124897b'),
 'history_date': datetime.datetime(2024, 3, 13, 17, 46, 18, 24980, tzinfo=<UTC>),
 'history_change_reason': None,
 'history_type': '~',
 'history_user_id': UUID('6810164a-e3ba-4c8d-be1f-d95c16420040')}

In [2]:
RecordPanelShowMobileMap._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: panel>,
 <django.db.models.fields.related.ForeignKey: user>)

In [12]:
RecordPanelShowWebMap._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: panel>,
 <django.db.models.fields.related.ForeignKey: user>)

In [5]:
rp_ss=RecordPanel.objects.filter(company__name='GSO - UHE-SALTO SANTIAGO')
user=User.objects.get(username='karla.zorzam')
user,rp_ss.count()

(<User: karla.zorzam - Karla Zorzam>, 5)

In [6]:
RecordPanelShowMobileMap.objects.filter(panel__in=rp_ss,user=user)

<QuerySet [<RecordPanelShowMobileMap: [GSO - UHE-SALTO SANTIAGO] 52d78cf9-d930-4609-b447-405dd1c5e653 - ZSS>, <RecordPanelShowMobileMap: [GSO - UHE-SALTO SANTIAGO] 6a4ca5ed-ac14-44a2-8be2-02b7d0a97358 - ZAS>, <RecordPanelShowMobileMap: [GSO - UHE-SALTO SANTIAGO] d8e9ff60-5773-4359-a33e-436f39d57da7 - Ponto de encontro>, <RecordPanelShowMobileMap: [GSO - UHE-SALTO SANTIAGO] 736ee6c1-5b04-413f-8d7e-9d410d353af3 - Alerta sonoro>, <RecordPanelShowMobileMap: [GSO - UHE-SALTO SANTIAGO] 0d3e29ea-8af4-4d1a-b6d2-ca4fe3fd8cb6 - Rota de fuga>]>

In [7]:
RecordPanelShowWebMap.objects.filter(panel__in=rp_ss,user=user)

<QuerySet [<RecordPanelShowWebMap: [GSO - UHE-SALTO SANTIAGO] 49447cb7-6f4e-4b28-aa11-510d198a96a0 - ZAS>, <RecordPanelShowWebMap: [GSO - UHE-SALTO SANTIAGO] e2d08e08-abe8-4dfa-934c-5712b3ccae29 - Ponto de encontro>, <RecordPanelShowWebMap: [GSO - UHE-SALTO SANTIAGO] 8d778110-dad4-4921-a9af-adefc2445f83 - ZSS>, <RecordPanelShowWebMap: [GSO - UHE-SALTO SANTIAGO] d3a480c3-6d61-4b9c-85d6-993ca58ededb - Alerta sonoro>, <RecordPanelShowWebMap: [GSO - UHE-SALTO SANTIAGO] 44e0a2a0-de59-4e8c-8393-de83fe6e5260 - Rota de fuga>]>

In [14]:
for uic in UserInCompany.objects.filter(company__name__startswith='GSO'):
    rp_ss=RecordPanel.objects.filter(company=uic.company)
    for a in rp_ss:
        if not RecordPanelShowMobileMap.objects.filter(panel=a,
                                 user=user).exists():
            rpsm=RecordPanelShowMobileMap(panel=a,
                                     user=user)
#             rpsm.save()
        if not RecordPanelShowWebMap.objects.filter(panel=a,
                                 user=user).exists():
            rpsw=RecordPanelShowWebMap(panel=a,
                                 user=user)
#             rpsw.save()
    

In [1]:
occ=OccurrenceType.objects.get(uuid='faa62cbd-c2ec-49fc-bc4a-9b1ab9edcca4')

In [4]:
for a in occ.form_fields['fields']:
    print(a['display_name'],a['data_type'],a['api_name'])
    if a['data_type'] == 'arrayOfObjects':
        for b in a['inner_fields']:
            print('\t',b['display_name'],b['data_type'],b['api_name'])

 arrayOfObjects irregularities
	 Identificação do local select placeIdentification
	 Constatação select finding
	 Constatação (detalhe) string unauthorizedConstruction
	 Irregularidades string unauthorizedConstructionIrregular
	 Outros - Descrição string otherDescription
	 Comprimento (m) boolean displayLength
	 Comprimento (m) float length
	 Largura (m) boolean displayWidth
	 Largura (m) float width
	 Altura (m) boolean displayHeight
	 Altura (m) float height
	 Área (m2) boolean displayArea
	 Área (m2) float area
	 Quantidade (un) boolean displayAmount
	 Quantidade (un) float amount
	 Volume (m3) boolean displayVolume
	 Volume (m3) float volume
	 Profundidade (m) boolean displayDeepen
	 Profundidade (m) float deepen
	 Perímetro (m) boolean displayPerimeter
	 Perimeter (m) float perimeter
	 Diâmetro (m) boolean displayDiameter
	 Diâmetro (m) float diameter
	 Cota (m) boolean displayQuote
	 Cota (m) float quote
	 Peso (Kg) boolean displayKg
	 Peso (Kg) float weight
Área afetada (m2) str

In [3]:
a

{'id': 11,
 'api_name': 'irregularities',
 'data_type': 'arrayOfObjects',
 'display_name': '',
 'inner_fields': [{'order': 1,
   'api_name': 'placeIdentification',
   'print_if': {'!!': {'var': {'cat': ['data.formData.irregularities',
       {'var': 'index.0'},
       '.placeIdentification']}}},
   'data_type': 'select',
   'display_name': 'Identificação do local',
   'select_options': {'options': [{'name': 'Área de preservação permanente',
      'value': '1'},
     {'name': 'Área remanescente', 'value': '2'},
     {'name': 'Área adquirida', 'value': '3'},
     {'name': 'Área de inundação', 'value': '4'},
     {'name': 'Área industrial', 'value': '5'}]}},
  {'order': 2,
   'api_name': 'finding',
   'print_if': {'!!': {'var': {'cat': ['data.formData.irregularities',
       {'var': 'index.0'},
       '.finding']}}},
   'data_type': 'select',
   'display_name': 'Constatação',
   'select_options': {'options': [{'name': 'Construção não autorizada',
      'value': '1'},
     {'name': 'Ocupaç

In [6]:
from django.contrib.admin.utils import flatten
from tqdm.notebook import tqdm

In [7]:
def remove_duplicates(x):
    return ', '.join(list(dict.fromkeys(x)))

In [8]:
def get_value_inner(field, inner_field, occ, value):

    values = occ.form_fields['fields']
    try:
        item_translation = flatten([item['innerFields'] for item in values if item['apiName']==field])
    except:
        item_translation = flatten([item['inner_fields'] for item in values if item['api_name']==field])
#     print(item_translation)
    try:
        mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == inner_field])
    except:
        mid_translation = flatten([item['select_options']['options'] for item in item_translation if item['display_name'] == inner_field])
#     print(mid_translation)
    final_translation = {item['value']: item['name'] for item in mid_translation}
#     print(final_translation)
    if type(value) is str:
        return final_translation[value]
    elif type(value) is list:
        return ', '.join(sorted(list([get_value_inner(field, inner_field, occ, a) for a in value])))

In [9]:
occurrences=OccurrenceRecord.objects.filter(search_tags__name="Patrimonial").distinct().order_by('company__name').exclude(Q(company__name='ATIVO-TREINAMENTO')|Q(company__name='UHE-TESTE')|Q(company__name='UHE-AAPL')|Q(company__name__icontains='POC')|Q(company__name='TESTE STORES')).prefetch_related('company', 'created_by', 'firm', 'river', 'city', 'location', 'occurrence_type')
occurrences.count()

10112

In [10]:
for a in tqdm(occurrences):
    r=remove_duplicates(sorted(list(get_value_inner('irregularities', 'Identificação do local', occ, ''.join(item['place_identification'])) for item in a.form_data['irregularities'] if item and 'place_identification' in item))) if 'irregularities' in a.form_data else None
    joined=', '.join(sorted(list(item['unauthorized_construction'] for item in a.form_data['irregularities'] if item and 'unauthorized_construction' in item))) if 'irregularities' in a.form_data else None
    print(r,joined)

KeyboardInterrupt: 

In [4]:
data=QueuedEmail.objects.all().values('title').annotate(qtd_email_tipo=Count('uuid')).order_by('-qtd_email_tipo').distinct()

In [7]:
import pandas as pd
df=pd.DataFrame(data=data)

In [12]:
df[:50].to_excel('dadosEmailEngie.xlsx')

In [18]:
df2=df[:50].index+1

In [19]:
df2

RangeIndex(start=1, stop=51, step=1)

In [1]:
company=Company.objects.get(name='GES - GEH')
company

<Company: 68e3af24-2fec-4ab0-b2ff-97e1076b02b3: GES - GEH>

In [20]:
aps = ApprovalStep.objects.filter(approval_flow__company=company,approval_flow__target_model__icontains='measurement')
[(i,a.name) for i,a in enumerate(aps)]

[(0, 'Aguardando Aprovação do Terceiro'),
 (1, 'Aguardando Aprovação do Preposto da Contratante'),
 (2, 'Aprovado')]

In [8]:
con = Contract.objects.get(uuid='9f6a440a-06e1-4250-bb41-a79dfb2fb6a1')

In [9]:
mb = MeasurementBulletin.objects.get(number='GEH-BM-2024.0020',contract=con,approval_step__approval_flow__company=company)
mb

<MeasurementBulletin: GEH-BM-2024.0020>

In [22]:
mb.approval_step = aps[0]
mb.editable = True
mb.save()

In [21]:
aps[0].name

'Aguardando Aprovação do Terceiro'

In [24]:
[a.__dict__ for a in ApprovalTransition.objects.filter(origin__approval_flow__company=company,origin__approval_flow__target_model__icontains='measurement')]

[{'_state': <django.db.models.base.ModelState at 0x75141c95c6a0>,
  'uuid': UUID('84a4e68b-ff64-44a7-83fd-18970b8a8e0f'),
  'name': 'Aprovar',
  'origin_id': UUID('b9d170ec-c486-4aaa-bc52-0fde70900ef7'),
  'destination_id': UUID('967dbff6-dac4-441b-a9de-484db43d21c8'),
  'condition': {'===': [{'var': 'request.action'}, 'Aprovar']},
  'callback': {'change_fields': [{'name': 'editable', 'value': False}],
   'send_notification': ['approved_measurement_bulletin'],
   'notification_subject': 'Boletim de medição aprovado pelo fornecedor',
   'notification_description': 'Um boletim de medição foi aprovado pelo fornecedor, e agora aguarda aprovação do preposto da contratante.',
   'measurement_bulletin_notification_firms': ['9f5e767f-0784-4e77-b583-14ccc10361de']},
  'button': {'icon': 'check', 'color': 'primary', 'variant': 'raised'},
  'order': ''},
 {'_state': <django.db.models.base.ModelState at 0x75141a4f0b20>,
  'uuid': UUID('71f0d4a3-fa53-4d71-bfd7-5e661a1e9dee'),
  'name': 'Aprovar',
 

In [2]:
!echo $DB_HOST

engie-production.cnsey7np045b.sa-east-1.rds.amazonaws.com


In [3]:
!cat .env

STAGE=ENGIE_STAGING


In [1]:
import json
for a in ApprovalTransition.objects.all():
    condition = json.dumps(a.condition)
    callback = json.dumps(a.callback)
    if 'ToDoAction' in condition:
        print('condition',condition)
    if 'ToDoAction' in callback:
        print('callback',callback)

In [1]:
reg=['UHMI-RG-2024.0033',
'UHMI-RG-2024.0034',
'UHMI-RG-2024.0035',
'UHSO-RG-2024.1781',]

In [10]:
OccurrenceRecord.objects.get(number='UHMI-RG-2024.0029').__dict__

{'_state': <django.db.models.base.ModelState at 0x785ab8afca30>,
 'uuid': UUID('2537b6af-3290-452d-bf56-e5a242a84835'),
 'datetime': datetime.datetime(2024, 3, 22, 13, 30, 33, 969000, tzinfo=<UTC>),
 'number': 'UHMI-RG-2024.0029',
 'created_at': datetime.datetime(2024, 3, 22, 13, 50, 47, 956967, tzinfo=<UTC>),
 'updated_at': datetime.datetime(2024, 4, 2, 17, 56, 50, 857456, tzinfo=<UTC>),
 'reviews': 0,
 'company_id': UUID('2fd0ca76-b741-47d7-af9c-607e6ed1eca2'),
 'editable': False,
 'is_approved': True,
 'uf_code': 31,
 'city_id': UUID('b3b6ec63-d544-468a-ac96-2e1c5d4c95ff'),
 'location_id': UUID('04decc09-3a77-4740-9ac2-f794633c56f0'),
 'place_on_dam': '9',
 'river_id': None,
 'point': None,
 'geometry': <GeometryCollection object at 0x785ab8b7fd90>,
 'properties': [{'label': 'UHMI-079AD  Aline Pioli de Moura'}],
 'distance_from_dam': 0.0,
 'other_reference': 'Cota de desapropriação 696,00m',
 'origin': '5',
 'origin_media': None,
 'informer': {},
 'territorial_administration': '2',


In [ ]:
ApprovalStep.objects.get()

In [ ]:
OccurrenceRecord.objects.all():

In [15]:
from tqdm.notebook import tqdm
for a in tqdm(Company.objects.filter(name__startswith="GES")):
    try:
        print(a.name)
        aps=ApprovalStep.objects.get(name='Registro Deferido',approval_flow__company=a,approval_flow__target_model__icontains='occurrence')
        ocrs=OccurrenceRecord.objects.filter(company=a,approval_step=aps,is_approved=False).prefetch_related('company','approval_step')
        print(ocrs.count())
        if ocrs.count() != 0:
            ocrs.update(is_approved=True)
    except:
        pass

  0%|          | 0/14 [00:00<?, ?it/s]

GES - UHE-CANA BRAVA
2
GES - UHE-ESTREITO
0
GES - PCH-RONDONÓPOLIS
GES - UHE-MACHADINHO
0
GES - UHE-SALTO SANTIAGO
0
GES - PCH-JOSÉ GELÁZIO DA ROCHA
GES - UHE-SÃO SALVADOR
0
GES - GEH
GES - UHE-ITA
0
GES - UHE-PASSO FUNDO
0
GES - UHE-SALTO OSORIO
0
GES - UHE-JAGUARA
0
GES - UHE-MIRANDA
14
GES - UHE-PONTE DE PEDRA
0


In [2]:
ServiceOrder.objects.filter(number__in=['UHMI-OS-2024.0010','UHMI-OS-2024.0009']).delete()

(31,
 {'service_orders.ServiceOrder_responsibles': 3,
  'service_orders.ServiceOrder_managers': 3,
  'service_orders.ServiceOrderAction': 21,
  'service_orders.Procedure': 2,
  'service_orders.ServiceOrder': 2})

In [2]:
lista=['UHCB-RG-2023.0267',
'UHCB-RG-2023.0452',
'UHCB-RG-2023.0432',
'UHCB-RG-2023.0397',
'UHCB-RG-2023.0455',
'UHCB-RG-2023.0334',
'UHCB-RG-2023.0407',
'UHCB-RG-2023.0270',
'UHCB-RG-2023.0454',
'UHCB-RG-2023.0450',
'UHCB-RG-2020.0002']
len(lista)

11

In [4]:
OccurrenceRecord.objects.filter(number__in=lista).delete()

(71,
 {'occurrence_records.OccurrenceRecord_search_tags': 30,
  'occurrence_records.OccurrenceRecord_service_orders': 10,
  'occurrence_records.OccurrenceRecord_active_shape_files': 19,
  'files.File': 1,
  'occurrence_records.OccurrenceRecord': 11})

In [11]:
ServiceOrderActionStatus.history.filter(uuid='4f89dd8b-dbe9-4cae-be66-ea7ffa2af61b',
                                        history_type='-').order_by('-history_date')[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x7bbb20fa1a60>,
 'uuid': UUID('4f89dd8b-dbe9-4cae-be66-ea7ffa2af61b'),
 'kind': 'OCCURRENCE_RECORD_STATUS',
 'name': 'Registro Deferido',
 'metadata': {'display_in_app': False, 'display_in_front': False},
 'is_final': False,
 'history_id': UUID('73a73db4-cbae-4e6c-89fd-358b0316dd2a'),
 'history_date': datetime.datetime(2024, 5, 2, 15, 45, 11, 313193, tzinfo=<UTC>),
 'history_change_reason': None,
 'history_type': '-',
 'history_user_id': UUID('0aa50773-b368-4a50-9f12-4a7d8dfaf256')}

In [13]:
ApprovalStep.objects.filter(approval_flow__company__name='PRE-GES - UHE-ESTREITO')

<QuerySet [<ApprovalStep: [PRE-GES - UHE-ESTREITO]: Aguardando Homologação>, <ApprovalStep: [PRE-GES - UHE-ESTREITO]: Registro Deferido>, <ApprovalStep: [PRE-GES - UHE-ESTREITO]: Registro Indeferido>, <ApprovalStep: [PRE-GES - UHE-ESTREITO]: Registro em Elaboração>, <ApprovalStep: [PRE-GES - UHE-ESTREITO]: Necessita Revisão>, <ApprovalStep: [PRE-GES - UHE-ESTREITO]: Atendido>, <ApprovalStep: [PRE-GES - UHE-ESTREITO]: Aguardando aprovação técnica>, <ApprovalStep: [PRE-GES - UHE-ESTREITO]: Aguardando Aprovação do Usuário-Chave do Sistema>, <ApprovalStep: [PRE-GES - UHE-ESTREITO]: Aguardando Aprovação do Usuário-Chave>, <ApprovalStep: [PRE-GES - UHE-ESTREITO]: Negado>, <ApprovalStep: [PRE-GES - UHE-ESTREITO]: Aguardando Aprovação do Supervisor>, <ApprovalStep: [PRE-GES - UHE-ESTREITO]: Aprovado>, <ApprovalStep: [PRE-GES - UHE-ESTREITO]: Aguardando Aprovação do Terceiro>, <ApprovalStep: [PRE-GES - UHE-ESTREITO]: Aguardando Aprovação do Preposto ENGIE>, <ApprovalStep: [PRE-GES - UHE-ESTREIT

In [15]:
aps=ApprovalStep.objects.get(uuid='4f7cc453-851a-4700-9510-f682414f198b')
aps

<ApprovalStep: [PRE-GES - UHE-ITA]: Registro em elaboração>

In [16]:
OccurrenceRecord.objects.filter(uuid='c09251e0-b230-40d9-8eaf-4fe75c50367c').update(approval_step=aps)

1

In [1]:
from django.db.models import Q
from tqdm.notebook import tqdm

In [38]:
up=UserPermission.objects.filter(Q(permissions__has_key='service_order')|Q(permissions__has_key='ServiceOrder'),companies__name__startswith='GES').distinct()

In [39]:
up.count()

15

In [40]:
for a in tqdm(up):
    if a.permissions.get('service_order') and a.permissions.get('service_order').get('can_create') == True:
        print(a.name)
    if a.permissions.get('serviceOrder') and a.permissions.get('serviceOrder').get('can_create') == True:
        print(a.name)
    if a.permissions.get('ServiceOrder') and a.permissions.get('ServiceOrder').get('can_create') == True:
        print(a.name)

  0%|          | 0/15 [00:00<?, ?it/s]

GES - LIDER TÉCNICO EXTERNO
GES - LÍDER TÉCNICO E COMERCIAL EXTERNO DE MONITORAMENTOS
GES - LIDER COMERCIAL EXTERNO
GES - LÍDER TÉCNICO E COMERCIAL CONSÓRCIO
GES - HOMOLOGADOR
GES - ADMINISTRADOR
GES - LÍDER TÉCNICO E COMERCIAL EXTERNO


In [32]:
UserPermission.objects.filter(name__icontains='administrador',companies__name__startswith='GES').distinct().first().permissions

{'City': {'can_edit': True,
  'can_view': True,
  'queryset': 'all',
  'can_create': True,
  'can_delete': True},
 'Firm': {'can_edit': True,
  'can_view': True,
  'queryset': 'all',
  'can_create': True,
  'can_delete': True},
 'ToDo': {'can_edit': True,
  'can_view': True,
  'queryset': 'self',
  'can_create': False,
  'can_delete': False},
 'User': {'can_edit': True,
  'can_view': True,
  'queryset': 'all',
  'can_create': True,
  'can_delete': True,
  'can_use_rh_api': True,
  'can_edit_all_fields': True},
 'River': {'can_edit': True,
  'can_view': True,
  'queryset': 'all',
  'can_create': True,
  'can_delete': True},
 'Entity': {'can_edit': True,
  'can_view': True,
  'queryset': 'all',
  'can_create': True,
  'can_delete': True},
 'Company': {'can_edit': True,
  'can_view': True,
  'queryset': 'self',
  'can_create': True,
  'can_delete': True,
  'can_change_custom_options': True},
 'Contract': {'can_edit': True,
  'can_view': True,
  'queryset': 'all',
  'can_create': True,
  '

In [2]:
from openpyxl import load_workbook

wb = load_workbook(filename='registros_nao_encontrados.xlsx')
sheet = wb.active
data = []

headers = [cell.value for cell in sheet[1]]

for row in sheet.iter_rows(min_row=2):
    row_dict = {}
    for key, cell in zip(headers, row):
        row_dict[key] = cell.value

    data.append(row_dict)

In [5]:
for a in data:
    ocr = OccurrenceRecord.objects.filter(number=a['registros_nao_encontrados'])
    if ocr.exists():
        pass
    else:
        print(a)

In [1]:
!cat .env

STAGE=ENGIE_PRODUCTION


In [3]:
import pandas as pd

In [4]:
data3=RecordPanel.objects.all().exclude(
    Q(company__name__icontains='demo')|
    Q(company__name__icontains='teste')|
    Q(company__name__icontains='old')
).values('company__name','name').order_by('company__name')

In [5]:
data3

<QuerySet [{'company__name': 'CCR - Rio Sp', 'name': 'Monitoração Vertical'}, {'company__name': 'CCR - Rio Sp', 'name': 'Monit. Sinalização Vertical'}, {'company__name': 'CCR - Rio Sp', 'name': 'OAC ANTT'}, {'company__name': 'CCR - Rio Sp', 'name': 'Monitoração Edificações 2023'}, {'company__name': 'CCR - Rio Sp', 'name': 'Drenagem'}, {'company__name': 'CCR - Rio Sp', 'name': 'Painel teste'}, {'company__name': 'CCR - Rio Sp', 'name': 'ITS'}, {'company__name': 'CCR - RodoAnel', 'name': 'Monitoração Drenagem superficial 2024'}, {'company__name': 'CCR - SPVias', 'name': 'MF 01'}, {'company__name': 'CCR - SPVias', 'name': 'MF 02'}, {'company__name': 'CCR - SPVias', 'name': 'PRE 02'}, {'company__name': 'CCR - SPVias', 'name': 'PRE 03'}, {'company__name': 'CCR - SPVias', 'name': 'SPT 02'}, {'company__name': 'CCR - SPVias', 'name': 'CP 03'}, {'company__name': 'CCR - SPVias', 'name': 'MF 03'}, {'company__name': 'CCR - SPVias', 'name': 'CM 01'}, {'company__name': 'CCR - SPVias', 'name': 'RM 19'

In [6]:
df2= pd.DataFrame(data=data3,columns=['company__name',"name"])

In [7]:
df2

,company__name,name
0,CCR - Rio Sp,Monitoração Vertical
1,CCR - Rio Sp,Monit. Sinalização Vertical
2,CCR - Rio Sp,OAC ANTT
3,CCR - Rio Sp,Monitoração Edificações 2023
4,CCR - Rio Sp,Drenagem
5,CCR - Rio Sp,Painel teste
6,CCR - Rio Sp,ITS
7,CCR - RodoAnel,Monitoração Drenagem superficial 2024
8,CCR - SPVias,MF 01
9,CCR - SPVias,MF 02


In [8]:
df4=df2.groupby('company__name')["name"].count()

In [9]:
df4

company__name
CCR - Rio Sp       7
CCR - RodoAnel     1
CCR - SPVias      11
Name: name, dtype: int64

In [10]:
writer = pd.ExcelWriter('unidadesRecordPanelCCR.xlsx', engine='xlsxwriter')

In [11]:
df2.to_excel(writer, sheet_name='Unidades-RecordPanelName')
df4.to_excel(writer, sheet_name='Total Por Unidade')

In [12]:
writer.close()